<a href="https://colab.research.google.com/github/jon-chun/sentimenttime/blob/main/sentimenttime_part1_lexrules_20210708_ver5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **An Analytic Methodology to Extract Narratives from Text: Using Sentiment Analysis to find the Arcs and Crux Points in Novels, Social Media and Chat Transcripts**

By: Jon Chun
12 Jun 2021

References:

* Coming...

TODO:
* Filter out non-printable characters
* Roll-over Crux-Points (SentNo+Sent/Parag) (plotly)
* Label/Roll-over Chapter/Sect No at Boundries
* Generate Report PDF/csv
* Option to select raw or discrete2continous transformation (Bing)



# Configuration (Auto)

**Global Configuration Constants**

In [1]:
# Hardcoded Sentiment Analysis Models

MODELS_LS = ['vader','textblob','stanza','afinn','bing','sentimentr','syuzhet','pattern','sentiword','senticnet','nrc']
            
# Minimum lengths for Sentences and Paragraphs
#   (Shorter Sents/Parags will be deleted)

MIN_CHAP_LEN = 5000
MIN_SECT_LEN = 5000  # Minimum char length to be included in section DataFrame
MIN_PARAG_LEN = 2
MIN_SENT_LEN = 2

# Min/Max statistics on each lexicon's sentiment values applied to corpus
corpus_lexicons_stats_dt = {}
corpus_cruxes_dt = {}


**Install Libraries**

In [2]:
# INSTALL LIBRARIES

!pip install sklearn

In [3]:
%pip install contractions

     |████████████████████████████████| 327kB 5.1MB/s 
     |████████████████████████████████| 286kB 51.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85392 sha256=08b1ea4c154473b83234fce028efc9573132ef494eda6e1fbf9314573a1dcf14
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


**Import Libraries**

In [4]:
import os
import sys
import io
import glob
import json
import contextlib

In [5]:
# IMPORT LIBRARIES

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import re
import string

In [7]:
import collections
from collections import OrderedDict

In [8]:
# Import libraries for logging

import logging
from datetime import datetime
import time                     # (TODO: check no dependencies and delete)
from time import gmtime, strftime

In [9]:
import nltk

# Download for sentence tokenization
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# Download for nltk/VADER sentiment analysis
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
from sklearn.preprocessing import MinMaxScaler   # To normalize time series
from sklearn.preprocessing import StandardScaler # To Standardize time series: center(sub mean) and rescale within 1 SD (only for well-behaved guassian distributions)
from sklearn.preprocessing import RobustScaler   # To Standardize time series: center(sub median) and rescale within 25%-75% (1st-3rd) IQR (better for noisy, outliers distributions)

In [11]:
mean_std_scaler = StandardScaler()
median_iqr_scaler = RobustScaler()

In [12]:
# Zoom interpolates new datapoints between existing datapoints to expand a time series 

from scipy.ndimage.interpolation import zoom

In [13]:
from scipy import interpolate
from scipy.interpolate import CubicSpline
from scipy import signal
from scipy.signal import argrelextrema

In [14]:
from statsmodels.nonparametric.smoothers_lowess import lowess as sm_lowess
from statsmodels import robust

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**Define Library-Dependent Objects**

In [15]:
import contractions

In [16]:
corpus_sects_df = pd.DataFrame()

**Configure Jupyter Notebook**

In [17]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [18]:
# Configure matplotlib and seaborn

# Plotting pretty figures and avoid blurry images
# %config InlineBackend.figure_format = 'retina'
# Larger scale for plots in notebooks
# sns.set_context('talk')

%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 8]
plt.rcParams['figure.dpi'] = 100

In [19]:
# Configure Jupyter

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from ipywidgets import widgets, interactive

# Configure Google Colab

%load_ext google.colab.data_table

In [20]:
# Text wrap

from IPython.display import HTML

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

**Configuration Details Snapshot**

In [21]:
# Snap Shot of Time, Machine, Data and Library/Version Blueprint
# TODO:

# Pick ONE Method (a) or (b) to Get Corpus Textfile

**Choose either (a) OR (b), not both**

## **Option (a): Connect to Google gDrive**

In [22]:
# Connect to Google gDrive

from google.colab import drive, files
drive.mount('/gdrive')
%cd /gdrive/MyDrive/

Mounted at /gdrive
/gdrive/MyDrive


In [23]:
# Select the Corpus subdirectory on your Google gDrive

gdrive_subdir = "./research/2021/sa_book_code/books_sa/imcewan_machineslikeme" #@param {type:"string"}
CORPUS_SUBDIR = gdrive_subdir
corpus_filename = ''
%cd $gdrive_subdir


/gdrive/MyDrive/research/2021/sa_book_code/books_sa/imcewan_machineslikeme


## **Option (b): Upload Corpus Textfile**

***Only do this if your Google subdirectory doesn't already contain a plain text file of your Corpus or you wish to overwrite it and use a newer version***

In [ ]:
# Execute this code cell to upload plain text file of corpus
#   Should be *.txt format with paragraphs separated by at least 2 newlines

uploaded = files.upload()

In [ ]:
# Verify file was uploaded

# Get uploaded filename
corpus_filename = list(uploaded.keys())[0]
print(f'Uploaded Corpus filename is: {corpus_filename}')
CORPUS_FILENAME = corpus_filename

!ls -al $corpus_filename

# **Configuration (Manual)**

In [24]:
# Verify subdirectory change

!pwd
!ls *.txt

# TODO: Intelligently automate the filling of form based upon directory

/gdrive/MyDrive/research/2021/sa_book_code/books_sa/imcewan_machineslikeme
'fsfitzgerald_thegreatgatsby (1).txt'
 fsfitzgerald_thegreatgatsby.txt
 imcewan_machineslikeme_sents.txt
 machines_like_me_all.txt
 machines_like_me_clean_final_hand.txt
 machines_like_me_clean.txt
 mlm_final_handclean_nosections.txt
 mlm_final_handclean_sections.txt
 mlm_final_hand.txt
 mlm_sentences.txt
 report_sentiment_cruxes_ianmcewan_machineslikeme_XLNet_SST_20210622_1232.txt


In [25]:
CORPUS_TITLE = 'Machines Like Me' #@param {type:"string"}
CORPUS_AUTHOR = "Ian McEwan" #@param {type:"string"}
CORPUS_FILENAME = "mlm_final_handclean_sections.txt" #@param {type:"string"}
CORPUS_SUBDIR = "./research/2021/sa_book_code/books_sa/imcewan_machineslikeme" #@param {type:"string"}

CORPUS_FULL = f'{CORPUS_TITLE} by: {CORPUS_AUTHOR}'

PLOT_OUTPUT = "Major" #@param ["None", "Major", "All"]

FILE_OUTPUT = "Major" #@param ["None", "Major", "All"]

gdrive_subdir = CORPUS_SUBDIR
corpus_filename = ''
author_str = ''.join(CORPUS_AUTHOR.split()).lower()
title_str = ''.join(CORPUS_TITLE.split()).lower()

print(f'\nWorking Corpus Datafile: {CORPUS_SUBDIR}')
print(f'\nFull Corpus Title/Author: {CORPUS_FULL}')

# Verify contents of Corpus File is Correctly Formatted
#   
# TODO: ./utils/verify_format.py



Working Corpus Datafile: ./research/2021/sa_book_code/books_sa/imcewan_machineslikeme

Full Corpus Title/Author: Machines Like Me by: Ian McEwan


In [26]:
FILE_OUTPUT="Major"

# **Utility Functions (Auto)**

## **Files**

In [27]:
# Generate full path and timestamp for new filepath/filename

def gen_pathfiletime(file_str, subdir_str=''):

  # Geenreate compressed author and title substrings
  author_raw_str = ''.join(CORPUS_AUTHOR.split()).lower()
  title_raw_str = ''.join(CORPUS_TITLE.split()).lower()

  # Generate current/unique datetime string
  datetime_str = str(datetime.now().strftime('%Y%m%d%H%M%S'))

  # Built fullpath+filename string
  file_base, file_ext = file_str.split('.')

  author_str = re.sub('[^A-Za-z0-9]+', '', author_raw_str)
  title_str = re.sub('[^A-Za-z0-9]+', '', title_raw_str)

  full_filepath_str = f'{subdir_str}{file_base}_{author_str}_{title_str}_{datetime_str}.{file_ext}'

  # print(f'Returning from gen_savepath() with full_filepath={full_filepath}')

  return full_filepath_str

# Test
# pathfilename_str = gen_pathfiletime('hist_paraglen.png')
# print(pathfilename_str)

In [28]:
# Tokenize into Sentences

def parag2sents(corpus_parags_ls):
  '''
  Given a list of paragraphs,
  Return a list of lists of Sentences [sent_no, parag_no, asent(text)]
  '''

  sent_no = 0
  # sent_base = 0
  corpus_sents_row_ls = []
  for parag_no,aparag in enumerate(corpus_parags_ls):
    sents_ls = sent_tokenize(aparag)
    # Delete (whitespace only) sentences
    sents_ls = [x.strip() for x in sents_ls if len(x.strip()) > MIN_SENT_LEN]
    # Delete (punctuation only) sentences
    sents_ls = [x for x in sents_ls if len((re.sub(r'[^\w\s]','',x)).strip()) > MIN_SENT_LEN]
    # Delete numbers (int or float) sentences
    sents_ls = [x for x in sents_ls if not (x.strip().isnumeric())]
    # TODO: may want to keep
    for s,asent in enumerate(sents_ls):
      corpus_sents_row_ls.append([sent_no, parag_no, asent])
      sent_no += 1

    # print(f'Returning with corpus_sents_row_ls length = {len(corpus_sents_row_ls)}')
  
  return corpus_sents_row_ls

# Test

'''
print(f'Length {len(corpus_parags_raw_ls)}')
corpus_sents_row_ls = parag2sents(corpus_parags_raw_ls)

print(f'First row {corpus_sents_row_ls[0]}')
print('\n')
print(f'Last row {corpus_sents_row_ls[-1]}')
''';

In [29]:
#This function converts to lower-case, removes square bracket, removes numbers and punctuation
 
def text_clean(text):
    text = text.lower()
    text = contractions.fix(text)  # Expand contrations
    text = re.sub("\\'s", " own", text)  # After expanding normal apostrophes, expand possessive apostrophes "Mary's car" -> "Mary own car"

    # TODO: More formally
    # https://towardsdatascience.com/nlp-building-text-cleanup-and-preprocessing-pipeline-eba4095245a0
    text = re.sub("-\n", "", text)       # Join end of line words split by continuation hyphens
    text = re.sub("-\n\r", "", text)
    text = re.sub("-\r", "", text)
    text = re.sub("\[.*?\]", " ", text)

    text = re.sub("-", " ", text)  # Special care for hypenated words well-known: choose option (a)
                                   # (a) 'well known', (b) 'wellknown' (c) 'well known' and 'wellknown' cf: https://datascience.stackexchange.com/questions/81072/how-to-process-the-hyphenated-english-words-for-any-nlp-problem
    text = re.sub("/", " ", text)  # sociability/conversation/interesting -> sociability conversation interesting                             
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\w*\d\w*", "", text)
    text = re.sub("[\n]", " ", text)  # Replace newline with space
    return text

In [30]:
# Read corpus into a single string then split into sections (chapters and subchapter)

'''
if len(corpus_filename) == 0:
  # If now file uploaded, use the file in Google gDrive
  corpus_filename = CORPUS_FILENAME
else:
  # The uploaded file has priority over the gDrive Corpus file
  pass
'''

def corpus2sects(corpus_filename):
  '''
  Given a corpus_filename (assuming already %cd into correct subdir)
  Return a list of min preprocessed raw sections/CHAPTERs (corpus_parags_raw_temp_ls)
  '''

  with open(corpus_filename, "r", encoding='windows-1252') as infp:
    corpus_raw_str = infp.read()

  # print(f'len(corpus_raw_str) = {len(corpus_raw_str)}')
  corpus_sects_ls = re.split(r'(CHAPTER [\d]{1,2}|-----)', corpus_raw_str) # , flags=re.I)

  # Filter out the Section separator '-----' lines
  corpus_sects_noseclines_ls = [x for x in corpus_sects_ls if not (x.strip().startswith('-----'))]
  # Filter out the CHAPTER lines
  corpus_sects_nochheads_ls = [x for x in corpus_sects_noseclines_ls if not (x.strip().startswith('CHAPTER '))]

  return corpus_sects_nochheads_ls

'''
  corpus_parags_raw_temp_ls = corpus_raw_str.split('\n\n')
  print(f'Corpus Paragraph Raw Count: {len(corpus_parags_raw_temp_ls)}')

  # Strip excess whitespace and drop empty lines
  corpus_parags_raw_temp_ls = [x.strip() for x in corpus_parags_raw_temp_ls if len(x.strip()) > MIN_PARAG_LEN]
  print(f'Corpus Paragraph -(whitespace only) Count: {len(corpus_parags_raw_temp_ls)}')

  # Drop lines that only contain punctuation (e.g. '"', '.', '...', etc)
  corpus_parags_raw_temp_ls = [x for x in corpus_parags_raw_temp_ls if len((re.sub(r'[^\w\s]','',x)).strip()) > MIN_PARAG_LEN]
  print(f'Corpus Paragraph -(punctuation only) Count: {len(corpus_parags_raw_temp_ls)}')

  return corpus_parags_raw_temp_ls

# Test  

corpus_parags_raw_ls = read_corpus_parags(CORPUS_FILENAME)
print(f'We found #{len(corpus_parags_raw_ls)} lines\n')

print('\nThe first 10 lines of the Corpus:')
print('-----------------------------------\n')
corpus_parags_raw_ls[:10]

print('\nThe last 10 lines of the Corpus:')
print('-----------------------------------\n')
corpus_parags_raw_ls[-10:]
print('\n')
print(sorted(corpus_parags_raw_ls, key=lambda x: (len(x), x)))
''';

In [31]:
def corpus2chaps(corpus_filename):
  '''
  Given a corpus_filename (assuming already %cd into correct subdir)
  Return a list of min preprocessed raw CHAPTERs (corpus_parags_raw_temp_ls)
  '''

  with open(corpus_filename, "r", encoding='windows-1252') as infp:
    corpus_raw_str = infp.read()

  # print(f'len(corpus_raw_str) = {len(corpus_raw_str)}')
  corpus_chaps_ls = re.split(r'(CHAPTER [\d]{1,2})', corpus_raw_str, flags=re.I) # , flags=re.I)

  corpus_chaps_nochheads_ls = [x for x in corpus_chaps_ls if not (x.strip().startswith('CHAPTER '))]

  return corpus_chaps_nochheads_ls

In [32]:
CORPUS_FILENAME

'mlm_final_handclean_sections.txt'

In [33]:
!pwd

/gdrive/MyDrive/research/2021/sa_book_code/books_sa/imcewan_machineslikeme


In [34]:
!ls -altr *.txt

-rw------- 1 root root 531671 Jun  5 20:10  machines_like_me_all.txt
-rw------- 1 root root 512517 Jun  5 20:10  machines_like_me_clean.txt
-rw------- 1 root root 513945 Jun  5 21:02  machines_like_me_clean_final_hand.txt
-rw------- 1 root root 486767 Jun  5 22:02  mlm_sentences.txt
-rw------- 1 root root 486767 Jun  5 22:02  imcewan_machineslikeme_sents.txt
-rw------- 1 root root 268684 Jun 16 13:52 'fsfitzgerald_thegreatgatsby (1).txt'
-rw------- 1 root root 274935 Jun 16 14:52  fsfitzgerald_thegreatgatsby.txt
-rw------- 1 root root    438 Jun 22 12:32  report_sentiment_cruxes_ianmcewan_machineslikeme_XLNet_SST_20210622_1232.txt
-rw------- 1 root root 513921 Jun 27 18:18  mlm_final_hand.txt
-rw------- 1 root root 513921 Jun 28 15:43  mlm_final_handclean_nosections.txt
-rw------- 1 root root 514194 Jun 28 15:44  mlm_final_handclean_sections.txt


In [35]:
!head -n 10 mlm_final_handclean_sections.txt

But remember, please, the Law by which we live, We are not built to comprehend a lie ...

--Rudyard Kipling, "The Secret of the Machines"

CHAPTER 1

It was religious yearning granted hope, it was the holy grail of science. Our ambitions ran high and low--for a creation myth made real, for a monstrous act of self-love. As soon as it was feasible, we had no choice but to follow our desires and hang the consequences. In loftiest terms, we aimed to escape our mortality, confront or even replace the Godhead with a perfect self. More practically, we intended to devise an improved, more modern version of ourselves and exult in the joy of invention, the thrill of mastery. In the autumn of the twentieth century, it came about at last, the first step towards the fulfilment of an ancient dream, the beginning of the long lesson we would teach ourselves that however complicated we were, however faulty and difficult to describe in even our simplest actions and modes of being, we could be imitated a

In [36]:
sections_ls = corpus2sects('./mlm_final_handclean_sections.txt') # (CORPUS_FILENAME)
len(sections_ls)
# r1 = re.findall(r'^CHAPTER 1$' ,corpus_raw_str)
# r1 = re.findall(r'-----',corpus_raw_str)
# print(r1)

# corpus_raw_str[:500]

31

In [37]:
len(sections_ls)
min(sections_ls, key=len) 

31

'But remember, please, the Law by which we live, We are not built to comprehend a lie ...\n\n--Rudyard Kipling, "The Secret of the Machines"\n\n'

In [38]:

target_string = "My name is maximums and my luck numbers are 12 45 78"
# split on white-space 
word_list = re.split(r"\s+", target_string)
print(word_list)

['My', 'name', 'is', 'maximums', 'and', 'my', 'luck', 'numbers', 'are', '12', '45', '78']


In [39]:
# Read corpus into a single string then split into paragraphs

'''
if len(corpus_filename) == 0:
  # If now file uploaded, use the file in Google gDrive
  corpus_filename = CORPUS_FILENAME
else:
  # The uploaded file has priority over the gDrive Corpus file
  pass
'''

def corpus2parags(corpus_filename):
  '''
  Given a corpus_filename (assuming already %cd into correct subdir)
  Return a list of min preprocessed raw paragraphs (corpus_parags_raw_temp_ls)
  '''

  with open(corpus_filename, "r", encoding='windows-1252') as infp:
    corpus_raw_str = infp.read()

  corpus_parags_raw_temp_ls = corpus_raw_str.split('\n\n')
  print(f'Corpus Paragraph Raw Count: {len(corpus_parags_raw_temp_ls)}')

  # Strip excess whitespace and drop empty lines
  corpus_parags_raw_temp_ls = [x.strip() for x in corpus_parags_raw_temp_ls if len(x.strip()) > MIN_PARAG_LEN]
  print(f'Corpus Paragraph -(whitespace only) Count: {len(corpus_parags_raw_temp_ls)}')

  # Drop lines that only contain punctuation (e.g. '"', '.', '...', etc)
  corpus_parags_raw_temp_ls = [x for x in corpus_parags_raw_temp_ls if len((re.sub(r'[^\w\s]','',x)).strip()) > MIN_PARAG_LEN]
  print(f'Corpus Paragraph -(punctuation only) Count: {len(corpus_parags_raw_temp_ls)}')

  return corpus_parags_raw_temp_ls

# Test  
'''
corpus_parags_raw_ls = read_corpus_parags(CORPUS_FILENAME)
print(f'We found #{len(corpus_parags_raw_ls)} lines\n')

print('\nThe first 10 lines of the Corpus:')
print('-----------------------------------\n')
corpus_parags_raw_ls[:10]

print('\nThe last 10 lines of the Corpus:')
print('-----------------------------------\n')
corpus_parags_raw_ls[-10:]
print('\n')
print(sorted(corpus_parags_raw_ls, key=lambda x: (len(x), x)))
''';

In [40]:
# Verify saved under newest filename

def get_recentfile(file_type='csv'):
  '''
  Given a file extension type,
  Return the most recently created file of that type 
  in the current directory
  '''
  file_pattern = "./*." + file_type
  print(f'file_pattern: {file_pattern}')
  list_of_files = glob.glob(file_pattern) # * means all if need specific format then *.csv
  latest_file = max(list_of_files, key=os.path.getmtime)

  return latest_file

# Test

# get_recentfile('txt')

## **Sentiment**

In [41]:
# Test to find longest String in Corpus (in terms of #tokens) 
#      must be <510 tokens for Transformers

# corpus_sents_df['sent_raw'].astype(str).apply(lambda x: len(x.split())).max() # split().len()).max()

In [42]:
def get_sentiments(model_base, sentiment_fn, sentiment_type='lexicon'):
  '''
  Given a model_base name and sentiment evaluation function
  Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants
  '''

  # Calculate Sentiment Polarities

  if sentiment_type == 'lexicon':
    corpus_sents_df[model_base] = corpus_sents_df['sent_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_parags_df[model_base] = corpus_parags_df['parag_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_sects_df[model_base] = corpus_sects_df['sect_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_chaps_df[model_base] = corpus_chaps_df['chap_clean'].apply(lambda text: sentiment_fn(str(text)))
  
  elif sentiment_type == 'compound':
    # VADER

    # Calculate dictionary of {neg/neu/pos/compound} values for sent_clean
    corpus_sents_df['scores'] = corpus_sents_df['sent_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_parags_df['scores'] = corpus_parags_df['parag_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_sects_df['scores'] = corpus_sects_df['sect_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_chaps_df['scores'] = corpus_chaps_df['chap_clean'].apply(lambda text: sentiment_fn(str(text)))

    # Extract Compound Sentiment
    corpus_sents_df[model_base]  = corpus_sents_df['scores'].apply(lambda score_dict: score_dict['compound'])
    corpus_parags_df[model_base]  = corpus_parags_df['scores'].apply(lambda score_dict: score_dict['compound'])
    corpus_sects_df[model_base]  = corpus_sects_df['scores'].apply(lambda score_dict: score_dict['compound'])
    corpus_chaps_df[model_base]  = corpus_chaps_df['scores'].apply(lambda score_dict: score_dict['compound'])

  elif sentiment_type == 'function':
    # TextBlob

    # Calculate dictionary of {neg/neu/pos/compound} values for sent_clean
    corpus_sents_df[model_base] = corpus_sents_df['sent_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_parags_df[model_base] = corpus_parags_df['parag_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_sects_df[model_base] = corpus_sects_df['sect_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_chaps_df[model_base] = corpus_chaps_df['chap_clean'].apply(lambda text: sentiment_fn(str(text)))

  else:
    print(f'ERROR: sentiment_type={sentiment_type} but must be one of (lexicon, compound, function)')
    return

  # Create new column names
  col_meanstd = f'{model_base}_meanstd'
  col_medianiqr = f'{model_base}_medianiqr'
  col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'
  col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'


  # Get Chapter Standardization with MeanSTD and RobustStandardization with MedianIQRScaling
  corpus_chaps_df[col_meanstd]  = mean_std_scaler.fit_transform(np.array(corpus_chaps_df[model_base]).reshape(-1, 1))
  corpus_chaps_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_chaps_df[model_base]).reshape(-1, 1))
  # Normalize the Chapter Sentiment by dividing by Chapter Length
  chaps_len_ls = list(corpus_chaps_df['token_len'])
  chaps_sentiment_ls = list(corpus_chaps_df[model_base])
  chaps_sentiment_norm_ls = [chaps_sentiment_ls[i]/chaps_len_ls[i] for i in range(len(chaps_len_ls))]
  # RobustStandardize Chapter sentiment values
  corpus_chaps_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(chaps_sentiment_norm_ls)).reshape(-1, 1))
  corpus_chaps_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_chaps_df[model_base]).reshape(-1, 1))
  corpus_chaps_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(chaps_sentiment_norm_ls)).reshape(-1, 1))

  # Get Section Standardization with MeanSTD and RobustStandardization with MedianIQRScaling
  corpus_sects_df[col_meanstd]  = mean_std_scaler.fit_transform(np.array(corpus_sects_df[model_base]).reshape(-1, 1))
  corpus_sects_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_sects_df[model_base]).reshape(-1, 1))
  # Normalize the Section Sentiment by dividing by Section Length
  sects_len_ls = list(corpus_sects_df['token_len'])
  sects_sentiment_ls = list(corpus_sects_df[model_base])
  sects_sentiment_norm_ls = [sects_sentiment_ls[i]/sects_len_ls[i] for i in range(len(sects_len_ls))]
  # RobustStandardize Section sentiment values
  corpus_sects_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(sects_sentiment_norm_ls)).reshape(-1, 1))
  corpus_sects_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_sects_df[model_base]).reshape(-1, 1))
  corpus_sects_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(sects_sentiment_norm_ls)).reshape(-1, 1))

  # Normalize the Paragraph Sentiment by dividing by Chapter Length
  parags_len_ls = list(corpus_parags_df['token_len'])
  parags_sentiment_ls = list(corpus_parags_df[model_base])
  parags_sentiment_norm_ls = [parags_sentiment_ls[i]/parags_len_ls[i] for i in range(len(parags_len_ls))]
  # RobustStandardize Paragraph sentiment values
  corpus_parags_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(parags_sentiment_norm_ls)).reshape(-1, 1))
  corpus_parags_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_parags_df[model_base]).reshape(-1, 1))
  corpus_parags_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(parags_sentiment_norm_ls)).reshape(-1, 1))

  # Normalize the Sentence Sentiment by dividing by Chapter Length
  sents_len_ls = list(corpus_sents_df['token_len'])
  sents_sentiment_ls = list(corpus_sents_df[model_base])
  sents_sentiment_norm_ls = [sents_sentiment_ls[i]/sents_len_ls[i] for i in range(len(sents_len_ls))]
  # RobustStandardize Sentence sentiment values
  corpus_sents_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(sents_sentiment_norm_ls)).reshape(-1, 1))
  corpus_sents_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_sents_df[model_base]).reshape(-1, 1))
  corpus_sents_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(sents_sentiment_norm_ls)).reshape(-1, 1))

  return

In [43]:
# Read in lexicon at given path into Dict[word]=polarity

def get_lexicon(lexicon_name, lexicon_format=2):
    """
    Read sentiment lexicon.csv file at lexicon_path
    into appropriate Dict[word]=polarity

    1. lexicon_dt[word] = <polarity value>

    Args:
        sa_lib (str, optional): [description]. Defaults to 'syuzhet'.
    """
    
    # global lexicon_df

    lexicon_df = pd.DataFrame()
    
    # print(os.getcwd())
    lexicons_ls = os.listdir('../sa_lexicons/')
    if (lexicon_name in lexicons_ls):
      print(f'Found {lexicon_name} in lexicon_directory)')
    # print(glob.glob('*.csv'))
    cp_cmd = f'copy ../sa_lexicons/{lexicon_name} ./'
    print(f'cp_cmd = {cp_cmd}')
    os.system(cp_cmd)
    os.system('cp ../sa_lexicons/' + lexicon_name.strip() + ' ./')
    os.listdir('.')  

    try:
      lexicon_df = pd.read_csv(lexicon_name)
      lexicon_df.info()
      # lexicon_df = lexicon_tmp_df.copy()
      # print(lexicon_df.head())
      return lexicon_df
    except:
      print(f'ERROR: Cannot read lexicon.csv at {lexicon_name}')
      return -1

'''
    print
    if (sa_lexicon == 'default'):
        lexicon_df = pd.read_csv(LEXICON_PATH)
        lexicon_df.columns = ['index_no', 'word', 'polarity']
        lexicon_df.drop(['index_no'], axis=1, inplace=True)
        lexicon_df.dropna(inplace=True)
        lexicon_dt = lexicon_df.set_index('word').T.to_dict('list')
        # unlist the polarity to type: float
        for key in lexicon_dt:
            lexicon_dt[key] = float(lexicon_dt[key][0])
        
    ### print(f"Exit get_sa_lex() with {len(lexicon_dt.keys())} entries in syuzhet_dt")
    return lexicon_dt
''';

In [44]:
# Sentence to Sentiment Polarity according to passed in Lexicon Dictionary

def text2sentiment(text_str, lexicon_dt):
  '''
  Given a text_str and lexicon_dt, calculate 
  the sentimety polarity.
  '''

  # Remove all not alphanumeric and whitespace characters
  text_str = re.sub(r'[^\w\s]', '', text_str) 

  text_str = text_str.strip().lower()
  if (len(text_str) < 1):
      print(f"ERROR: text2sentiment() given empty/null/invalid string: {text_str}")

  text_ls = text_str.split()
  # print(f'text_ls: {text_ls}')

  # Accumulated Total Sentiment Polarity for entire Sentence
  text_sa_tot = 0.0

  for aword in text_ls:
      # print(f'getting sa for word: {aword}')
      try:
          word_sa_fl = float(lexicon_dt[aword])
          text_sa_tot += word_sa_fl
          # print(f">>{aword} has a sentiment value of {word_sa_fl}")
      except TypeError: # KeyError:
          # aword is not in lexicon so it adds 0 to the sentence sa sum
          # print(f"TypeError: cannot convert {lexicon_dt[aword]} to float")
          continue
      except KeyError:
          # print(f"KeyError: missing key {aword} in defaultdict syuzhet_dt")
          continue
      except:
          e = sys.exc_info()[0]
          # print(f"ERROR {e}: sent2lex_sa() cannot catch aword indexing into syuzhet_dt error")
  
  # print(f"Leaving sent2lex_sa() with sentence sa value = {str(text_sa_tot)}")
  
  return text_sa_tot


# Test

# sent2sentiment('I hate and despise and abhor and dislike and am disgusted by Mondays.', lexicon_jockersrinker_dt)
# sent2sentiment('hate Mondays.', lexicon_jockersrinker_dt)

In [45]:
def plot_smas(section_view=True, model_name='vader', text_unit='sentence', wins_ls=[20], alpha=0.5, subtitle_str='', y_height=0, save2file=False):
  '''
  Given a model, text_unit
  Plot a SMA using default values and wrapping the function get_smas()
  '''

  if (section_view == True) and not any(x == text_unit for x in ['sentence', 'paragraph']):
    print(f'ERROR: You can only plot SMA within a Section with Sentence or Paragraph text units')
    return -99

  if text_unit == 'sentence':
    if section_view == False:
      ts_df = corpus_sents_df
    else:
      ts_df = section_sents_df
    wins_ls = [5,10,20]
  elif text_unit == 'paragraph':
    if section_view == False:
      ts_df = corpus_parags_df
    else:
      ts_df = section_parags_df
    wins_ls = [5,10,20]
  elif text_unit == 'section':
    ts_df = corpus_sects_df
    wins_ls=[20]
  else:
    print(f'ERROR: {text_unit} must be sentence, paragraph or section')

  sectno_loc = ts_df[model_name].min()

  if section_view ==False:
    # At Section boundries draw blue vertical lines 
    section_boundries_ls = list(corpus_sects_df['sent_no_start'])
    for i, sent_no in enumerate(section_boundries_ls):
      plt.text(sent_no, y_height, f'Sec#{i}', alpha=0.2, rotation=90)
      plt.axvline(sent_no, color='blue', alpha=0.1)
      # 'BigNews1', xy=(sent_no, 0.5), xytext=(-10, 25), textcoords='offset points',                   rotation=90, va='bottom', ha='center', annotation_clip=True)

      # plt.text(sent_no, -.5, 'goodbye',rotation=90, zorder=0)

    # At Chapter boundaries draw red vertical lines
    chapter_boundries_ls = list(corpus_chaps_df['sent_no_start'])
    for i, sent_no in enumerate(chapter_boundries_ls):
      plt.axvline(sent_no, color='navy', alpha=0.1)
      # plt.text(sent_no, .5, 'hello', rotation=90, zorder=0)

  get_smas(ts_df, model_name=model_name, text_unit=text_unit, wins_ls=wins_ls, alpha=alpha, subtitle_str=subtitle_str, save2file=save2file)

  if (save2file == True):
    # Save graph to file.
    plot_filename = f'plot_sma_sents_{model_name}.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return

In [46]:
# SMA 5% Sentiment of Sentence Sentiment

def get_smas(ts_df, model_name, text_unit='sentence', wins_ls=[5,10], alpha=0.5, scale_factor=1., subtitle_str='', mean_adj=0., do_plot=True, save2file=False):
  '''
  Given a model_name and time series DataFrame and list of win_rolls in percentages
  Return the rolling means of the time series using the window sizes in win_rolls
  '''

  temp_roll_df = pd.DataFrame() # TODO: save sma rolling values into temp_df and return this value

  win_1per = int(ts_df.shape[0]*0.01)
  if text_unit ==  'sentence':
    # win_1per = win_s1per
    x_idx = 'sent_no'
    fname_abbr = 'sents'
  elif text_unit == 'paragraph':
    # win_1per = win_p1per
    x_idx = 'parag_no'
    fname_abbr = 'parags'
  elif text_unit == 'section':
    win_1per = 1
    wins_ls = [int(0.1 * corpus_sects_df.shape[0])]  # Edge case to deal with very few Section data points
    x_idx = 'sect_no'
    fname_abbr = 'sects'
  else:
    print(f'ERROR: text_unit={text_unit} but must be either sentence, paragraph or section')
  
  for i, awin_size in enumerate(wins_ls):
    if len(str(awin_size)) == 1:
      awin_str = '0'+str(awin_size)+'0'
    else:
      awin_str = str(awin_size)+ '0'
    col_roll_str = f'{model_name}_mean_roll{awin_str}'
    win_size = awin_size*win_1per
    ts_df[col_roll_str] = ts_df[model_name].rolling(window=win_size, center=True).mean()
  
    if do_plot == True:
      alabel = f'{model_name} (win={awin_size})'
      ts_df['y_scaled'] = ts_df[col_roll_str]*scale_factor + mean_adj 
      sns.lineplot(data=ts_df, x=x_idx, y='y_scaled', legend='brief', label=alabel, alpha=alpha)
      
  plt.title(f'{CORPUS_FULL} (Model: {model_name}: {subtitle_str}) \nSMA Smoothed {text_unit} Sentiment Plot (windows={wins_ls})')
  # plt.legend(loc='best')

  if save2file == True:
    # Save graph to file.
    plot_filename = f'plot_{fname_abbr}_sa_mean_050100sma.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return temp_roll_df

In [47]:
def get_lexstats(ts_df, model_name, text_unit='sentence'):
  '''
  Given a model name
  calculate, store and return time series stats
  '''
  
  global corpus_lexicons_stats_dt

  temp_dt = {}
  
  if text_unit == 'sentence':
    stat_idx = f'{model_name}_sents'
  elif text_unit == 'paragraph':
    stat_idx = f'{model_name}_parags'
  elif text_unit == 'section':
    stat_idx = f'{model_name}_sects'
  elif text_unit == 'chapter':
    stat_idx = f'{model_name}_chaps'
  else:
    print(f'ERROR: {text_unit} must either be sentence, paragraph, or section')

  sentiment_min = ts_df[model_name].min()
  sentiment_max = ts_df[model_name].max()

  temp_dt = {'sentiment_min' : sentiment_min,
             'sentiment_max' : sentiment_max}

  corpus_lexicons_stats_dt[stat_idx] = temp_dt
                                     
  return 

# Test
# get_lexstats('afinn')
# corpus_lexicons_stats_dt

In [48]:
def lex_discrete2continous_sentiment(text, lexicon):
  '''
  Given a plain text string, give it to
    Stanford Stanza (OpenNLP) to calculate sentiment for each word on a 3 point scale 0-2
  Return a sentiment value for the entire sentence (sum of word sentiments/log(len of sentence)) 
    that approximates a normal distribution for all values
    In order to get more fine grained measure of overall Sentence sentiment
    Sentiment values will be Normalized/Standardized so absolute precision is not required
  '''
  text_sentiment_tot = 0.
  text_ls = text.split()
  text_len = len(text_ls)
  for aword in text_ls:
    word_sentiment = text2sentiment(str(aword), lexicon)
    text_sentiment_tot += word_sentiment
  text_sentiment_norm = text_sentiment_tot/(np.log(text_len)+0.01)

  return text_sentiment_norm

In [49]:
def clip_outliers(floats_ser):
  '''
  Given a pd.Series of float values
  Return a list with outliers removed, values limited within 3 median absolute deviations from median
  '''
  # https://www.statsmodels.org/stable/generated/statsmodels.robust.scale.mad.html#statsmodels.robust.scale.mad

  # Old mean/std, less robust
  # ser_std = floats_ser.std()
  # ser_median = floats_ser.mean() # TODO: more robust: asym/outliers -> median/IQR or median/median abs deviation

  floats_np = np.array(floats_ser)
  ser_median = floats_ser.median()
  ser_mad = robust.mad(floats_np)
  print(f'ser_median = {ser_median}')
  print(f'ser_mad = {ser_mad}')

  if ser_mad == 0:
    # for TS with small ranges (e.g. -1.0 to +1.0) Median Abs Deviation = 0
    #   so pass back the original time series
    floats_clip_ls = list(floats_ser)

  else:
    ser_oldmax = floats_ser.max()
    ser_oldmin = floats_ser.min()
    print(f'ser_max = {ser_oldmax}')
    print(f'ser_min = {ser_oldmin}')

    ser_upperlim = ser_median + 2.5*ser_mad
    ser_lowerlim = ser_median - 2.5*ser_mad
    print(f'ser_upperlim = {ser_upperlim}')
    print(f'ser_lowerlim = {ser_lowerlim}')

    # Clip outliers to max or min values
    floats_clip_ls = np.clip(floats_np, ser_lowerlim, ser_upperlim)
    # print(f'max floast_ls {floats_ls.max()}')

    # def map2range(value, low, high, new_low, new_high):
    #   '''map a value from one range to another'''
    #   return value * 1.0 / (high - low + 1) * (new_high - new_low + 1)

    # Map all float values to range [-1.0 to 1.0]
    # floats_clip_sig_ls = [map2range(i, ser_oldmin, ser_oldmax, ser_upperlim, ser_lowerlim) for i in floats_clip_ls]

    # listmax_fl = float(max(floats_ls))
    # floats_ls = [i/listmax_fl for i in floats_ls]
    #floats_ls = [1/(1+math.exp(-i)) for i in floats_ls]

  return floats_clip_ls  # floats_clip_sig_ls

# Test
# Will not work on first run as corpus_sents_df is not defined yet
'''
data = np.array([1, 4, 4, 7, 12, 13, 16, 19, 22, 24])
test_ls = clip_outliers(corpus_sents_df['vader'])
print(f'new min is {min(test_ls)}')
print(f'new max is {max(test_ls)}')
''';

## **Pandas**

In [50]:
def rename_cols(ts_df, col_old_ls, suffix_str='_raw'):
  '''
  Given a DataFrame, list of columns in DataFrame and a suffix,
  Return a Dictionary mapping old col names to new col name (orig+suffix)
  '''

  col_new_ls = []
  for acol in col_old_ls:
    acol_new = acol + suffix_str
    col_new_ls.append(acol_new)

  # Create dict for col mapping: keys=old col names, value=new col names
  col_rename_dt = dict(zip(col_old_ls, col_new_ls))

  # ts_df.rename(columns=col_rename_dt, errors="raise")

  return col_rename_dt

# test_ls = [col for col in corpus_sents_df.columns if not(renaming_fun(col) is None)]
# print(f'test_ls: {test_ls}')

# Test
# col_rename_dt = rename_cols(corpus_sents_df, sentiment_only_cols_ls)
# col_rename_dt

## **Time Series**

In [51]:
def norm2negpos1(data_ser):
  '''
  Given a series of floating number
  Return a a list of same values normed between -1.0 and +1.0
  '''
  # data_np = np.matrix(data_ser)

  scaler=MinMaxScaler(feature_range=(-1.0, 1.0))
  temp_ser = scaler.fit_transform(np.matrix(data_ser))
  
  return temp_ser

# Test
'''
temp_np = norm2negpos1(corpus_all_df[['xlnet_sst5']])
print(type(temp_np))
temp_np.shape
''';

In [52]:
def standardize_ts(data_ser):
  '''
  Given a series of floating number
  Return a a list of same values normed between -1.0 and +1.0
  '''
  # data_np = np.matrix(data_ser)

  std_scaler = StandardScaler()
  df_std = std_scaler.fit_transform(np.array(data_ser))
  
  return df_std

# Test
'''
temp_np = norm2negpos1(corpus_all_df[['xlnet_sst5']])
print(type(temp_np))
temp_np.shape
temp_np
''';

In [53]:
MODELS_LS

['vader',
 'textblob',
 'stanza',
 'afinn',
 'bing',
 'sentimentr',
 'syuzhet',
 'pattern',
 'sentiword',
 'senticnet',
 'nrc']

In [54]:
# This must be defined AFTER the corpus_sects_df DataFrame is created in the Preprocessing Step below

# Raw Plot of Section Sentiments (Adjusted for (x-axis) mid-Section Sentence No and (y-axis) Sentiment weighted by Section length )

# corpus_sects_df = pd.DataFrame()  # Create empty early as required by some utility functions

def plot_crux_sections(model_names_ls, semantic_type='section', subtitle_str='', label_token_ct=0, title_xpos = 0.8, title_ypos=0.2, sec_y_height=0, save2file=False):
  '''
  Given a Sections DataFrame, model_name and semantic type,
  Return a Plot of the Cruxes
  '''

  crux_points_dt = {}
  model_stand_names_ls = []
  section_boundries_ls = []


  # print(f'Using model_names: {model_names_ls}')

  # sns.lineplot(data=ts_df, x='sent_no_mid', y=amodel_stand, markers=['o'], alpha=0.5, label=amodel_stand); # .set_title(f'{CORPUS_FULL} \n Plot Section Sentiment (Bing Lexicon)')


  # At Section boundries draw blue vertical lines 
  section_boundries_ls = list(corpus_sects_df['sent_no_start'])
  for i, sent_no in enumerate(section_boundries_ls):
    plt.text(sent_no, sec_y_height, f'Sec#{i}', alpha=0.2, rotation=90)
    plt.axvline(sent_no, color='blue', alpha=0.1);

  # At Chapter boundaries draw red vertical lines
  chapter_boundries_ls = list(corpus_chaps_df['sent_no_start'])
  for i, sent_no in enumerate(chapter_boundries_ls):
    plt.axvline(sent_no, color='navy', alpha=0.1);

  # Error check and assign DataFrame associated with each semantic_type
  if semantic_type == 'section':
    # Get midpoints of each Section
    ts_df=corpus_sects_df
    midpoints_ls = list(corpus_sects_df['sent_no_mid'])
  elif semantic_type == 'chapter':
    # Get midpoints of each Chapter
    ts_df=corpus_chaps_df
    midpoints_ls = list(corpus_chaps_df['sent_no_mid'])
  else:
    print(f"ERROR: semantic_type={semantic_type} must be either 'section' or 'chapter'")
    return -1

  # How many sentiment time series are we plotting?
  if len(model_names_ls) == 1:
    
    # Plotting only one model
    model_name_full = str(model_names_ls[0])
    model_name_root = model_name_full.split('_')[0]
    print(f'model_name_full: {model_name_full} and model_name_root: {model_name_root}')
    if model_name_root in MODELS_LS:
      # Plot
      print(f'about to sns.lineplot model: ') # {ts_df}')
      g = sns.lineplot(data=ts_df, x='sent_no_mid', y=model_name_full, markers=['o'], alpha=0.5, label=model_name_full) # .set_title(f'{CORPUS_FULL} \n Plot Section Sentiment and Cruxes (Model: {models_names_ls[0].capitalize()})')
      # g._legend.remove()
      # print(f'model_name_full={model_name_full}')
      # plt.plot(ts_df.sent_no_mid, ts_df[model_name_full], markers="o", alpha=0.5, label=model_name_full)
    else:
      print(f'ERROR: model_names_ls[0]={model_name_root} is invalid,\n    must be one of {MODELS_LS}')
      return -1

    # If plotting only one model, add labels
    midpoints_sentiment_ls = list(ts_df[model_name_full])
    sect_ct = 0
    for x,y in zip(midpoints_ls, midpoints_sentiment_ls): 
      label_token_int = int(label_token_ct)
      if label_token_int < 0:
        label = ''
      elif label_token_int == 0:
        # if arg label_token_ct == 0, just print sent_no
        label = f"#{x}({sect_ct})"
      else:
        # if arg label_token_ct > 0, print the first label_token_ct words of sentence at crux point
        label = f"#{x}({sect_ct}) {' '.join(corpus_sents_df.iloc[x-1]['sent_raw'].split()[:label_token_int])}"; # \nPolarity: {y:.2f}'

      # Save Crux point in crux_points_dt Dictionary if plotting Cruxes for a single/specific Model
      crux_full_str = ' '.join(corpus_sents_df.iloc[x]['sent_raw'].split())
      crux_points_dt[x] = [y, crux_full_str]

      plt.annotate(label,
                   (x,y),
                   textcoords='offset points',
                   xytext=(0,10),
                   ha='center',
                   rotation=90)
      sect_ct += 1

    plt.title(f'{CORPUS_FULL} \n Plot {semantic_type.capitalize()} Sentiment ({model_name_full.capitalize()})\n{subtitle_str}', x=title_xpos, y=title_ypos);
    # Plot
    plt.plot(midpoints_ls, midpoints_sentiment_ls, marker="o", ms=6) # , markevery=[0,1])

  else:
    # If plotting multiple models
    model_names_str = 'Multiple Models'
    for i, model_name_full in enumerate(model_names_ls):
      # Error check and assign correct model names
      model_name_root = model_name_full.split('_')[0]
      if model_name_root in MODELS_LS:
        # Plot
        g = sns.lineplot(data=ts_df, x='sent_no_mid', y=model_name_full, markers=['o'], alpha=0.5, label=model_name_full) # .set_title(f'{CORPUS_FULL} \n Plot Section Sentiment and Cruxes (Model: {models_names_ls[0].capitalize()})')
        # g._legend.remove()
        # plt.plot(ts_df.sent_no_mid, ts_df[model_name_full], marker="o", alpha=0.5, label=model_name_full)
      else:
        print(f'ERROR: model_names_ls[]={model_name_root} is invalid,\n    must be one of {MODELS_LS}')
        return -1

      # Plot
      g = sns.lineplot(data=ts_df, x='sent_no_mid', y=model_name_full, markers=['o'], alpha=0.5, label=model_name_full) # .set_title(f'{CORPUS_FULL} \n Plot Section Sentiment and Cruxes (Model: {models_names_ls[0].capitalize()})')
      # g._legend.remove()

    plt.title(f'{CORPUS_FULL} \n Plot {semantic_type.capitalize()} Sentiment (Standardized Models)\n{subtitle_str}', x=title_xpos, y=title_ypos)

  # plt.legend(loc='best');

  if (save2file == True):
    # Save graph to file.
    models_names_ls = [x[:2] for x in model_names_ls]
    models_names_str = ''.join(models_names_ls)
    plot_filename = f'plot_cruxes_{semantic_type}_{models_names_str}_{models_names_str}.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return crux_points_dt

In [55]:
def plot_histogram(model_name='vader', text_unit='sentence', save2file=False):
  '''
  Given a model, text_unit
  Plot a Histogram using the default DataFrame
  '''

  if text_unit == 'sentence':
    ts_df = corpus_sents_df

  elif text_unit == 'paragraph':
    ts_df = corpus_parags_df

  elif text_unit == 'section':
    ts_df = corpus_sects_df

  elif text_unit == 'chapter':
    ts_df = corpus_chaps_df

  else:
    print(f'ERROR: {text_unit} must be sentence, paragraph or section')

  sns.histplot(ts_df[model_name], kde=True).set_title(f'{CORPUS_FULL} \n Histogram {text_unit.capitalize()} Sentiment (Model {model_name.capitalize()})')
  # get_smas(ts_df, model_name=model_name, text_unit=text_unit, win_ls=wins_def_ls)

  if (save2file == True):
    # Save graph to file.
    plot_filename = f'plot_hist_{text_unit}_{model_name}.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return

In [56]:
# Raw Plot of Section Sentiments (Not scaled by mid-Section Sentence No to match Sentence/Paragraph x-axes)

def plot_raw_sections(ts_df='corpus_sents_df', model_name='vader', semantic_type='sentence', save2file=False):
  '''
  Given a DataFrame, model_name column, semantic_type 
  Plot the raw sentiment types
  Options to save2file
  ''' 
  
  # if (PLOT_OUTPUT == 'All') | (PLOT_OUTPUT == 'Major'):
  sns.lineplot(data=ts_df, x='sect_no', y=model_name, alpha=0.5).set_title(f'{CORPUS_FULL} \n Plot {semantic_type} Sentiment (Raw {model_name.capitalize()})')

  if save2file == True:
    # Save graph to file.
    plot_filename = f'plot_nostand_sects_{model_name}.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return

# Test
# plot_raw_sections(ts_df=corpus_sects_df, model_name='pattern', semantic_type='section', save2file=False);

In [57]:
# Raw Plot of Section Sentiments (Not scaled by mid-Section Sentence No to match Sentence/Paragraph x-axes)

def plot_raw_sentiments(model_name='vader', semantic_type='sentence', save2file=False):
  '''
  Given a DataFrame, model_name column, semantic_type 
  Plot the raw sentiment types
  Options to save2file
  ''' 
  
  if semantic_type == 'sentence':
    ts_df = corpus_sents_df
    x_units = 'sent_no'
  elif semantic_type == 'paragraph':
    ts_df = corpus_parags_df
    x_units = 'parag_no'
  elif (semantic_type == 'section') | (semantic_type == 'section_stand'):
    ts_df = corpus_sects_df
    x_units = 'sect_no'
  elif (semantic_type == 'chapter') | (semantic_type == 'chapter_stand'):
    ts_df = corpus_chaps_df
    x_units = 'chap_no'
    
  else:
    print(f'ERROR: {semantic_type} must be sentence, paragraph or section')


  # if (PLOT_OUTPUT == 'All') | (PLOT_OUTPUT == 'Major'):
  sns.lineplot(data=ts_df, x=x_units, y=model_name, alpha=0.5, label=model_name).set_title(f'{CORPUS_FULL} \n Plot {semantic_type} Sentiment (Raw {model_name.capitalize()})')
  
  plt.legend(loc='best')

  if save2file == True:
    # Save graph to file.
    plot_filename = f'plot_raw_sentiments_{semantic_type}_{model_name}.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return

# Test
# plot_raw_sections(ts_df=corpus_sects_df, model_name='pattern', semantic_type='section', save2file=False);

In [58]:
# TODO: must plot in order to save, cannot save without first plotting

def get_lowess(ts_df='corpus_parags_df', models_ls=MODELS_LS, text_unit='paragraph', plot_subtitle='', alabel='', afrac=1./10, ait=5, alpha=0.5, do_plot=True, save2file=False):
  '''
  Given a DataFrame, list of column to plot, LOWESS params fraction and iterations,
  Return a DataFrame with LOWESS values
  If 'plot=True', also output plot
  '''

  # global corpus_all_df

  lowess_df = pd.DataFrame()

  # Step 1: Calculate LOWESS smoothed values
  for i,acol in enumerate(models_ls):
    sm_x, sm_y = sm_lowess(endog=ts_df[acol].values, exog=ts_df.index.values, frac=afrac, it=ait, return_sorted = True).T
    col_new = f'{acol}_lowess'
    lowess_df[col_new] = pd.Series(sm_y)
    # Optionally plot LOWESS for all models
    if do_plot:
      if alabel == '':
        alable == acol
      plt.plot(sm_x, sm_y, label=alabel, alpha=alpha, linewidth=2)

  lowess_df['median'] = lowess_df.median(axis=1) # sm_y # corpus_all_df[df_cols_ls].median(axis=1)
  
  # Step 2: Optionally plot LOWESS for median
  if do_plot:
    # sm_x, sm_y = sm_lowess(endog=lowess_df.median, exog=lowess_df.index.values,  frac=afrac, it=ait, return_sorted = True).T
    # plt.plot(sm_x, sm_y, label='median', alpha=0.9, linewidth=2, color='black')
    
    frac_str = str(round(100*afrac))
    plt.title(f'{CORPUS_FULL} \n {plot_subtitle} {text_unit} Standardized Sentiment Smoothed with LOWESS (frac={frac_str})')
    plt.legend(title='Sentiment Model')

  # Step 3: Optionally save to file
  if save2file:
    # Save Plot to file.
    plot_filename = f'plot_{text_unit}_lowess_{plot_subtitle.split()[0].lower()}_{author_str}_{title_str}.png'
    # plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plot_filename, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');


  return lowess_df

# Test
'''
new_lowess_col = f'{sa_model}_lowess'
my_frac = 1./10
my_frac_per = round(100*my_frac)
new_lowess_col = f'{sa_model}_lowess_{my_frac_per}'
corpus_all_df[new_lowess_col] = plot_lowess(corpus_all_df, [sa_model], afrac=my_frac)
corpus_all_df.head()
''';

In [59]:
def get_sent2dets(sent_no):
  '''
  Given a Sentence Number
  Return the corresponding Paragraph, Section and Chapter Numbers that contain it
  '''

  # Get Paragraph No containing given Sentence No
  sent_parag_no = int(corpus_sents_df[corpus_sents_df['sent_no']==sent_no]['parag_no'])

  # Get Section No containing given Sentence No.
  corpus_sects_ls = list(corpus_sects_df['sect_no'])
  for asect_no in corpus_sects_ls:
    if (int(corpus_sects_df[corpus_sects_df['sect_no'] == asect_no]['sent_no_start']) > sent_no):
      break
    sent_sect_no = asect_no
    # print(f'asect={asect_no}')

  # Get Chapter No containing given Sentence No.
  corpus_chaps_ls = list(corpus_chaps_df['chap_no'])
  for achap_no in corpus_chaps_ls:
    if (int(corpus_chaps_df[corpus_chaps_df['chap_no'] == achap_no]['sent_no_start']) > sent_no):
      break
    sent_chap_no = achap_no
    # print(f'achap={achap_no}')


  return sent_parag_no, sent_sect_no, sent_chap_no

# Test
# sent_parag_no, sent_sect_no, sent_chap_no = get_sent2dets(1408)
# print(f'sent_parag_no={sent_parag_no}\nsent_sect_no={sent_sect_no}\nsent_chap_no={sent_chap_no}')

In [60]:
def get_sentnocontext(sent_no=1, n_sideparags=1, sent_highlight=True):
  '''
  Given a sentence number in the Corpus
  Return the containing paragraph and n-paragraphs on either side
  (e.g. if n=2, return 2+1+2=5 paragraphs)
  '''

  parag_target_no = int(corpus_sents_df[corpus_sents_df['sent_no'] == sent_no]['parag_no'])
  # print(f'parag_target_no = {parag_target_no} and type: {type(parag_target_no)}')

  if n_sideparags == 0:
    parags_context_ls = list(corpus_parags_df[corpus_parags_df['parag_no'] == parag_target_no]['parag_raw'])

  else:
    parag_start = parag_target_no - n_sideparags
    parag_end = parag_target_no + n_sideparags + 1
    parags_context_ls = list(corpus_parags_df.iloc[parag_start:parag_end]['parag_raw'])


  if sent_highlight == True:
    parag_match_str = str(parags_context_ls[n_sideparags])
    # print(f'parag_match_str:\n  {parag_match_str}')
    sent_idx = sent_no
    sent_str = (corpus_sents_df[corpus_sents_df['sent_no']==sent_idx]['sent_raw'].values)[0]
    sent_str_up = sent_str.upper()
    # print(f'sent_str:\n  {sent_str}')
    # parags_context_ls[n_sideparags] 
    parags_context_ls[n_sideparags] = parag_match_str.replace(sent_str, sent_str_up)

  return parags_context_ls

# Te
# context_highlighted = get_sentnoparags(sent_no=1051, n_sideparags=1)
# print(context_highlighted)

In [61]:
def get_sentnocontext_report(the_sent_no=7, the_n_sideparags=1, the_sent_highlight=True):
  '''
  Wrapper function around  get_sentnocontext()
  Prints a nicely formatted context report
  '''

  context_noparags = the_n_sideparags*2+1

  # print('-------------------------------------------------------------')
  print(f'The {context_noparags} Paragraph(s) Context around the Sentence #{Crux_Sentence_No} Crux Point:')
  print('-------------------------------------------------------------')
  print(f"\nCrux Sentence #{the_sent_no} Raw Text: -------------------------------\n\n    {str(corpus_sents_df[corpus_sents_df['sent_no'] == the_sent_no]['sent_raw'].values[0])}\n") # iloc[the_sent_no]['sent_raw']}")

  sent_parag_no, sent_sect_no, sent_chap_no = get_sent2dets(the_sent_no)
  print(f"\nCrux Sentence #{the_sent_no} is Contained in: ---------------------------\n\n    Paragraph #{sent_parag_no}\n      Section #{sent_sect_no}\n      Chapter #{sent_chap_no}\n")

  print(f"\n{context_noparags} Paragraph(s) Context: ------------------------------")
  context_parags_ls = get_sentnocontext(sent_no=the_sent_no, n_sideparags=the_n_sideparags, sent_highlight=the_sent_highlight)
  context_len = len(context_parags_ls)
  context_mid = context_len//2
  for i, aparag in enumerate(context_parags_ls):
    if i==context_mid:
      # print(f'\n>>> Paragraph #{i}: <<< Crux Point Sentence CAPITALIZED within this Paragraph\n\n    {aparag}')
      print(f'\n<*> {aparag}')
    else:
      # print(f'\n    Paragraph #{i}:\n\n    {aparag}')
      print(f'\n    {aparag}')

  return

# Test
# get_sentnocontext_report(sent_no=1051, n_sideparags=1, sent_highlight=True)

In [62]:
def get_section_timeseries(sect_no):
  '''
  Given a Section No in the current Corpus
  Return the start,mid and ending Sent No for this Section as well as the Sentiment Time Series between the start/end Sentence for this Section
  '''
  
  section_count = corpus_sects_df.shape[0]

  # Compute the start, mid and end Sentence numbers for the selected Section
  if Select_Section_No >= section_count:
    print(f'ERROR: You picked Section #{Select_Section_No}.\n  Section for this Corpus must be between 0 and {section_count-1}')
    return -1

  else:

    # Get the starting and middle Sentence No of this Section
    sect_sent_start = int(corpus_sects_df[corpus_sects_df['sect_no'] == Select_Section_No]['sent_no_start'].values)
    # sect_sent_mid = int(corpus_sects_df[corpus_sects_df['sect_no'] == Select_Section_No]['sent_no_mid'].values)

    # Calculate last Sentence No of this Section
    if Select_Section_No == (section_count-1):   
      print(f'You selected the last Section of this Corpus')
      sect_sent_end = corpus_sents_df.shape[0] - 1
    else:
      sect_sent_end = int(corpus_sects_df[corpus_sects_df['sect_no'] == Select_Section_No+1]['sent_no_start'].values) # - 1
      
    print(f'Section #{sect_no}:----------')
    print(f'\nsect_sent_start: {sect_sent_start}')
    # print(f'sect_sent_mid: {sect_sent_mid}')
    print(f'sect_sent_end: {sect_sent_end}')


  # Comput the start, and end Paragraph numbers for the selected Section
  sect_parag_start = int(corpus_sents_df[corpus_sents_df['sent_no'] == sect_sent_start]['parag_no'].values)
  sect_parag_end = int(corpus_sents_df[corpus_sents_df['sent_no'] == sect_sent_end]['parag_no'].values)

  print(f'\nsect_parag_start: {sect_parag_start}')
  print(f'sect_parag_end: {sect_parag_end}')


  # Extract and Return both a Sentence and Paragraph DataFrame for this Section 

  section_sents_df = corpus_sents_df.iloc[sect_sent_start:sect_sent_end]

  section_parags_df = corpus_parags_df.iloc[sect_parag_start:sect_parag_end]


  return section_sents_df, section_parags_df

# Test

# section_sents_df, section_parags_df = get_section_timeseries(Select_Section_No)

# section_sents_df.head()

# print(f'\nsection_sents_df.shape: {section_sents_df.shape}')
# print(f'section_parags_df.shape: {section_parags_df.shape}')

In [63]:
"""


def get_crux_points(col_series, semantic_type='sentence', win_lowess=5, do_plot=True, save2file=False):
  '''
  Given a DataFrame and a Time Series Column within it and a LOWESS window
  Return a list of Min/Max Crux Point (x,y) coordinate tuples for that Column Time Series
  '''

  crux_ls = []

  if semantic_type == 'sentence':
    ts_df = corpus_sents_df
    x_units = 'sent_no'
  elif semantic_type == 'paragraph':
    ts_df = corpus_parags_df
    x_units = 'parag_no'
  elif (semantic_type == 'section') | (semantic_type == 'section_stand'):
    ts_df = corpus_sects_df
    x_units = 'sect_no'
  elif (semantic_type == 'chapter') | (semantic_type == 'chapter_stand'):
    ts_df = corpus_chaps_df
    x_units = 'chap_no'
    
  else:
    print(f'ERROR: {semantic_type} must be sentence, paragraph or section')



  series_len = ts_df.shape[0]

  series_no_min = ts_df[x_units].min()
  seires_no_max = ts_df[x_units].max()

  sm_x = ts_df.index.values
  sm_y = ts_df[col_series].values

  half_win = int((win_lowess/100)*series_len)

  # Find peaks(max).
  # peak_indexes = signal.argrelextrema(sm_y, np.greater, order=half_win, mode='wrap') argrelextrema will not detect flat peaks
  peak_indexes = signal.find_peaks(sm_y, distance=half_win) # np.greater, order=half_win, mode='wrap')
  peak_indexes = peak_indexes[0]

  peak_x_ls = list(peak_indexes)
  peak_y_ls = list(sm_y[peak_indexes])

  # Find valleys(min).
  # valley_indexes = signal.argrelextrema(sm_y, np.less, order=half_win, mode='clip')
  valley_indexes = signal.find_peaks(-sm_y, distance=half_win) # np.less, order=half_win, mode='clip')
  valley_indexes = valley_indexes[0]
  
  valley_x_ls = list(valley_indexes)
  valley_y_ls = list(sm_y[valley_indexes])

  # Save all peaks/valleys as list of (x,y) coordinate tuples
  print(f'type peak_x_ls is: {type(peak_x_ls)}')
  x_all_ls = peak_x_ls + valley_x_ls
  y_all_ls = peak_y_ls + valley_y_ls
  crux_coord_ls = tuple(zip(x_all_ls, y_all_ls)) 

  print(f'Original Series length={series_len} vs LOWESS Series length={len(x_all_ls)}')


  if do_plot == True:
    # Plot main graph.
    (fig, ax) = plt.subplots()
    ax.plot(sm_x, sm_y)

    win_half = 0 # 2500

    # Plot peaks.
    # ax.plot(peak_x + win_half, peak_y, marker='o', linestyle='none', color='green', label="Peaks")
    ax.scatter(peak_x_ls, peak_y_ls)
    for i, txt in enumerate(list(peak_x_ls)):
        ax.annotate(f'  Sent #{txt}', (peak_x_ls[i], peak_y_ls[i]), rotation=90, annotation_clip=True)

    # Plot valleys.
    # ax.plot(valley_x + win_half, valley_y, marker='o', linestyle='none', color='red', label="Valleys")
    ax.scatter(valley_x_ls, valley_y_ls)
    for i, txt in enumerate(list(valley_x_ls)):
        ax.annotate(f'Sent #{txt}', (valley_x_ls[i], valley_y_ls[i]), rotation=270, xytext=(valley_x_ls[i], valley_y_ls[i]-4))

    # for i, txt in enumerate(list(valley_x_ls)):
    #     ax.annotate(f'\n\n\nSent No.\n   {txt}', (valley_x_ls[i], valley_y_ls[i]))
    # plt.plot(x, y, 'bo')
    # texts = [plt.text(valley_x_ls[i], valley_y_ls[i], 'Sent No.\n   %s' %valley_x_ls[i], ha='right', va='top') for i in range(len(valley_x_ls))]
    # adjust_text(texts)

    # Confidence Interval (Min/Max Range)
    # plt.fill_between(sentiment_lowess_df['x_value'], sentiment_lowess_df['min'], sentiment_lowess_df['max'], alpha=.3, color='lightskyblue')

    plt.title(f'{CORPUS_FULL}\nRaw Sentence Sentiments with selected Section #{Select_Section_No}')
    plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')

    # locs, labels = xticks()  # Get the current locations and labels.
    plt.xticks(np.arange(sent_no_min, sent_no_max, step=10))  # Set label locations.

    plt.ylabel(f'Sentiment Value')
    plt.legend(loc='best');
  
  if save2file == True:
    # Save graph to file.
    plt.title(f'{BOOK_TITLE_FULL} \n LOWESS Smoothed Median Sentiment Curve with Crux Points via SciPy.argrelextrema')
    plt.legend(loc='best')
    plt.savefig('argrelextrema.png')

  return crux_coord_ls


  # if (PLOT_OUTPUT == 'All') | (PLOT_OUTPUT == 'Major'):
  sns.lineplot(data=ts_df, x=x_units, y=model_name, alpha=0.5, label=model_name).set_title(f'{CORPUS_FULL} \n Plot {semantic_type} Sentiment (Raw {model_name.capitalize()})')
  
  plt.legend(loc='best')

  if save2file == True:
    # Save graph to file.
    plot_filename = f'plot_raw_sentiments_{semantic_type}_{model_name}.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return



""";

In [64]:

def get_crux_points(ts_df, col_series, text_type='sentence', win_lowess=5, y_height=0, do_plot=True, save2file=False):
  '''
  Given a DataFrame and a Time Series Column within it and a LOWESS window
  Return a list of Min/Max Crux Point (x,y) coordinate tuples for that Column Time Series
  '''

  crux_ls = []

  series_len = ts_df.shape[0]

  sent_no_min = ts_df.sent_no.min()
  sent_no_max = ts_df.sent_no.max()
  # print(f'sent_no_min {sent_no_min}')

  sm_x = ts_df.index.values
  sm_y = ts_df[col_series].values

  half_win = int((win_lowess/100)*series_len)

  # Find peaks(max).
  # peak_indexes = signal.argrelextrema(sm_y, np.greater, order=half_win, mode='wrap') argrelextrema will not detect flat peaks
  peak_indexes = signal.find_peaks(sm_y, distance=half_win) # np.greater, order=half_win, mode='wrap')
  # peak_indexes = peak_indexes + sent_no_min
  # print(f'peak_indexes[0]: {peak_indexes_np[0]}')
  # print(f'peak_indexes type: {type(peak_indexes_np[0])}')
  # peak_indexes_np = peak_indexes_np + sent_no_min
  peak_indexes = peak_indexes[0]

  peak_x_ls = list(peak_indexes)
  peak_y_ls = list(sm_y[peak_indexes])

  # Find valleys(min).
  valley_indexes = signal.argrelextrema(sm_y, np.less, order=half_win, mode='clip')
  valley_indexes = valley_indexes[0]
  
  valley_x_ls = list(valley_indexes)
  valley_y_ls = list(sm_y[valley_indexes])

  # Save all peaks/valleys as list of (x,y) coordinate tuples
  # print(f'type peak_x_ls is: {type(peak_x_ls)}')
  x_all_ls = peak_x_ls + valley_x_ls
  # readjust starting Sentence No to start with first sentence in segement window
  x_all_ls = [x+sent_no_min for x in x_all_ls]
  y_all_ls = peak_y_ls + valley_y_ls
  crux_coord_ls = tuple(zip(x_all_ls, y_all_ls)) 

  # print(f'Original Series length={series_len} vs LOWESS Series length={len(x_all_ls)}')


  if do_plot == True:
    # Plot main graph.
    (fig, ax) = plt.subplots()
    ax.plot(sm_x, sm_y)

    if text_type == 'sentence':
      paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
      for i, aparag in enumerate(paragraph_boundries_ls):
        if i%5 == 0:
          # Plot every 5th paragraph
          sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
          plt.text(sent_no, y_height, f'Paragraph #{aparag}', alpha=0.2, rotation=90)
          plt.axvline(sent_no, color='blue', alpha=0.1)
    elif text_type == 'paragraph':
      paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
      for i, aparag_no in enumerate(paragraph_boundries_ls):
        if i%5 == 0:
          # Plot every 5th paragraph
          sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
          plt.text(aparag_no, y_height, f'Paragraph #{aparag_no}', alpha=0.2, rotation=90)
          plt.axvline(aparag_no, color='blue', alpha=0.1)    
    else:
      print(f"ERROR: text_type is {text_type} but must be either 'sentence' or 'paragarph'")

    win_half = 0 # 2500

    # Plot peaks.
    # ax.plot(peak_x + win_half, peak_y, marker='o', linestyle='none', color='green', label="Peaks")

    # readjust starting Sentence No to start with first sentence in segement window
    peak_x_ls = [x+sent_no_min for x in peak_x_ls]
    ax.scatter(peak_x_ls, peak_y_ls)
    for i, txt in enumerate(list(peak_x_ls)):
        ax.annotate(f'  Sent #{txt}', (peak_x_ls[i], peak_y_ls[i]), rotation=90, annotation_clip=True)

    # Plot valleys.
    # ax.plot(valley_x + win_half, valley_y, marker='o', linestyle='none', color='red', label="Valleys")
    # readjust starting Sentence No to start with first sentence in segement window
    valley_x_ls = [x+sent_no_min for x in valley_x_ls]
    ax.scatter(valley_x_ls, valley_y_ls)
    for i, txt in enumerate(list(valley_x_ls)):
        ax.annotate(f'Sent #{txt}', (valley_x_ls[i], valley_y_ls[i]), rotation=270, xytext=(valley_x_ls[i], valley_y_ls[i]-4))

    # for i, txt in enumerate(list(valley_x_ls)):
    #     ax.annotate(f'\n\n\nSent No.\n   {txt}', (valley_x_ls[i], valley_y_ls[i]))
    # plt.plot(x, y, 'bo')
    # texts = [plt.text(valley_x_ls[i], valley_y_ls[i], 'Sent No.\n   %s' %valley_x_ls[i], ha='right', va='top') for i in range(len(valley_x_ls))]
    # adjust_text(texts)

    # Confidence Interval (Min/Max Range)
    # plt.fill_between(sentiment_lowess_df['x_value'], sentiment_lowess_df['min'], sentiment_lowess_df['max'], alpha=.3, color='lightskyblue')

    plt.title(f'{CORPUS_FULL}\nRaw Sentence Sentiments with selected Section #{Select_Section_No}')
    plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')

    # locs, labels = xticks()  # Get the current locations and labels.
    # plt.xticks(np.arange(sent_no_min, sent_no_max, step=10))  # Set label locations.

    plt.ylabel(f'Sentiment Value')
    plt.legend(loc='best');
  
  if save2file == True:
    # Save graph to file.
    plt.title(f'{BOOK_TITLE_FULL} \n LOWESS Smoothed Median Sentiment Curve with Crux Points via SciPy.argrelextrema')
    plt.legend(loc='best')
    plt.savefig('argrelextrema.png')

  return crux_coord_ls

In [65]:
def crux_sortsents(crux_ls, top_n=3, get_peaks=True):
  '''
  Given a list of tuples (sent_no, sentiment value), top_n cruxes to retrieve and bool flag get_peaks
  Return a sorted list of peaks/valleys (sentiment_value, sent_no, sent_raw) from greatest down for top_n items
  '''

  crux_old_ls = []
  crux_new_ls = []

  crux_old_ls = sorted(crux_ls, key=lambda tup: (tup[1]), reverse=get_peaks)

  if get_peaks == True:
    crux_old_ls = [x for x in crux_old_ls if x[1] > 0]
  else:
    crux_old_ls = [x for x in crux_old_ls if x[1] < 0]

  # Return only the n_top cruxes if more cruxes than n_top else return all cruxes
  if len(crux_old_ls) >= top_n:
    crux_old_ls = crux_old_ls[:top_n]

  for asent_no, asentiment_val in crux_old_ls:
    asent_raw = str(corpus_sents_df[corpus_sents_df['sent_no'] == asent_no]['sent_raw'].values[0])
    crux_new_ls.append((int(asent_no), float(f'{asentiment_val:.3f}'), str(asent_raw),))

  return crux_new_ls

# Test
# crux_n_top_ls = crux_sortsents(section_crux_ls, top_n=3, get_peaks=True)

In [66]:
def crux_sortsents_report(crux_ls, top_n=3, get_peaks=True, n_sideparags=1):
  '''
  Wrapper function to produce report based upon 'crux_sortsents() described as:
    Given a list of tuples (sent_no, sentiment value), top_n cruxes to retrieve and bool flag get_peaks
    Return a sorted list of peaks/valleys (sentiment_value, sent_no, sent_raw) from greatest down for top_n items
  '''

  crux_n_top_ls = crux_sortsents(crux_ls=crux_ls, top_n=top_n, get_peaks=get_peaks)

  if get_peaks == True:
    crux_label = 'Peak'
  else:
    crux_label = 'Valley'

  print('------------------------------')
  print(f'Section #{Select_Section_No} Top {top_n} {crux_label}s\n')
  for i,crux_sent_tup in enumerate(crux_n_top_ls):
    print(f'   {crux_label} #{i} at Sentence #{crux_sent_tup[0]} with Sentiment Value {crux_sent_tup[1]}')
  # print('------------------------------\n')
  # print('Sent_No  Sentiment   Sentence (Raw Text)\n')
  
  for sent_no, sent_pol, sent_raw in crux_n_top_ls: 
    sent_no = int(sent_no)
    print('\n\n-------------------------------------------------------------')
    print(f'Sentence #{sent_no}   Sentiment: {sent_pol:.3f}\n') #     {sent_raw}\n')
    # print('------------------------------')
    get_sentnocontext_report(the_sent_no=sent_no, the_n_sideparags=n_sideparags, the_sent_highlight=Highlight_Crux_Sentence)
    # get_sentnocontext(sent_no=sent_no, the_n_sideparags=n_sideparags, the_sent_highlight=Highlight_Crux_Sentence)


In [67]:
# For the selected Section, create an expanded Paragraph DataFrame to match the number of Sentences in the Section

def expand_parags2sents(parags_df='corpus_parags_df', sents_df='corpus_sents_df', model_name='vader_lnorm_medianiqr'):
  '''
  Given a Corpus Paragraph DataFrame and a longer Sentence DataFrame that cover the same Section of a Corpus
  Return an expanded version of the Paragraph DataFrame of equal length to the Sentence DataFrame so they can be plotted/compared along the same x-axis
  '''

  parag_sentiment_expanded_ls = []
  parags_midpoint_ls = []
  sent_sum = 0
  parag_start = section_parags_df.parag_no.min()
  print(f'parag_start: {parag_start}')
  parag_end = section_parags_df.parag_no.max() + 1 # shape[0] + 3
  print(f'parag_end: {parag_end}')
  parags_range_ls = list(range(parag_start, parag_end, 1))
  print(f'parags_range_ls: {parags_range_ls}')
  for i, aparag_no in enumerate(parags_range_ls):
    aparag_sentiment_fl = float(corpus_parags_df[corpus_parags_df['parag_no']==aparag_no][model_name])
    sent_ct = len(corpus_sents_df[corpus_sents_df.parag_no == aparag_no])
    parag_midpoint_int = int(sent_ct//2 + sent_sum)
    parags_midpoint_ls.append(parag_midpoint_int)
    for asent in range(sent_ct):
      parag_sentiment_expanded_ls.append(aparag_sentiment_fl)
    sent_sum += sent_ct
    print(f'#{i}: Paragraph #{aparag_no} has {sent_ct} Sentences and Avg Sentiment: {aparag_sentiment_fl:.3f}')

  print(f'\nSentence Total: {sent_sum} vs Original section_sents_df: {section_sents_df.shape[0]}')
  print(f'  Paragraph Sentiment length: {len(parag_sentiment_expanded_ls)}')

  # section_sents_parags_df = section_sents_df.copy()
  
  # section_sents_parags_df.head(1);

  # corpus_sents_df['']

  return parag_sentiment_expanded_ls, parags_midpoint_ls

# Test
# section_sents_df['vader_lnorm_medianiqr_parag'] = expand_parags2sents(parags_df='corpus_parags_df', sents_df='corpus_sents_df')


# **Preprocess and Review Corpus Text (Auto)**

In [68]:
corpus_filename

''

In [69]:
# TODO: Spell check and correct common OCR errors

# SymSpellPy
# JamSpell
# OCR - https://github.com/Alvant/MIL-OCR

In [70]:
# !pip install -U symspellpy

In [71]:
# Did not need these

# dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
# bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")

In [72]:
"""
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# lookup suggestions for single-word input strings
input_term = "memebers"  # misspelling of "members"
input_term = "summermorning"
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST,
                               max_edit_distance=2)
# display suggestion term, term frequency, and edit distance
for suggestion in suggestions:
    print(suggestion)



import pkg_resources
from symspellpy.symspellpy import SymSpell

# Set max_dictionary_edit_distance to avoid spelling correction
sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# a sentence without any spaces
input_term = "thequickbrownfoxjumpsoverthelazydog"
input_term = "summermorning"
result = sym_spell.word_segmentation(input_term)
print("{}, {}, {}".format(result.corrected_string, result.distance_sum,
                          result.log_prob_sum))
""";

In [73]:
# Read corpus into a single string then split into CHAPTERs

# corpus_path = f'{CORPUS_SUBDIR}/{CORPUS_FILENAME}'
# corpus_path = f'./{CORPUS_FILENAME}'

corpus_chaps_preraw_ls  = corpus2chaps(CORPUS_FILENAME)
print(f'We found #{len(corpus_chaps_preraw_ls)} prelen sections\n')

corpus_chaps_raw_ls = [item for item in corpus_chaps_preraw_ls if len(item)>=MIN_CHAP_LEN]

print(f'We found #{len(corpus_chaps_raw_ls)} chapters\n')

print('\nThe first 3 Chapters of the Corpus begin:')
print('-----------------------------------\n')
# corpus_chaps_raw_ls[:10]

print('\nThe last 3 Chapters of the Corpus begin:')
print('-----------------------------------\n')
# corpus_chaps_raw_ls[-10:]
print('\n')

n_shortest = 1
print(f'The {n_shortest} shortest Chapters in the Corpus are:')
print('--------------------------------------------')
temp_ls = sorted(corpus_chaps_raw_ls, key=lambda x: (len(x), x))
"""
for i, asent in enumerate(temp_ls[:n_shortest]):
  print(f'Shortest Section #{i}: {asent}')
"""
print(f'Chapter Count is {len(corpus_chaps_raw_ls)}')

We found #11 prelen sections

We found #10 chapters


The first 3 Chapters of the Corpus begin:
-----------------------------------


The last 3 Chapters of the Corpus begin:
-----------------------------------



The 1 shortest Chapters in the Corpus are:
--------------------------------------------


"\nfor i, asent in enumerate(temp_ls[:n_shortest]):\n  print(f'Shortest Section #{i}: {asent}')\n"

Chapter Count is 10


In [74]:
corpus_chaps_raw_ls[9][:100]

'\n\nOur immediate duty was to introduce Maxfield to the notion that I was not a robot and that I was g'

In [75]:
# Read corpus into a single string then split into Sections (Chapter or sub-chapter divisions)

# corpus_path = f'{CORPUS_SUBDIR}/{CORPUS_FILENAME}'
# corpus_path = f'./{CORPUS_FILENAME}'

corpus_sects_preraw_ls  = corpus2sects(CORPUS_FILENAME)
print(f'We found #{len(corpus_sects_preraw_ls)} prelen sections\n')

'''
corpus_sects_raw_ls = []
for i,asect in enumerate(corpus_sects_preraw_ls):
  if len(asect) > MIN_SECT_LEN:
    print(f'Including Section #{i}: Length is {len(asect)}')
    corpus_sects_raw_ls.append(asect)
  else:
    print(f'EXCLUDING Section #{i}: Length is {len(asect)}')
'''

corpus_sects_raw_ls = [item for item in corpus_sects_preraw_ls if len(item)>=MIN_SECT_LEN]

print(f'We found #{len(corpus_sects_raw_ls)} sections\n')

print('\nThe first 3 Sections of the Corpus:')
print('-----------------------------------\n')
# corpus_sects_raw_ls[:10]

print('\nThe last 3 Sections of the Corpus:')
print('-----------------------------------\n')
# corpus_sects_raw_ls[-10:]
print('\n')

n_shortest = 2
print(f'The {n_shortest} shortest Sections in the Corpus are:')
print('--------------------------------------------')
temp_ls = sorted(corpus_sects_raw_ls, key=lambda x: (len(x), x))
for i, asent in enumerate(temp_ls[:n_shortest]):
  print(f'Shortest Section #{i}: {asent}')

print(f'Section Count is {len(corpus_sects_raw_ls)}')

We found #31 prelen sections



"\ncorpus_sects_raw_ls = []\nfor i,asect in enumerate(corpus_sects_preraw_ls):\n  if len(asect) > MIN_SECT_LEN:\n    print(f'Including Section #{i}: Length is {len(asect)}')\n    corpus_sects_raw_ls.append(asect)\n  else:\n    print(f'EXCLUDING Section #{i}: Length is {len(asect)}')\n"

We found #30 sections


The first 3 Sections of the Corpus:
-----------------------------------


The last 3 Sections of the Corpus:
-----------------------------------



The 2 shortest Sections in the Corpus are:
--------------------------------------------
Shortest Section #0: 

When I entered the kitchen in the morning, later than usual, Adam's eyes were open. They were pale blue, flecked with minuscule vertical rods of black. The eyelashes were long and thick, like a child's. But his blink mechanism had not yet kicked in. It was set at irregular intervals and adjusted for mood and gestures, and primed to react to the actions and speech of others. Reluctantly, I would read the handbook into the night. He was equipped with a blink reflex to protect his eyes from flying objects. At present, his gaze was empty of meaning or intent and therefore unaffecting, as lifeless as the stare of a shop-window mannequin. So far, he was showing none of the fractional movements that warmly typify t

In [76]:
# Read corpus into a single string then split into paragraphs

corpus_parags_raw_ls = corpus2parags(CORPUS_FILENAME)
print(f'We found #{len(corpus_parags_raw_ls)} paragraphs\n')

print('\nThe first 10 Paragraphs of the Corpus:')
print('-----------------------------------\n')
corpus_parags_raw_ls[:10]

print('\nThe last 10 Paragraphs of the Corpus:')
print('-----------------------------------\n')
corpus_parags_raw_ls[-10:]
print('\n')

n_shortest = 10
print(f'The {n_shortest} shortest Paragraphs in the Corpus are:')
print('--------------------------------------------')
temp_ls = sorted(corpus_parags_raw_ls, key=lambda x: (len(x), x))
for i, asent in enumerate(temp_ls[:n_shortest]):
  print(f'Shortest Paragraph #{i}: {asent}')

Corpus Paragraph Raw Count: 1455
Corpus Paragraph -(whitespace only) Count: 1453
Corpus Paragraph -(punctuation only) Count: 1428
We found #1428 paragraphs


The first 10 Paragraphs of the Corpus:
-----------------------------------



['But remember, please, the Law by which we live, We are not built to comprehend a lie ...',
 '--Rudyard Kipling, "The Secret of the Machines"',
 'CHAPTER 1',
 'It was religious yearning granted hope, it was the holy grail of science. Our ambitions ran high and low--for a creation myth made real, for a monstrous act of self-love. As soon as it was feasible, we had no choice but to follow our desires and hang the consequences. In loftiest terms, we aimed to escape our mortality, confront or even replace the Godhead with a perfect self. More practically, we intended to devise an improved, more modern version of ourselves and exult in the joy of invention, the thrill of mastery. In the autumn of the twentieth century, it came about at last, the first step towards the fulfilment of an ancient dream, the beginning of the long lesson we would teach ourselves that however complicated we were, however faulty and difficult to describe in even our simplest actions and modes of being, we could be


The last 10 Paragraphs of the Corpus:
-----------------------------------



['"So--knowing not much about the mind, you want to embody an artificial one in social life. Machine learning can only take you so far. you will need to give this mind some rules to live by. How about a prohibition against lying? According to the Old Testament--Proverbs, I think--it is an abomination to God. But social life teems with harmless or even helpful untruths. How do we separate them out? Who is going to write the algorithm for the little white lie that spares the blushes of a friend? Or the lie that sends a rapist to prison who\'d otherwise go free? We do not yet know how to teach machines to lie. And what about revenge? Permissible sometimes, according to you, if you love the person who is exacting it. Never, according to your Adam."',
 'He paused and looked away from me again. From his profile, not only from his tone, I sensed a change was coming and my pulse was suddenly heavy. I could hear it in my ears. He proceeded calmly.',
 '"My hope is that one day, what you did to A



The 10 shortest Paragraphs in the Corpus are:
--------------------------------------------
Shortest Paragraph #0: "And?"
Shortest Paragraph #1: "Wee."
Shortest Paragraph #2: "Why?"
Shortest Paragraph #3: "Yes!"
Shortest Paragraph #4: "Yes."
Shortest Paragraph #5: "Yes."
Shortest Paragraph #6: "Yes."
Shortest Paragraph #7: "Yes."
Shortest Paragraph #8: "Yes."
Shortest Paragraph #9: "Yes."


In [77]:
# Tokenize Paragraphs into Sentences

'''
sent_no = 0
# sent_base = 0
corpus_sents_row_ls = []
for parag_no,aparag in enumerate(corpus_parags_raw_ls):
  sents_ls = sent_tokenize(aparag)
  # Delete (whitespace only) sentences
  sents_ls = [x.strip() for x in sents_ls if len(x.strip()) > MIN_SENT_LEN]
  # print(f'Corpus Sentences -(whitespace only) Count: {len(sents_ls)}')
  # Delete (punctuation only) sentences
  sents_ls = [x for x in sents_ls if len((re.sub(r'[^\w\s]','',x)).strip()) > MIN_SENT_LEN]
  # print(f'Corpus Sentences -(punctuation only) Count: {len(sents_ls)}')
  # sent_no = sent_base
  for s,asent in enumerate(sents_ls):
    corpus_sents_row_ls.append([sent_no, parag_no, asent])
    sent_no += 1
  # sent_base = sent_no 


print(f'{len(corpus_sents_row_ls)}')

print(f'First row {corpus_sents_row_ls[0]}')
print('\n')
print(f'Last row {corpus_sents_row_ls[-1]}')
'''

corpus_sents_row_ls = parag2sents(corpus_parags_raw_ls)
print(f'{len(corpus_sents_row_ls)}')

print(f'First row {corpus_sents_row_ls[0]}')
print('\n')
print(f'Last row {corpus_sents_row_ls[-1]}');

7681
First row [0, 0, 'But remember, please, the Law by which we live, We are not built to comprehend a lie ...']


Last row [7680, 1427, 'I ran along the empty corridor, found the emergency stairs, took them two at a time down into the street and set off on my journey southwards across London towards my troubled home.']


In [78]:
# Create Corpus Sentence DataFrame

corpus_sents_df = pd.DataFrame(corpus_sents_row_ls)
corpus_sents_df.columns = ['sent_no', 'parag_no', 'sent_raw']
corpus_sents_df['sent_raw'] = corpus_sents_df['sent_raw'].astype('string')
# Double check to drop any rows where raw Sentence is NaN or empty string ''
corpus_sents_df.dropna(subset=['sent_raw'], inplace=True)


print(f'First 10 Sentences of {CORPUS_FULL}')
corpus_sents_df.head(10)
corpus_sents_df.info()

First 10 Sentences of Machines Like Me by: Ian McEwan


,sent_no,parag_no,sent_raw
0,0,0,"But remember, please, the Law by which we live..."
1,1,1,"--Rudyard Kipling, ""The Secret of the Machines"""
2,2,2,CHAPTER 1
3,3,3,"It was religious yearning granted hope, it was..."
4,4,3,Our ambitions ran high and low--for a creation...
5,5,3,"As soon as it was feasible, we had no choice b..."
6,6,3,"In loftiest terms, we aimed to escape our mort..."
7,7,3,"More practically, we intended to devise an imp..."
8,8,3,"In the autumn of the twentieth century, it cam..."
9,9,3,"And I was there as a young man, an early and e..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7681 entries, 0 to 7680
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sent_no   7681 non-null   int64 
 1   parag_no  7681 non-null   int64 
 2   sent_raw  7681 non-null   string
dtypes: int64(2), string(1)
memory usage: 240.0 KB


In [79]:
corpus_sents_df[corpus_sents_df['sent_raw'].str.contains('summer')]

,sent_no,parag_no,sent_raw
1400,1400,235,"I went into my bedroom, undressed, leaving my ..."
1652,1652,272,"On the windowsill, where yellow gingham curtai..."
1792,1792,292,"For me, it was far too early for bed and it wa..."
2095,2095,344,"Here she was beside me, close enough for me to..."
2613,2613,486,The summer was hot and something was coming to...
2952,2952,559,If the court would suspend proceedings and ser...
3698,3698,714,Our last summer at school came around.
3866,3866,725,I would had two boyfriends by that summer and ...
4410,4410,801,"Now she was writing a short essay, to be read ..."
4676,4676,850,I wanted to lie down on the worn-out grass of ...


In [80]:
# Create Corpus Paragraph DataFrame

parag_no_ls = []
parag_raw_ls = []

corpus_parags_df = pd.DataFrame()

for i, aparag in enumerate(corpus_parags_raw_ls):
  parag_no_ls.append(i)
  parag_raw_ls.append(aparag)

corpus_parags_df = pd.DataFrame(
    {'parag_no': parag_no_ls,
     'parag_raw': parag_raw_ls,
    })

# Test 
print(f'First 10 Paragraphs of {CORPUS_FULL}')
corpus_parags_df.head(10)
corpus_parags_df.info()

First 10 Paragraphs of Machines Like Me by: Ian McEwan


,parag_no,parag_raw
0,0,"But remember, please, the Law by which we live..."
1,1,"--Rudyard Kipling, ""The Secret of the Machines"""
2,2,CHAPTER 1
3,3,"It was religious yearning granted hope, it was..."
4,4,But artificial humans were a cliche long befor...
5,5,"I was among the optimists, blessed by unexpect..."
6,6,"Twelve of this first edition were called Adam,..."
7,7,"At last, with cardboard and polystyrene wrappi..."
8,8,"Adam was not a sex toy. However, he was capabl..."
9,9,"He was advertised as a companion, an intellect..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   parag_no   1428 non-null   int64 
 1   parag_raw  1428 non-null   object
dtypes: int64(1), object(1)
memory usage: 22.4+ KB


In [81]:
# Create Corpus Section DataFrame

sect_no_ls = []
sect_raw_ls = []

# corpus_sects_df = pd.DataFrame()

for i, asect in enumerate(corpus_sects_raw_ls):
  sect_no_ls.append(i)
  sect_raw_ls.append(asect)


corpus_sects_df = pd.DataFrame(
    {'sect_no': sect_no_ls,
     'sect_raw': sect_raw_ls,
    })


# Calculate the sentence number at the mid-point of each Section

sect_mid_sentnos_ls = []
sect_start_sentnos_ls = []
sect_sentno_base = 0
for i, sect_text in enumerate(corpus_sects_df.sect_raw):
  if len(sect_text) > MIN_SECT_LEN:
    sect_sents_ls = sent_tokenize(sect_text)
    # Calc and save the sent_no that begins each Section
    sect_first_sent = sect_sents_ls[0].strip()
    # print(f'Searching for first sentence: {sect_first_sent}')
    sect_start_sentnos_ls.append(int(corpus_sents_df[corpus_sents_df['sent_raw'].str.contains(sect_first_sent)]['sent_no']))
    # Calc and save the sent_no in the middle of each Section
    sect_sents_len = len(sect_sents_ls)
    sect_mid_sentno = int(sect_sents_len/2) + sect_sentno_base
    # print(f'Section #{i}: {len(sect_sents_ls)} Sentences, midpoint: {sect_mid_sentno}, cumulative midpoint: {sect_mid_sentno}')
    sect_mid_sentnos_ls.append(sect_mid_sentno)
    sect_sentno_base += sect_sents_len

corpus_sects_df['sent_no_start'] = pd.Series(sect_start_sentnos_ls)
corpus_sects_df['sent_no_mid'] = pd.Series(sect_mid_sentnos_ls)

# Test 
print(f'First 2 Sections of {CORPUS_FULL}')
# corpus_sects_df.head(2)
corpus_sects_df.info()

First 2 Sections of Machines Like Me by: Ian McEwan
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   sect_no        30 non-null     int64 
 1   sect_raw       30 non-null     object
 2   sent_no_start  30 non-null     int64 
 3   sent_no_mid    30 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.1+ KB


In [82]:
# Create Corpus Chapter DataFrame

chap_no_ls = []
chap_raw_ls = []

# corpus_chaps_df = pd.DataFrame()

for i, achap in enumerate(corpus_chaps_raw_ls):
  chap_no_ls.append(i)
  chap_raw_ls.append(achap)


corpus_chaps_df = pd.DataFrame(
    {'chap_no': chap_no_ls,
     'chap_raw': chap_raw_ls,
    })


# Calculate the sentence number at the mid-point of each Chapter

chap_mid_sentnos_ls = []
chap_start_sentnos_ls = []
chap_sentno_base = 0
for i, chap_text in enumerate(corpus_chaps_df.chap_raw):
  if len(chap_text) > MIN_CHAP_LEN:
    chap_sents_ls = sent_tokenize(chap_text)
    # Calc and save the sent_no that begins each Chapter
    chap_first_sent = chap_sents_ls[0].strip()
    # print(f'Searching for first sentence: {chap_first_sent}')
    chap_start_sentnos_ls.append(int(corpus_sents_df[corpus_sents_df['sent_raw'].str.contains(chap_first_sent)]['sent_no']))
    # Calc and save the sent_no in the middle of each Chapter
    chap_sents_len = len(chap_sents_ls)
    chap_mid_sentno = int(chap_sents_len/2) + chap_sentno_base
    # print(f'Chapter #{i}: {len(chap_sents_ls)} Sentences, midpoint: {chap_mid_sentno}, cumulative midpoint: {chap_mid_sentno}')
    chap_mid_sentnos_ls.append(chap_mid_sentno)
    chap_sentno_base += chap_sents_len

corpus_chaps_df['sent_no_start'] = pd.Series(chap_start_sentnos_ls)
corpus_chaps_df['sent_no_mid'] = pd.Series(chap_mid_sentnos_ls)

# Test 
print(f'First 2 Chapters of {CORPUS_FULL}')
# corpus_chaps_df.head(2)
corpus_chaps_df.info()

First 2 Chapters of Machines Like Me by: Ian McEwan
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   chap_no        10 non-null     int64 
 1   chap_raw       10 non-null     object
 2   sent_no_start  10 non-null     int64 
 3   sent_no_mid    10 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 448.0+ bytes


In [83]:
print(corpus_sents_df.iloc[237]['sent_raw'])

At thirty-two, I was completely broke.


In [84]:
corpus_sects_df.head(2)

,sect_no,sect_raw,sent_no_start,sent_no_mid
0,0,"\n\nIt was religious yearning granted hope, it...",3,117
1,1,"\n\nAt thirty-two, I was completely broke. Was...",237,300


In [85]:
# Raw Plot of Section Sentiments (Adjusted for (x-axis) mid-Section Sentence No and (y-axis) Sentiment weighted by Section length )
# This function must be defined AFTER the corpus_sect_df DataFrame is created in the previous block

# corpus_sects_df = pd.DataFrame()  # Create empty early as required by some utility functions
""" 
# SECOND COPY

def plot_stand_crux_sections(ts_df=corpus_sects_df, model_names_ls=['vader'], semantic_type='section', label_token_ct=0, title_xpos = 0.8, title_ypos=0.2, save2file=False):
  '''
  Given a Sections DataFrame, model_name and semantic type,
  Return a Plot of the Cruxes
  '''

  crux_points_dt = {}
  model_stand_names_ls = []
  section_boundries_ls = []

  for i, amodel in enumerate(model_names_ls):
    amodel_stand_str = f'{amodel}_stand'
    model_stand_names_ls.append(amodel_stand_str)
    # sns.lineplot(data=ts_df, x='sent_no_mid', y=amodel_stand, markers=['o'], alpha=0.5, label=amodel_stand); # .set_title(f'{CORPUS_FULL} \n Plot Section Sentiment (Bing Lexicon)')

  # Draw vertical lines at Section boundries
  section_boundries_ls = list(corpus_sects_df['sent_no_start'])
  for i, sent_no in enumerate(section_boundries_ls):
    plt.axvline(sent_no, color='blue', alpha=0.1);

  # Get midpoints of each Section
  section_midpoints_ls = list(corpus_sects_df['sent_no_mid']);

  # How many sentiment time series are we plotting?
  if len(model_stand_names_ls) == 1:
    # Plotting only one model
    model_names_str = model_names_ls[0]
    sns.lineplot(data=ts_df, x='sent_no_mid', y=model_names_str, markers=['o'], alpha=0.5, label=model_stand_names_ls[0]) # .set_title(f'{CORPUS_FULL} \n Plot Section Sentiment and Cruxes (Model: {models_names_ls[0].capitalize()})')

    # If plotting only one model, add labels
    section_midpoints_sentiment_ls = list(corpus_sects_df[amodel_stand]);
    for x,y in zip(section_midpoints_ls, section_midpoints_sentiment_ls):
      label_token_int = int(label_token_ct)
      if label_token_int < 0:
        label = ''
      elif label_token_int == 0:
        label = f"#{x}"
      else:
        label = f"#{x} {' '.join(corpus_sents_df.iloc[x]['sent_raw'].split()[:label_token_int])}"; # \nPolarity: {y:.2f}'

      crux_full_str = ' '.join(corpus_sents_df.iloc[x]['sent_raw'].split())
      crux_points_dt[x] = (y, crux_full_str)

      plt.annotate(label,
                   (x,y),
                   textcoords='offset points',
                   xytext=(0,10),
                   ha='center',
                   rotation=90)
      
    plt.title(f'{CORPUS_FULL} \n Plot Section Sentiment (Standardized {model_names_ls[0].capitalize()})', x=title_xpos, y=title_ypos);
    plt.plot(section_midpoints_ls, section_midpoints_sentiment_ls, marker="o", ms=6) # , markevery=[0,1])

  else:
    # If plotting multiple models
    model_names_str = 'multilex'
    for i, amodel_stand_name in enumerate(model_stand_names_ls):
      sns.lineplot(data=ts_df, x='sent_no_mid', y=amodel_stand_name, markers=['o'], alpha=0.5, label=amodel_stand_name) # .set_title(f'{CORPUS_FULL} \n Plot Section Sentiment and Cruxes (Model: {models_names_ls[0].capitalize()})')

    plt.title(f'{CORPUS_FULL} \n Plot Section Sentiment (Standardized All Lex Models)', x=title_xpos, y=title_ypos)

  plt.legend(loc='best');

  if (save2file == True):
    # Save graph to file.
    plot_filename = f'plot_standnote_sects_{model_names_str}.png'
    plotpathfilename_str = gen_pathfiletime(plot_filename)
    plt.savefig(plotpathfilename_str, format='png', dpi=300)
    print(f'Plot saved: {plot_filename}');

  return crux_points_dt
""";

In [86]:
# TODO: More General Cleanup

In [87]:
# TODO: Normalize Paragraphs by Lengths (Smart Aggregate/Split)

In [88]:
'''
# Generate full path and timestamp for new filepath/filename

def gen_pathfiletime(file_str, subdir_str=''):

  # Geenreate compressed author and title substrings
  author_raw_str = ''.join(CORPUS_AUTHOR.split()).lower()
  title_raw_str = ''.join(CORPUS_TITLE.split()).lower()

  # Generate current/unique datetime string
  datetime_str = str(datetime.now().strftime('%Y%m%d%H%M%S'))

  # Built fullpath+filename string
  file_base, file_ext = file_str.split('.')

  author_str = re.sub('[^A-Za-z0-9]+', '', author_raw_str)
  title_str = re.sub('[^A-Za-z0-9]+', '', title_raw_str)

  full_filepath_str = f'{subdir_str}{file_base}_{author_str}_{title_str}_{datetime_str}.{file_ext}'

  # print(f'Returning from gen_savepath() with full_filepath={full_filepath}')

  return full_filepath_str

# Test
# pathfilename_str = gen_pathfiletime('hist_paraglen.png')
# print(pathfilename_str)
''';

In [89]:
corpus_sects_df.columns

Index(['sect_no', 'sect_raw', 'sent_no_start', 'sent_no_mid'], dtype='object')

In [90]:
# Calculate some char/token metrics and do some EDA on them

# This code must be run AFTER corpus_sects_df is created below

corpus_sents_df['char_len'] = corpus_sents_df['sent_raw'].apply(lambda x: len(x))
corpus_sents_df['token_len'] = corpus_sents_df['sent_raw'].apply(lambda x: len(x.split())) 

corpus_parags_df['char_len'] = corpus_parags_df['parag_raw'].apply(lambda x: len(x))
corpus_parags_df['token_len'] = corpus_parags_df['parag_raw'].apply(lambda x: len(x.split())) 

corpus_sects_df['char_len'] = corpus_sects_df['sect_raw'].apply(lambda x: len(x))
corpus_sects_df['token_len'] = corpus_sects_df['sect_raw'].apply(lambda x: len(x.split())) 

corpus_chaps_df['char_len'] = corpus_chaps_df['chap_raw'].apply(lambda x: len(x))
corpus_chaps_df['token_len'] = corpus_chaps_df['chap_raw'].apply(lambda x: len(x.split())) 

# corpus_sents_df.head()


In [91]:
# Default cleaned raw text

# Sentences
# Let's take a look at the updated text
corpus_sents_df['sent_clean'] = corpus_sents_df['sent_raw'].apply(lambda x: text_clean(x))
# Ensure to drop all Sentences with NaN or '' Raw Text
corpus_sents_df.replace("", np.nan, regex=True, inplace=True)
corpus_sents_df.dropna(how='any', axis=0, subset=['sent_raw'], inplace=True)
corpus_sents_df.dropna(how='any', axis=0, subset=['sent_clean'], inplace=True)

print('\nCompare Raw and Cleaned Sentences:')
print('--------------------------------------')
corpus_sents_df.head(2)


# Paragraphs
# Let's take a look at the updated text
corpus_parags_df['parag_clean'] = corpus_parags_df['parag_raw'].apply(lambda x: text_clean(x))
# Ensure to drop all Sentences with NaN or '' Raw Text
corpus_parags_df.replace("", np.nan, regex=True, inplace=True)
corpus_parags_df.dropna(how='any', axis=0, subset=['parag_raw'], inplace=True)

print('\nCompare Raw and Cleaned Paragraphs:')
print('--------------------------------------')
corpus_parags_df.head(2)


# Sections
# Let's take a look at the updated text
corpus_sects_df['sect_clean'] = corpus_sects_df['sect_raw'].apply(lambda x: text_clean(x))
# Ensure to drop all Sentences with NaN or '' Raw Text
corpus_sects_df.replace("", np.nan, regex=True, inplace=True)
corpus_sects_df.dropna(how='any', axis=0, subset=['sect_raw'], inplace=True)

print('\nCompare Raw and Cleaned Sections:')
print('--------------------------------------')
# corpus_sects_df.head(2)


# Chapters
# Let's take a look at the updated text
corpus_chaps_df['chap_clean'] = corpus_chaps_df['chap_raw'].apply(lambda x: text_clean(x))
# Ensure to drop all Chapters with NaN or '' Raw Text
corpus_chaps_df.replace("", np.nan, regex=True, inplace=True)
corpus_chaps_df.dropna(how='any', axis=0, subset=['chap_raw'], inplace=True)

print('\nCompare Raw and Cleaned Chapters:')
print('--------------------------------------')
# corpus_sects_df.head(2)


Compare Raw and Cleaned Sentences:
--------------------------------------


,sent_no,parag_no,sent_raw,char_len,token_len,sent_clean
0,0,0,"But remember, please, the Law by which we live...",88,18,but remember please the law by which we live w...
1,1,1,"--Rudyard Kipling, ""The Secret of the Machines""",47,7,rudyard kipling the secret of the machines



Compare Raw and Cleaned Paragraphs:
--------------------------------------


,parag_no,parag_raw,char_len,token_len,parag_clean
0,0,"But remember, please, the Law by which we live...",88,18,but remember please the law by which we live w...
1,1,"--Rudyard Kipling, ""The Secret of the Machines""",47,7,rudyard kipling the secret of the machines



Compare Raw and Cleaned Sections:
--------------------------------------

Compare Raw and Cleaned Chapters:
--------------------------------------


In [92]:
# Verify 

print(f'corpus_sents_df.shape: {corpus_sents_df.shape}')
print(f'corpus_parags_df.shape: {corpus_parags_df.shape}')
print(f'corpus_sects_df.shape: {corpus_sects_df.shape}')
print(f'corpus_chaps_df.shape: {corpus_chaps_df.shape}')

corpus_sents_df.shape: (7679, 6)
corpus_parags_df.shape: (1428, 5)
corpus_sects_df.shape: (30, 7)
corpus_chaps_df.shape: (10, 7)


In [93]:
corpus_sents_df[corpus_sents_df['sent_clean'].str.contains('destructionsmashed')]

,sent_no,parag_no,sent_raw,char_len,token_len,sent_clean


In [94]:
corpus_sents_df[corpus_sents_df['sent_clean'].str.contains('summer')]

,sent_no,parag_no,sent_raw,char_len,token_len,sent_clean
1400,1400,235,"I went into my bedroom, undressed, leaving my ...",118,23,i went into my bedroom undressed leaving my cl...
1652,1652,272,"On the windowsill, where yellow gingham curtai...",167,26,on the windowsill where yellow gingham curtain...
1792,1792,292,"For me, it was far too early for bed and it wa...",88,19,for me it was far too early for bed and it was...
2095,2095,344,"Here she was beside me, close enough for me to...",83,15,here she was beside me close enough for me to ...
2613,2613,486,The summer was hot and something was coming to...,56,11,the summer was hot and something was coming to...
2952,2952,559,If the court would suspend proceedings and ser...,202,35,if the court would suspend proceedings and ser...
3698,3698,714,Our last summer at school came around.,38,7,our last summer at school came around
3866,3866,725,I would had two boyfriends by that summer and ...,64,14,i would had two boyfriends by that summer and ...
4410,4410,801,"Now she was writing a short essay, to be read ...",145,25,now she was writing a short essay to be read a...
4676,4676,850,I wanted to lie down on the worn-out grass of ...,76,16,i wanted to lie down on the worn out grass of ...


##**Save Preprocess Corpus DataFrames**

In [95]:
# Save Preprocessed Corpus Sentences DataFrame

author_str = ''.join(CORPUS_AUTHOR.split()).lower()
title_str = ''.join(CORPUS_TITLE.split()).lower()
datetime_now = datetime.utcnow().strftime("%Y%m%d_%H%M")

# Sentences
corpus_sents_filename = f'corpus_sents_clean_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving to file: {corpus_sents_filename}')

corpus_sents_df.to_csv(corpus_sents_filename)

# Paragraphs
corpus_parags_filename = f'corpus_parags_clean_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving to file: {corpus_parags_filename}')

corpus_parags_df.to_csv(corpus_parags_filename)

# Sections
corpus_sects_filename = f'corpus_sects_clean_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving to file: {corpus_sects_filename}')

corpus_sects_df.to_csv(corpus_sects_filename)

# Chapters
corpus_chaps_filename = f'corpus_chaps_clean_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving to file: {corpus_chaps_filename}')

corpus_chaps_df.to_csv(corpus_chaps_filename)

Saving to file: corpus_sents_clean_ianmcewan_machineslikeme_20210708_1508.csv
Saving to file: corpus_parags_clean_ianmcewan_machineslikeme_20210708_1508.csv
Saving to file: corpus_sects_clean_ianmcewan_machineslikeme_20210708_1508.csv
Saving to file: corpus_chaps_clean_ianmcewan_machineslikeme_20210708_1508.csv


# (Optional) EDA Raw Text Features: Interactive

**(Optional) Can Skip Ahead to: 'EDA of Raw Text and Extracted Features'**

In [ ]:
# Review Cleaned Up Sentences

corpus_sents_df.head(5)

In [ ]:
# Summary Statistics

corpus_sents_df.describe()
corpus_sents_df['token_len'].value_counts()

In [ ]:
# Create histogram of Paragraph lengths

sns.histplot(data=corpus_sents_df['char_len'], kde=True).set_title(f'{CORPUS_FULL} \n Histogram of Paragraph Lengths');

if (PLOT_OUTPUT == 'All'):
  # Save graph to file.
  plot_filename = 'hist_paraglen.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

In [ ]:
# Plot histogram of Sentence lengths

sns.histplot(data=corpus_sents_df['token_len'], kde=True).set_title(f'{CORPUS_FULL} \n Histogram of Sentence Lengths')

if (PLOT_OUTPUT == 'All'):
  # Save graph to file.
  plot_filename = 'hist_sentlen.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

In [ ]:
# SELECT CORPUS TYPE
# TODO: Customized Preprocessing (e.g. Tweets) by Corpus Type

# Novel, Tweets, Chat Transcript

# Processing Options

# Apply first level cleaning

# (Optional) Manually Create Sentiment Arc Baseline

***Can skip to Section [Load Sentiment Polarities...] or [Calculate VADER...]***

**Interactively Enter Cruxes and Edge Cases**

In [ ]:
# Setup data structures for endpoints of Sentiment Time Series

#   [-1.0 to +1.0] = [v.neg, neg, neutral, pos, v.pos]

corpus_man_crux_ols = []  # working datastructure to dynamically build ordered list of manually selected Crux Points
corpus_man_cruxes_odt = OrderedDict() # Once all manual Crux points selected, this will be working data structure

corpus_sents_len = corpus_sents_df.shape[0] - 1

corpus_parags_len = corpus_sents_df.parag_no.max() # make sure no omissions/repeats/skips


In [ ]:
# <INPUT> Set the Begining and Ending Sentiment Values (Manual Versions)

# Start of Corpus Sentiment Analysis Time Series
Corpus_Starting_Sentiment = -0.1 #@param {type:"slider", min:-1, max:1, step:0.1}
# corpus_sa_begin = Corpus_Starting_Sentiment

# End of Corpus Sentiment Analysis Time Series
Corpus_Ending_Sentiment = -1 #@param {type:"slider", min:-1, max:1, step:0.1}
# corpus_sa_end = Corpus_Ending_Sentiment

corpus_man_crux_ols = [tuple((0, Corpus_Starting_Sentiment)), tuple((corpus_sents_len, Corpus_Ending_Sentiment))]
# corpus_man_cruxes_dt[0.] = corpus_sa_begin
# corpus_man_cruxes_dt[float(corpus_sents_len)] = corpus_sa_end

print(f'Manual Cruxes with Start/End: {corpus_man_crux_ols}')

**Seach for Key Words that suggest Min/Max Sentiment Crux**
* Specific to the Novel: Introduction of Pivotal Character, Scene, Factual Reveal, McGuffin, etc...
* General to Events/Themes: Death, Birth, Fight, Accident, Money, Sex, etc... 

In [ ]:
# <INPUT> Search Corpus for Line No of Peaks/Valleys
# TODO: Better Vis
Search_String = "Death" #@param {type:"string"}
if (Search_String == ""):
  search_str = "accident"
else:
  search_str = Search_String.lower()

# search the list of cleaned paragraphs
# results_ls = [x for x in search_match_ls if re.search(subs, x)]

# creating and passsing series to new column
match_sents_ser = corpus_sents_df["sent_clean"].str.find(search_str)

# print(f'Found #{len(match_index>0)} Matches')
match_sents_df = corpus_sents_df.loc[match_sents_ser > 0]
print(f'Found #{match_sents_df.shape[0]} Matching Sentences')
print('------------------------------------')
# print(f'  {match_sents_df}')
match_sents_df[['sent_no', 'parag_no', 'sent_raw', 'token_len']]

**Get Context for Matched Sentence by Retrieving Surrounding Paragraph**

In [ ]:
# Extract Surrounding Paragraphs for context on matching Sentences

def get_parag4sentno(asent_no):
  '''
  Return the original raw paragraph containing a 
  given sentence number.
  '''
  # parag_df = pd.DataFrame()
  # print(f'Passed in sent_no: {asent_no}')
  aparag_no = int(corpus_sents_df.loc[corpus_sents_df['sent_no'] == asent_no]['parag_no'])
  # print(f'  This sent_no {asent_no} is in parag_no: {aparag_no}')
  aparag_str = corpus_sents_df.loc[corpus_sents_df['parag_no'] == aparag_no]['sent_raw'].str.cat() # ['sent_clean']
  # sentno_parag_df = corpus_sents_df[corpus_sents_df['sent_no']==asent_no]
  # print(f'Sent #{asent_no} is in the paragraph: ')
  # print(aparag)
  # print(f'returning aparag_no: [{aparag_no}]: {aparag}')
  return aparag_no, aparag_str

'''
# Testing
asent_no = 7
print(f'Searching for paragraph containing Sentence #{asent_no}')

aparag_no, aparag_str = get_parag4sentno(asent_no)
print(f'\n  Found in Paragraph #{aparag_no} \n\n{aparag_str}')
''';

In [ ]:
# Extract Surrounding Paragraphs for context on matching Sentences

def get_parag_str(aparag_no):
  '''
  Return the original raw paragraph containing a 
  given sentence number.
  '''
  # parag_df = pd.DataFrame()
  # print(f'Passed in sent_no: {asent_no}')
  # aparag_no = int(corpus_sents_df.loc[corpus_sents_df['sent_no'] == asent_no]['parag_no'])
  # print(f'  This sent_no {asent_no} is in parag_no: {aparag_no}')
  aparag_str = corpus_sents_df.loc[corpus_sents_df['parag_no'] == aparag_no]['sent_raw'].str.cat() # ['sent_clean']
  # sentno_parag_df = corpus_sents_df[corpus_sents_df['sent_no']==asent_no]
  # print(f'Sent #{asent_no} is in the paragraph: ')
  # print(aparag)
  # print(f'returning aparag_no: [{aparag_no}]: {aparag}')
  return aparag_str

In [ ]:
# Summarize current status of manually selected Crux Points
# TODO:

def crux_sum_short():
  print(f'\nOrdered list of all manually selected Crux Points')
  print('---------------------------------------')
  for i, acrux_tp in enumerate(corpus_man_crux_ols):
    asent_no, asent_pol = acrux_tp
    asent_str = corpus_sents_df[corpus_sents_df.sent_no==asent_no].sent_raw.str.cat()
    # print(f'Type: {type(asent_str)}')
    print(f'Sent No {asent_no:4d}: Polarity: {asent_pol}\nText: {asent_str}\n\n')

In [ ]:
# Summarize current manually selected Crux Points

def crux_summary():
  print(f'\nOrdered list of all manually selected Crux Points')
  print('---------------------------------------\n\n')
  for i, acrux_tp in enumerate(corpus_man_crux_ols):
    asent_no, asent_pol = acrux_tp
    asent_str = corpus_sents_df[corpus_sents_df.sent_no==asent_no].sent_raw.str.cat()
    # print(f'Type: {type(asent_str)}')
    print(f'Sent No {asent_no:4d}: Polarity: {asent_pol}')
    print('------------------------------')
    print(f'Text: {asent_str}\n\n')

In [ ]:
# View the Paragraph containing your Matching Sentence:

def get_nparags_context(crux_sent_no, parag_ct):

  parag_win = int(parag_ct)
  parag_crux_str = ''

  parag_crux_no = 0

  if (crux_sent_no < 0) | (crux_sent_no > corpus_sents_len):
    print(f'ERROR: Pick a Sentence No between 0-{corpus_sents_len-1}')
  else:
    # get_sent_no = crux_sent_no
    # print(f'Retrieving Sentence No: {get_sent_no}')
    # print('----------')

    parag_crux_no, aparag_str = get_parag4sentno(crux_sent_no)
    if parag_win == 1:
      print(f'Match #{i}: Sentence No. {asent_no} found in Paragraph No. {parag_crux_no}')
      print('----------------------------')
      print(f'Sentence:\n')
      # print(f'     {corpus_sents_df[corpus_sents_df.sent_no == crux_sent_no]}\n\n')
      corpus_sents_df[corpus_sents_df.sent_no == crux_sent_no]
      print('----------------------------')
      print(f'Paragraph Context:\n')
      print(f'     {aparag_str}\n\n')
    else:
      parag_half_win = int((parag_win-1)/2)
      parag_start = parag_crux_no - parag_half_win
      parag_end = parag_crux_no + parag_half_win
      print(f'Retrieving {parag_ct} Contextual Paragraphs Nos {parag_start} to {parag_end}')
      print(f'  for Crux Point centered on Sentence No {crux_sent_no}')
      for i in range(parag_start, parag_end + 1, 1):
        if i == parag_crux_no:
          print(f'\n   ---------------------------------------------------------')
          print(f'** Crux Point Paragraph #{i} with Sentence No. {crux_sent_no} **')
          print(f'   ---------------------------------------------------------')
          parag_crux_str = get_parag_str(i)
          print(parag_crux_str)
        else:
          print(f'\n   ----------------------')
          print(f'   Regular Paragraph #{i}')
          print(f'   ----------------------')
          print(get_parag_str(i))

In [ ]:
# Insert new crux point into ordered list: corpus_man_crux_ls

# NOTE: For very long lists, use Python simple bisect library (at cost of additional dependency)


def insert_ord_tp_list(crux_ord_ols, crux_tp):
  '''
  Insert new crux tuple: crux_tp = (sent_no, sentiment_polarity)
  into ordered list of tuples while maintaining sent_no order
  '''
  sent_no, senti_pol = crux_tp

  # Searching for the position
  for i in range(len(crux_ord_ols)):
    if crux_ord_ols[i][0] == sent_no:
      # Attempting to insert duplicate
      return crux_ord_ols
    elif crux_ord_ols[i][0] < sent_no:
      insert_idx = i
    else:
      break
      
  # Inserting n in the list
  list = crux_ord_ols[:i] + [crux_tp] + crux_ord_ols[i:]
  return list

'''
# Test
crux_test_ls = [(1,0), (5,1), (10,-1)]
crux_test_tp = (3,10)
  
print(insert_ord_tp_list(crux_test_ls, crux_test_tp))
''';

**Start of Human in the Loop Manual Crux Point Identification**

In [ ]:
# Instructions

print('Enter a Sentence number based upon your search above to see the ')
print('  surrounding Paragraph context.')
print('----------------------------------------')
print(f'(Enter an integer between 0 and {corpus_sents_len-1})\n\n')

print('\n')
print('Enter an ODD NUMBER for the Number of surrounding Paragraphs ')
print('  around the Sentence No to give Context.')
print('----------------------------------------')
print(f'(Enter an integer: 3, 5, 7\n\n')

In [ ]:
# Input your Context Retrieval Parameters

Sentence_No =  2692#@param {type:"integer"}
No_Paragraphs_Context = "3" #@param ["1", "3", "5"]

get_nparags_context(Sentence_No, No_Paragraphs_Context)

**Add Crux to Manually Generated Sentiment Arc**

In [ ]:
# Instructions to add current sentence as a Crux Point

crux_summary()

print('--------------------------------------------------------')
print(f'INSTRUCTIONS To current Sentence No: {Sentence_No} as a Crux Point')
print('--------------------------------------------------------')

print("\nCheck this box if you want to add the Sentence/Paragraph above ")
print("  as a new Min/Max Crux Point with your approximation ")
print("  for a Sentiment Polarity value between -1.0 to +1.0\n\n")

print(f"Crux Sentence No: {Sentence_No} in Paragraph No: {parag_crux_no}\n")
print(parag_crux_str)

print("\n\nLeave Add_Sentence_Crux 'unchecked' to not add")

In [ ]:
# <INPUT> Option to add this Sentence/Paragraph as a Min/Max Crux Point

Sentiment_Polarity = -0.4 #@param {type:"slider", min:-1, max:1, step:0.1}
Add_Sentence_Crux = True #@param {type:"boolean"}


# Add Crux if selected and give current summary status

if Add_Sentence_Crux == True:
  crux_new_tp = tuple((Sentence_No, Sentiment_Polarity))
  corpus_man_crux_ols = insert_ord_tp_list(corpus_man_crux_ols, crux_new_tp)
  if (corpus_man_crux_ols):
    print(f'Successfully inserted new Crux = {crux_new_tp}')
    print(f'Added Crux at Sentence No={Sentence_No} with Polarity={Sentiment_Polarity}')
    # corpus_man_cruxes_dt[Sentence_No] = Sentiment_Polarity
  else:
    print(f'ERROR: Could not insert new Crux = {crux_new_tp}')

In [ ]:
# Summary of current Crux Points after addition

print('\n------------------------------------------------------------')
print(f'After addition of new Crux Point (Sentence No {Sentence_No})')
print('------------------------------------------------------------\n')

crux_summary()

**Delete Manually Selected Crux Points**

In [ ]:
len(corpus_man_crux_ols)

In [ ]:
crux_tp = (1, 2)
a, b = crux_tp
print(f'a is {a} and b is {b}')

In [ ]:
# Insert new crux point into ordered list: corpus_man_crux_ls

# FIX: 20210616 and move to utility functions

# NOTE: For very long lists, use Python simple bisect library (at cost of additional dependency)


def del_ord_tp_list(acorpus_man_crux_ols, crux_tp):
  '''
  Insert new crux tuple: crux_tp = (sent_no, sentiment_polarity)
  into ordered list of tuples while maintaining sent_no order
  '''
  crux_ct = len(acorpus_man_crux_ols)
  sent_no = crux_tp[0]
  print(f'Deleting sent_no: {sent_no} over crux_ls len={len(acorpus_man_crux_ols)}')

  # Searching for the positionk
  del_idx = -1
  for i in range(len(acorpus_man_crux_ols)):
    acrux_sent_no = acorpus_man_crux_ols[i][0]
    print(f'Crux #{i} is sent_no={acrux_sent_no}')
    if acrux_sent_no == sent_no:
      print(f'Matching index at {i}')
      del_idx = i
      
  # Delete n in the list
  print(f'Deletion index = {del_idx}')
  if del_idx == 0:
    # Delete the first Crux
    list = acorpus_man_crux_ols[1:]
    return list
  elif del_idx == crux_ct -1:
    # Delete the last Crux
    list = acorpus_man_crux_ols[:-1]
    return list    
  elif (del_idx > 0) & (del_idx < crux_ct):
    # Delete an interior Crux
    before_idx = i - 1
    after_idx = i
    list = acorpus_man_crux_ols[:before_idx] + acorpus_man_crux_ols[after_idx:]
    print(f'Returning list: {list}')
    return list
  else:
    print('No matching Crux tuple found')
    return acorpus_man_crux_ols
  

# Test
crux_test_ls = [(1,0), (5,1), (10,-1)]
crux_test_tp = (5,5)
  
print(del_ord_tp_list(crux_test_ls, crux_test_tp))

In [ ]:
# Instructions to Delete a Crux Point

crux_summary()

print('--------------------------------------------------------')
print('INSTRUCTIONS To Delete a Crux Point')
print('--------------------------------------------------------')

print("\nEnter the Sentence No of a Crux you want to delete.\n")
print(f'     Current Crux Points by Sentence No: {corpus_man_crux_ols}\n\n')
print(" Skip this if you want to keep all manually selected Crux Points.\n")

In [ ]:
# Instructions

print("\nEnter the Sentence No of a Crux you want to delete.\n")
print(f'     Current Crux Points by Sentence No: {corpus_man_crux_ols}\n\n')
print(" Skip this if you want to keep all manually selected Crux Points.\n")

In [ ]:
Delete_Sent_No =  777#@param {type:"integer"}
# Select a Crux to Delete
# TODO: Drop down list

# corpus_man_crux_ols
corpus_man_crux_temp_ols = []

crux_sent_set = set([x[0] for x in corpus_man_crux_ols])
if not(Delete_Sent_No in crux_sent_set):
  print(f'ERROR: {Delete_Sent_No} is not a Crux Point Sentence No')
else:
  # Keep the same tuple format for uniformity and future features
  crux_del_tp = tuple((Delete_Sent_No, 'dummy_sentence'))
  print(f'Selected {(crux_del_tp)} to delete')
  # corpus_man_crux_temp_ols = 
  print(f'WTF: {del_ord_tp_list(corpus_man_crux_ols, crux_del_tp)}')
  corpus_man_crux_old = del_ord_tp_list(corpus_man_crux_ols, crux_del_tp)
  print(f"corpus_man_crux_ols: {corpus_man_crux_ols}")
  # get_sent_no = Sentence_No
  # print(f'Retrieving Sentence No: {get_sent_no}')
  # print('----------')
  print(f'Updated Crux Points by Sentence No: {corpus_man_crux_ols}\n\n')


**Review Summary of all Manually Selected Crux Points**

In [ ]:
# Generate Report Summary of All Manually Selected Cruxes

f = io.StringIO()
with contextlib.redirect_stdout(f):
    crux_summary()
crux_summary = f.getvalue()

# Print Manual Crux Report Summary to Screen

# print(crux_summary)

In [ ]:
# Save Manual Crux Summary Report

plot_filename = 'man_cruxes.txt'
plotpathfilename_str = gen_pathfiletime(plot_filename)

with open(plotpathfilename_str, 'a+') as outfp:
  outfp.write(crux_summary)

# Verify 

!ls -alt $plotpathfilename_str

In [ ]:
# Verify Report Content

!cat man_cruxes_fscottfitzgerald_thegreatgatsby_20210616214050.txt

**Clean and Organize Manual Crux Points into new Data Structures**

In [ ]:
print(corpus_sents_df[corpus_sents_df['sent_no']==5]['sent_raw'].squeeze())

In [ ]:
# Convert and assemble all the Crux values in lists to save in a new Crux DataFrame


pol_val_ls = [x[1] for x in corpus_man_crux_ols]
sent_no_ls = [x[0] for x in corpus_man_crux_ols]
parag_no_ls = [(get_parag4sentno(x))[0] for x in sent_no_ls]
parag_str_ls = [get_parag_str(x) for x in parag_no_ls]
sent_no_ls = [x[0] for x in corpus_man_crux_ols]
sent_raw_ls = [corpus_sents_df[corpus_sents_df['sent_no']==x]['sent_raw'].squeeze() for x in sent_no_ls]
sent_raw_ls
sent_clean_ls = [corpus_sents_df[corpus_sents_df['sent_no']==x]['sent_clean'].squeeze() for x in sent_no_ls]
sent_clean_ls

In [ ]:
# Create a Dict of Crux Points to Tuples (Polarity, Raw Sentence)

# First Create the Tuples for each Sentence No (Sentiment Polarity, Raw Text)
def merge(list1, list2):
    merged_list = tuple(zip(list1, list2)) 
    return merged_list
      
crux_tp_ls = merge(pol_val_ls, sent_raw_ls)

# Second, Create the Dictionary C
corpus_man_cruxes_dt = {sent_no_ls[i]: crux_tp_ls[i] for i in range(len(crux_tp_ls))}

# Verify
corpus_man_cruxes_dt

In [ ]:
corpus_sents_df.head()

**Plot Interpolated Manual Sentiment Arc**

In [ ]:
corpus_man_sa_df = pd.DataFrame({'sent_no':xn, 'sentiment':yn, 'sent_raw':corpus_sents_df.sent_raw.values})
corpus_man_sa_df

In [ ]:
# Hermite Interpolation with SciPy

pol_val_ls = [x[1] for x in corpus_man_crux_ols]
sent_no_ls = [x[0] for x in corpus_man_crux_ols]
parag_no_ls = [(get_parag4sentno(x))[0] for x in sent_no_ls]
parag_str_ls = [get_parag_str(x) for x in parag_no_ls]
sent_no_ls = [x[0] for x in corpus_man_crux_ols]
sent_raw_ls = [corpus_sents_df[corpus_sents_df['sent_no']==x]['sent_raw'].squeeze() for x in sent_no_ls]
sent_raw_ls
sent_clean_ls = [corpus_sents_df[corpus_sents_df['sent_no']==x]['sent_clean'].squeeze() for x in sent_no_ls]
sent_clean_ls

In [ ]:
sent_no_ls
pol_val_ls

In [ ]:
corpus_man_crux_np = np.asarray(corpus_man_crux_ols)
corpus_man_crux_np

In [ ]:
import numpy as np

In [ ]:
x2 = np.array(sent_no_ls)
y2 = np.array(pol_val_ls)

xn = np.linspace(0, corpus_sents_len, corpus_sents_len)
yn = interpolate.pchip_interpolate(x2, y2, xn)

crux_man_df = pd.DataFrame(
    {'sent_no': sent_no_ls,
     'pol_val': pol_val_ls
     }
)

# plt.plot(x2, y2, 'ok', label='True values')
# plt.plot(xn, yn, label='Hermite Interpolation')

# plt.plot(xn, yn4, label='Spline order 4')
# plt.plot(xn, yn5, label='Spline order 5')
# plt.plot(xn, yn6, label='Spline order 6')
# plt.plot(xn, yn7, label='Spline order 7')
plt.legend()
plt.show()


# sns.histplot(data=corpus_sents_df['char_len'], kde=True).set_title(f'{CORPUS_FULL} \n Histogram of Paragraph Lengths')
sns.histplot(data=crux_man_df, x='sent_no', y='pol_val', kde=True).set_title(f'{CORPUS_FULL} \n Manual Cruxes with Hermite Smoothing')


# Save graph to file.
plot_filename = 'man_crux_plot.png'
plotpathfilename_str = gen_pathfiletime(plot_filename)
plt.savefig(plotpathfilename_str, format='png', dpi=300)


**Gaussian Process Regression**

* https://blog.dominodatalab.com/fitting-gaussian-process-models-python/

# (Optional) Load Sentiment Polarities: Interactive

***If you upload a file of Sentiment Values you don't have to Calculate them in the following sections***

In [ ]:
!ls -altr *.csv

In [ ]:
# Test

files.download('sentiments_raw_all_virginiawoolf_tothelighthouse_20210618161224.csv')

In [ ]:
# Upload your precomputed Sentiment Values

uploaded = files.upload()

In [ ]:
%whos DataFrame

In [ ]:
# Verify the file was uploaded correctly

newest_csvfile = get_recentfile().split('/')[-1]
print(f'The most recently updated *.csv file is: {newest_csvfile}')

!head -n 10 $newest_file

In [ ]:
%whos DataFrame


In [ ]:
# Upload file into DataFrame

corpus_test_df = pd.read_csv(newest_csvfile)
corpus_test_df['sent_clean'] = corpus_test_df['sent_clean'].astype('string')
corpus_test_df['sent_raw'] = corpus_test_df['sent_raw'].astype('string')
corpus_test_df.head()
corpus_test_df.info()

***Skip to Section <Calculate Median of All...> if SA Loaded***

In [ ]:
corpus_lexicons_stats_dt

# **Either (Load) and/or (Calculate) Sentiment Values**

Sentiment Models

* VADER [-1.0 to 1.0] zero peak
* TextBlob [-1.0 to 1.0] zero peak
* Stanza outliers [-1.0 to 199.0] pos, outliers(+peak)
* AFINN [-14 (-8 to 8) 20] discrete
* SentimentR 11,710 [-5.4 to 8.8] norm
* Syuzhet [-5.4 to 8.8] norm
* Bing [-100.0 (-20.0 to 20.0) 100] discrete, outliers
* Pattern [-1.0 to 1.0] norm
* SentiWord [-3.8 to 4.4] norm
* SenticNet [-3.8 to 10] norm
* NRC [-100.0 (-5.0 to 5.0) 100] zero, outliers

## **Read Verification**

In [96]:
%whos DataFrame

Variable           Type         Data/Info
-----------------------------------------
corpus_chaps_df    DataFrame       chap_no  ...          <...>\n\n[10 rows x 7 columns]
corpus_parags_df   DataFrame          parag_no  ...      <...>\n[1428 rows x 5 columns]
corpus_sects_df    DataFrame        sect_no  ...         <...>\n\n[30 rows x 7 columns]
corpus_sents_df    DataFrame          sent_no  ...       <...>\n[7679 rows x 6 columns]


In [97]:
# Verfiy there are no NaN or Empty strings that passed the cleaning process

corpus_sents_df[corpus_sents_df['sent_clean'].isnull()]

corpus_sents_df[corpus_sents_df['sent_clean'].apply(lambda x: len(str(x)) <= 0)]

,sent_no,parag_no,sent_raw,char_len,token_len,sent_clean


,sent_no,parag_no,sent_raw,char_len,token_len,sent_clean


In [98]:
# Verify that hyphenated words are correctly handled (e.g. 'summer-mroning' -> 'summer morning')

corpus_sents_df[corpus_sents_df['sent_clean'].str.contains('summer', na=False)]

,sent_no,parag_no,sent_raw,char_len,token_len,sent_clean
1400,1400,235,"I went into my bedroom, undressed, leaving my ...",118,23,i went into my bedroom undressed leaving my cl...
1652,1652,272,"On the windowsill, where yellow gingham curtai...",167,26,on the windowsill where yellow gingham curtain...
1792,1792,292,"For me, it was far too early for bed and it wa...",88,19,for me it was far too early for bed and it was...
2095,2095,344,"Here she was beside me, close enough for me to...",83,15,here she was beside me close enough for me to ...
2613,2613,486,The summer was hot and something was coming to...,56,11,the summer was hot and something was coming to...
2952,2952,559,If the court would suspend proceedings and ser...,202,35,if the court would suspend proceedings and ser...
3698,3698,714,Our last summer at school came around.,38,7,our last summer at school came around
3866,3866,725,I would had two boyfriends by that summer and ...,64,14,i would had two boyfriends by that summer and ...
4410,4410,801,"Now she was writing a short essay, to be read ...",145,25,now she was writing a short essay to be read a...
4676,4676,850,I wanted to lie down on the worn-out grass of ...,76,16,i wanted to lie down on the worn out grass of ...


## **(Optional) Load Raw Sentence Sentiment Time Series**

### **Select Datafile to Read**

In [99]:
# By convention, Summary Sentiment files are named with the RegEx template 'sum_sentiments_*.csv'

!ls -altr sum_sentiments_*.csv

-rw------- 1 root root 651742 Jul  8 13:44 sum_sentiments_syuzhetR_4models.csv
-rw------- 1 root root 681263 Jul  8 14:48 sum_sentiments_sentimentR_7models.csv


#### **Import SentimentR Sentiment Data**

In [103]:
corpus_sentimentr_df.head(2)

,element_id,sentence_id,word_count,jockers_rinker,jockers,huliu,senticnet,sentiword,nrc,lmcd
0,1,1,17.0,0.169775,0.169775,0.242536,0.595849,0.151585,0.0,0.242536
1,2,1,7.0,0.000000,0.000000,0.000000,0.226023,0.094491,0.0,0.000000


In [109]:
corpus_sentimentr_df.shape

(7700, 10)

In [101]:
# Test

!pip install kmeans1d

     |████████████████████████████████| 102kB 4.8MB/s 


In [114]:
import kmeans1d

k = corpus_sentimentr_df.shape[0]//500  

clusters, centroids = kmeans1d.cluster(np.array(corpus_sentimentr_df['jockers_rinker']), k)
type(clusters)

[[x,clusters.count(x)] for x in set(clusters)]
centroids

list

[[0, 10],
 [1, 92],
 [2, 215],
 [3, 324],
 [4, 437],
 [5, 559],
 [6, 626],
 [7, 2958],
 [8, 618],
 [9, 624],
 [10, 523],
 [11, 366],
 [12, 232],
 [13, 106],
 [14, 10]]

[-1.368015009855796,
 -0.7910295756635374,
 -0.5451577889958344,
 -0.4022212257614058,
 -0.28858183852377894,
 -0.18794187257306585,
 -0.09116783093956264,
 0.0003301146611772096,
 0.09738225284515498,
 0.18855992595452264,
 0.2841837109825539,
 0.40558874055845845,
 0.5622555762714548,
 0.8145846115190027,
 1.500069915199759]

In [102]:
# (Optional) Read Sentiment Series generated in RStudio by SentimentR into DataFrame: corpus_sentimentr_df
#            SKIP if no SyuzhetR sentiment datafile to read in

sum_sentiment_sentimentR_filename = 'sum_sentiments_sentimentR_7models.csv'
corpus_sentimentr_df = pd.read_csv(sum_sentiment_sentimentR_filename)

corpus_sentimentr_df.head(2)
corpus_sentimentr_df.info()
corpus_sentimentr_df.columns

corpus_sentimentr_len = corpus_sentimentr_df.shape[0]

# BUG FIX: SentimentR seems to create one additional row that must be deleted 
#          to enable it to be merged with other Sentiment Models on the same Corpus
#
#          Simplification, 1D cluster jockers_rinker column as proxy for full interrow distance features
#                          and delete one row near the median from the largest cluster

import kmeans1d

# Approximate k cluster number as 1 cluster for every 500 sentences in Corpus
k = corpus_sentimentr_df.shape[0]%500  
clusters, centroids = kmeans1d.cluster(np.array(corpus_sentimentr_df['jockers_rinker']), k)

def del_oneincluster(df, cluster_per=1):
  '''
  TODO: Skip for now and use kmeans1d instead
  Given a DataFrame and a Cluster Percent to calculate a sliding window
  Return DataFrame with one row removed within a sliding window cluster with most self-similiar rows
  '''

  # Compute sliding window for cluster size
  win_cluster_len = int(cluster_per/100 * df.shape[0])
  win_start = 0
  win_stop = df.shape[0] - win_cluster_len

  # Get numeric columns
  numeric_df = df.select_dtypes(include=numerics)

  most_selfsimilar_value = 0
  most_selfsimilar_index = 0
  for i in range(win_start, win_stop, 1):
    selfsim_score = selfsim_metric(numeric_df.iloc[i:win_cluster_len+1])
    if selfsim_score > most_selfsimilar_value:
      most_selfsimilar_index = i

  oneless_df = del_onerow(most_selfsimilar_index)

  return oneless_df

if corpus_sentimentr_len != corpus_sents_df.shape[0]:
  print('\n\n\n======================================================================\n')
  print(f'ERROR: sentence sentiment values read into corpus_syuzhetr (len={corpus_sentimentr_len})')
  print(f'       is not the same length as corpus_sents_df (len={corpus_sents_df.shape[0]}) ')
  print(f'\nRECOMMENDATION: Use the preprocessed corpus output created by this notebook ')
  print(f'                as input to SentimentR in RStudio to generate sentiment series')
  print(f'                and then retry importing')
  print('\n======================================================================\n')

,element_id,sentence_id,word_count,jockers_rinker,jockers,huliu,senticnet,sentiword,nrc,lmcd
0,1,1,17.0,0.169775,0.169775,0.242536,0.595849,0.151585,0.0,0.242536
1,2,1,7.0,0.000000,0.000000,0.000000,0.226023,0.094491,0.0,0.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7700 entries, 0 to 7699
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   element_id      7700 non-null   int64  
 1   sentence_id     7700 non-null   int64  
 2   word_count      7691 non-null   float64
 3   jockers_rinker  7700 non-null   float64
 4   jockers         7700 non-null   float64
 5   huliu           7700 non-null   float64
 6   senticnet       7700 non-null   float64
 7   sentiword       7700 non-null   float64
 8   nrc             7700 non-null   float64
 9   lmcd            7700 non-null   float64
dtypes: float64(8), int64(2)
memory usage: 601.7 KB


Index(['element_id', 'sentence_id', 'word_count', 'jockers_rinker', 'jockers',
       'huliu', 'senticnet', 'sentiword', 'nrc', 'lmcd'],
      dtype='object')





ERROR: sentence sentiment values read into corpus_syuzhetr (len=7700)
       is not the same length as corpus_sents_df (len=7679) 

RECOMMENDATION: Use the preprocessed corpus output created by this notebook 
                as input to SentimentR in RStudio to generate sentiment series
                and then retry importing




#### **Import SyuzhetR Sentiment Data**

In [ ]:
# (Optional) Read Sentiment Series generated in RStudio by SyuzhetR into DataFrame: corpus_syuzhetr_df
#            SKIP if no SyuzhetR sentiment datafile to read in

sum_sentiment_syuzhetR_filename = 'sum_sentiments_syuzhetR_4models.csv'
corpus_syuzhetr_df = pd.read_csv(sum_sentiment_syuzhetR_filename)

corpus_syuzhetr_df.head(2)
corpus_syuzhetr_df.info()
corpus_syuzhetr_df.columns

corpus_syuzhetr_len = corpus_syuzhetr_df.shape[0]

if corpus_syuzhetr_len != corpus_sents_df.shape[0]:
  print('\n\n\n======================================================================\n')
  print(f'ERROR: sentence sentiment values read into corpus_syuzhetr (len={corpus_syuzhetr_len})')
  print(f'       is not the same length as corpus_sents_df (len={corpus_sents_df.shape[0]}) ')
  print(f'\nRECOMMENDATION: Use the preprocessed corpus output created by this notebook ')
  print(f'                as input to SyuzhetR in RStudio to generate sentiment series')
  print(f'                and then retry importing')
  print('\n======================================================================\n')

,sent_no,sent_raw,syuzhet,bing,afinn,nrc
0,0,"But remember, please, the Law by which we live...",1.3,-1,1,0
1,1,"--Rudyard Kipling, ""The Secret of the Machines""",0.0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7679 entries, 0 to 7678
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sent_no   7679 non-null   int64  
 1   sent_raw  7679 non-null   object 
 2   syuzhet   7679 non-null   float64
 3   bing      7679 non-null   int64  
 4   afinn     7679 non-null   int64  
 5   nrc       7679 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 360.1+ KB


Index(['sent_no', 'sent_raw', 'syuzhet', 'bing', 'afinn', 'nrc'], dtype='object')

#### **Import Transformer Sentiment Data**

In [ ]:
# Read in Section DataFrame: corpus_sents_df

corpus_sects_df = pd.read_csv('corpus_sections_lexrules_vatestsesyafbipasesenr_ianmcewan_machineslikeme_20210629_1749.csv')
corpus_sects_df.head(2)
corpus_sects_df.info()
corpus_sects_df.columns


print(f\n\n==================================================)
print(f)


In [ ]:
# Verify 

### **Calculate Lexical Statistics**

In [ ]:
# models_ls = ['vader', 'textblob', 'stanza', 'afinn', 'sentimentr', 'syuzhet', 'bing', 'pattern', 'sentiword', 'senticnet', 'nrc']

# corpus_lexicons_stats_dt = {}

# for amodel in models_ls:
#   get_lexstats(amodel)

# Validate
# corpus_lexicons_stats_dt

In [ ]:
corpus_sents_df.shape

(7679, 51)

## **(Optional) Calculate Sentiment Polarities (Auto)**

### **Select Sentiment Models (Manual)**

NOTE:

* Stanza (Stanford OpenNLP) can take upto 50 minutes to run

* Listed in increasing order of (approx) run time

* MPQA/SentiStrength not yet implemented (placeholders only for now)

In [ ]:
SentimentR_Arc = True #@param {type:"boolean"}
Syuzhet_Arc = True #@param {type:"boolean"}
Bing_Arc = True #@param {type:"boolean"}
SentiWord_Arc = True #@param {type:"boolean"}
SenticNet_Arc = True #@param {type:"boolean"}
NCR_Arc = True #@param {type:"boolean"}
AFINN_Arc = True #@param {type:"boolean"}
VADER_Arc = True #@param {type:"boolean"}
TextBlob_Arc = True #@param {type:"boolean"}
Pattern_Arc = True #@param {type:"boolean"}
Stanza_Arc = False #@param {type:"boolean"}
MPQA_Arc = False #@param {type:"boolean"}
SentiStrength_Arc = False #@param {type:"boolean"}


In [ ]:
# Create and Verify custom list of Models to include

MODELS_CUSTOM_LS = []

if VADER_Arc:
  MODELS_CUSTOM_LS.append('vader')
if TextBlob_Arc:
  MODELS_CUSTOM_LS.append('textblob')
if Stanza_Arc:
  MODELS_CUSTOM_LS.append('stanza')
if SentimentR_Arc:
  MODELS_CUSTOM_LS.append('sentimentr')
if Syuzhet_Arc:
  MODELS_CUSTOM_LS.append('syuzhet')
if AFINN_Arc:
  MODELS_CUSTOM_LS.append('afinn')
if Bing_Arc:
  MODELS_CUSTOM_LS.append('bing')
if Pattern_Arc:
  MODELS_CUSTOM_LS.append('pattern')
if SentiWord_Arc:
  MODELS_CUSTOM_LS.append('sentiword')
if SenticNet_Arc:
  MODELS_CUSTOM_LS.append('senticnet')
if NCR_Arc:
  MODELS_CUSTOM_LS.append('nrc')

print(f'Here are the Models we are using to ensemble and save:\n   {MODELS_CUSTOM_LS}')

models_incl_ls = []
for amodel in MODELS_CUSTOM_LS:
  models_incl_ls.append(amodel[:2])
models_incl_str = ''.join(models_incl_ls)

print(f'Here is a custom name abbr: {models_incl_str}')

In [ ]:
# Calculate (win_(x)1per) 1% of Corpus length for smallest (odd-valued) rolling window

# Sentences
corpus_sents_len = corpus_sents_df.shape[0]

win_raw_s1per = int(corpus_sents_len * 0.01)
# print(f'1% Rolling Window: {win_raw_s1per}')

if win_raw_s1per % 2:
  win_s1per = win_raw_s1per
else:
  win_s1per = win_raw_s1per + 1

# Paragraphs

corpus_parags_len = corpus_parags_df.shape[0]

win_raw_p1per = int(corpus_parags_len * 0.01)
# print(f'1% Rolling Window: {win_raw_1per}')

if win_raw_p1per % 2:
  win_p1per = win_raw_p1per
else:
  win_p1per = win_raw_p1per + 1


# Sections

# NO NEED FOR SLIDING WINDOW ON SECTIONS


print(f'Sentence 1 Percent window: {win_s1per}')
print(f'Paragraph 1 Percent window: {win_p1per}')

Sentence 1 Percent window: 77
Paragraph 1 Percent window: 15


In [ ]:
# Verify hash files for sentiment lexicons

!pwd
!ls hash*csv

/gdrive/My Drive/research/2021/sa_book_code/books_sa/imcewan_machineslikeme
hash_sentiment_bing.csv       hash_sentiment_sentimentr.csv
hash_sentiment_nrc.csv	      hash_sentiment_sentiword.csv
hash_sentiment_senticnet.csv  hash_sentiment_syuzhet.csv


### **Calculate SentimentR (Jockers-Rinker) Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'sentimentr'
model_name = 'sentimentr_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
# Read Lexicon into DataFrame 

if SentimentR_Arc == True:

  lexicon_sentimentr_df = get_lexicon('hash_sentiment_sentimentr.csv')
  lexicon_sentimentr_df['x'] = lexicon_sentimentr_df['x'].astype('string')

  # Clean/Reorg DataFrame
  lexicon_sentimentr_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
  lexicon_sentimentr_df.rename(columns={'x':'word', 'y':'polarity'}, inplace=True)

  # Verify
  if (PLOT_OUTPUT == 'All'):
    lexicon_sentimentr_df.head()
    lexicon_sentimentr_df.info()

Found hash_sentiment_sentimentr.csv in lexicon_directory)
cp_cmd = copy ../sa_lexicons/hash_sentiment_sentimentr.csv ./
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11710 entries, 0 to 11709
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11710 non-null  int64  
 1   x           11710 non-null  object 
 2   y           11710 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 274.6+ KB


In [ ]:
# Convert DataFrame to Dict[word] = polarity

if SentimentR_Arc == True:

  id = lexicon_sentimentr_df.word.values
  values = lexicon_sentimentr_df.polarity.values

  lexicon_sentimentr_dt = dict(zip(id, values))
  # lexicon_sentimentr_dt

  # Test
  sent_test='I hate Mondays.'
  print(text2sentiment(sent_test, lexicon_sentimentr_dt))

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Sentiment evaluation function
def sentiment_sentimentr(text_str):
  '''
  Given a text string
  Return corresponding sentiment value based on sentimentr lexicon
  '''
  
  sentiment_val = text2sentiment(str(text_str), lexicon_sentimentr_dt)

  return sentiment_val 

# Calculate all Sentiment values and variants
if SentimentR_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=sentiment_sentimentr, sentiment_type='lexicon')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'sentimentr_chaps': {'sentiment_max': 148.9999999999999,
  'sentiment_min': 3.1000000000000116},
 'sentimentr_parags': {'sentiment_max': 17.55, 'sentiment_min': -11.7},
 'sentimentr_sects': {'sentiment_max': 96.14999999999986,
  'sentiment_min': -17.149999999999977},
 'sentimentr_sents': {'sentiment_max': 8.9, 'sentiment_min': -6.4}}

### **Calculate Syuzhet (Jockers) Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'syuzhet'
model_name = 'syuzhet_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
# Read Lexicon into DataFrame 

if Syuzhet_Arc == True:

  lexicon_syuzhet_df = get_lexicon('hash_sentiment_syuzhet.csv')
  lexicon_syuzhet_df['word'] = lexicon_syuzhet_df['word'].astype('string')

  # Clean/Reorg DataFrame
  lexicon_syuzhet_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
  lexicon_syuzhet_df.rename(columns={'x':'word', 'y':'polarity'}, inplace=True)

  # Verify
  if (PLOT_OUTPUT == 'All'):
    lexicon_syuzhet_df.head()
    lexicon_syuzhet_df.info()

Found hash_sentiment_syuzhet.csv in lexicon_directory)
cp_cmd = copy ../sa_lexicons/hash_sentiment_syuzhet.csv ./
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10748 entries, 0 to 10747
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10748 non-null  int64  
 1   word        10748 non-null  object 
 2   value       10748 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 252.0+ KB


In [ ]:
# Convert DataFrame to Dict[word] = polarity

if Syuzhet_Arc == True:

  id = lexicon_syuzhet_df.word.values
  values = lexicon_syuzhet_df.value.values

  lexicon_syuzhet_dt = dict(zip(id, values))
  # lexicon_sentimentr_dt

  # Test
  sent_test='I hate Mondays.'
  print(text2sentiment(sent_test, lexicon_syuzhet_dt))

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Sentiment evaluation function
def sentiment_syuzhet(text_str):
  '''
  Given a text string
  Return corresponding sentiment value based on sentimentr lexicon
  '''
  
  sentiment_val = text2sentiment(str(text_str), lexicon_syuzhet_dt)

  return sentiment_val 

# Calculate all Sentiment values and variants

if Syuzhet_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=sentiment_syuzhet, sentiment_type='lexicon')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'sentimentr_chaps': {'sentiment_max': 148.9999999999999,
  'sentiment_min': 3.1000000000000116},
 'sentimentr_parags': {'sentiment_max': 17.55, 'sentiment_min': -11.7},
 'sentimentr_sects': {'sentiment_max': 96.14999999999986,
  'sentiment_min': -17.149999999999977},
 'sentimentr_sents': {'sentiment_max': 8.9, 'sentiment_min': -6.4},
 'syuzhet_chaps': {'sentiment_max': 145.19999999999985,
  'sentiment_min': -1.000000000000015},
 'syuzhet_parags': {'sentiment_max': 18.05, 'sentiment_min': -11.7},
 'syuzhet_sects': {'sentiment_max': 91.19999999999989,
  'sentiment_min': -17.049999999999976},
 'syuzhet_sents': {'sentiment_max': 8.9, 'sentiment_min': -6.4}}

### **Calculate Bing (HuLiu) Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'bing'
model_name = 'bing_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
# Read Lexicon into DataFrame 

if Bing_Arc == True:

  lexicon_bing_df = get_lexicon('hash_sentiment_bing.csv')
  lexicon_bing_df['x'] = lexicon_bing_df['x'].astype('string')

  # Clean/Reorg DataFrame
  lexicon_bing_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
  lexicon_bing_df.rename(columns={'x':'word', 'y':'polarity'}, inplace=True)

  # Verify
  if (PLOT_OUTPUT == 'All'):
    lexicon_bing_df.head()
    lexicon_bing_df.info()

Found hash_sentiment_bing.csv in lexicon_directory)
cp_cmd = copy ../sa_lexicons/hash_sentiment_bing.csv ./
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6874 entries, 0 to 6873
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  6874 non-null   int64  
 1   x           6874 non-null   object 
 2   y           6874 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 161.2+ KB


In [ ]:
# Convert DataFrame to Dict[word] = polarity

if Bing_Arc == True:

  id = lexicon_bing_df.word.values
  values = lexicon_bing_df.polarity.values

  lexicon_bing_dt = dict(zip(id, values))


In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Sentiment evaluation function
def sentiment_bing(text_str):
  '''
  Given a text string
  Return corresponding sentiment value based on sentimentr lexicon
  '''
  
  sentiment_val = text2sentiment(str(text_str), lexicon_bing_dt)

  return sentiment_val 

# Calculate all Sentiment values and variants
if Bing_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=sentiment_bing, sentiment_type='lexicon')

In [ ]:
# Calculate Bing Sentiment [0,1,2]

def bing_discrete2continous_sentiment(text):
  '''
  Given a plain text string, give it to
    Stanford Stanza (OpenNLP) to calculate sentiment for each word on a 3 point scale 0-2
  Return a sentiment value for the entire sentence (sum of word sentiments/log(len of sentence)) 
    that approximates a normal distribution for all values
    In order to get more fine grained measure of overall Sentence sentiment
    Sentiment values will be Normalized/Standardized so absolute precision is not required
  '''
  text_sentiment_total = 0.
  text_ls = text.split()
  text_len = len(text_ls)
  for aword in text_ls:
    text_sentiment_total += lex_discrete2continous_sentiment(str(aword), lexicon_bing_dt)
  text_sentiment_norm = text_sentiment_total/(np.log(text_len)+1)

  return text_sentiment_norm

In [ ]:
# Test

if Bing_Arc == True:
  sent_test='I hate Mondays.'
  print(bing_discrete2continous_sentiment(sent_test))

-47.650535804050435


In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Calculate all Sentiment values and variants
if Bing_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=bing_discrete2continous_sentiment, sentiment_type='function')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'sentimentr_chaps': {'sentiment_max': 148.9999999999999,
  'sentiment_min': 3.1000000000000116},
 'sentimentr_parags': {'sentiment_max': 17.55, 'sentiment_min': -11.7},
 'sentimentr_sects': {'sentiment_max': 96.14999999999986,
  'sentiment_min': -17.149999999999977},
 'sentimentr_sents': {'sentiment_max': 8.9, 'sentiment_min': -6.4},
 'syuzhet_chaps': {'sentiment_max': 145.19999999999985,
  'sentiment_min': -1.000000000000015},
 'syuzhet_parags': {'sentiment_max': 18.05, 'sentiment_min': -11.7},
 'syuzhet_sects': {'sentiment_max': 91.19999999999989,
  'sentiment_min': -17.049999999999976},
 'syuzhet_sents': {'sentiment_

### **Calculate SentiWord Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'sentiword'
model_name = 'sentiword_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
# Read Lexicon into DataFrame 

if SentiWord_Arc == True:

  lexicon_sentiword_df = get_lexicon('hash_sentiment_sentiword.csv')
  lexicon_sentiword_df['x'] = lexicon_sentiword_df['x'].astype('string')

  # Clean/Reorg DataFrame
  lexicon_sentiword_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
  lexicon_sentiword_df.rename(columns={'x':'word', 'y':'polarity'}, inplace=True)

  # Verify
  if (PLOT_OUTPUT == 'All'):
    lexicon_sentiword_df.head()
    lexicon_sentiword_df.info()

Found hash_sentiment_sentiword.csv in lexicon_directory)
cp_cmd = copy ../sa_lexicons/hash_sentiment_sentiword.csv ./
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20093 entries, 0 to 20092
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  20093 non-null  int64  
 1   x           20092 non-null  object 
 2   y           20093 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 471.1+ KB


In [ ]:
# Convert DataFrame to Dict[word] = polarity

if SentiWord_Arc == True:

  id = lexicon_sentiword_df.word.values
  values = lexicon_sentiword_df.polarity.values

  lexicon_sentiword_dt = dict(zip(id, values))
  # lexicon_sentiword_dt

  # Test
  sent_test='I hate Mondays.'
  print(text2sentiment(sent_test, lexicon_sentiword_dt))

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Sentiment evaluation function
def sentiment_sentiword(text_str):
  '''
  Given a text string
  Return corresponding sentiment value based on sentimentr lexicon
  '''
  
  sentiment_val = text2sentiment(str(text_str), lexicon_sentiword_dt)

  return sentiment_val 

# Calculate all Sentiment values and variants
if SentiWord_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=sentiment_sentiword, sentiment_type='lexicon')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'sentimentr_chaps': {'sentiment_max': 148.9999999999999,
  'sentiment_min': 3.1000000000000116},
 'sentimentr_parags': {'sentiment_max': 17.55, 'sentiment_min': -11.7},
 'sentimentr_sects': {'sentiment_max': 96.14999999999986,
  'sentiment_min': -17.149999999999977},
 'sentimentr_sents': {'sentiment_max': 8.9, 'sentiment_min': -6.4},
 'sentiword_chaps': {'sentiment_max': 42.542095634920614,
  'sentiment_min': -1.5734063492063277},
 'sentiword_parags': {'sentiment_max': 7.645183333333333,
  'sentiment_min': -6.530059523809523},
 'sentiword_sects': {'sentiment_max': 39.65655039682539,
  'sentiment_min': -14.01572857142857

### **Calculate SenticNet Sentiment Polarities (Optional: Auto)**

* https://sentic.net/

In [ ]:
model_base = 'senticnet'
model_name = 'senticnet_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
# Read Lexicon into DataFrame 

if SenticNet_Arc == True:

  lexicon_senticnet_df = get_lexicon('hash_sentiment_senticnet.csv')
  lexicon_senticnet_df['x'] = lexicon_senticnet_df['x'].astype('string')

  # Clean/Reorg DataFrame
  lexicon_senticnet_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
  lexicon_senticnet_df.rename(columns={'x':'word', 'y':'polarity'}, inplace=True)

  # Verify
  if (PLOT_OUTPUT == 'All'):
    lexicon_senticnet_df.head()
    lexicon_senticnet_df.info()

Found hash_sentiment_senticnet.csv in lexicon_directory)
cp_cmd = copy ../sa_lexicons/hash_sentiment_senticnet.csv ./
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23626 entries, 0 to 23625
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  23626 non-null  int64  
 1   x           23625 non-null  object 
 2   y           23626 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 553.9+ KB


In [ ]:
# Convert DataFrame to Dict[word] = polarity

if SenticNet_Arc == True:

  id = lexicon_senticnet_df.word.values
  values = lexicon_senticnet_df.polarity.values

  lexicon_senticnet_dt =dict(zip(id, values))
  # lexicon_jockersrinker_dt

  # Test
  sent_test='I hate Mondays.'
  text2sentiment(sent_test, lexicon_senticnet_dt)

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Sentiment evaluation function
def sentiment_senticnet(text_str):
  '''
  Given a text string
  Return corresponding sentiment value based on senticnet lexicon
  '''
  
  sentiment_val = text2sentiment(str(text_str), lexicon_senticnet_dt)

  return sentiment_val 

# Calculate all Sentiment values and variants
if SenticNet_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=sentiment_senticnet, sentiment_type='lexicon')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'senticnet_chaps': {'sentiment_max': 354.84699999999896,
  'sentiment_min': 235.29899999999932},
 'senticnet_parags': {'sentiment_max': 21.69200000000001,
  'sentiment_min': -5.824},
 'senticnet_sects': {'sentiment_max': 187.32800000000006,
  'sentiment_min': 42.55199999999999},
 'senticnet_sents': {'sentiment_max': 10.079999999999998,
  'sentiment_min': -4.052},
 'sentimentr_chaps': {'sentiment_max': 148.9999999999999,
  'sentiment_min': 3.1000000000000116},
 'sentimentr_parags': {'sentiment_max': 17.55, 'sentiment_min': -11.7},
 'sentimentr_sects': {'sentiment_max': 96.14999999999986,
  'sentiment_min': -17.1499999999

### **Calculate NCR Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'nrc'
model_name = 'nrc_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
# Read Lexicon into DataFrame 

if NCR_Arc == True:

  lexicon_nrc_df = get_lexicon('hash_sentiment_nrc.csv')
  lexicon_nrc_df['x'] = lexicon_nrc_df['x'].astype('string')

  # Clean/Reorg DataFrame
  lexicon_nrc_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
  lexicon_nrc_df.rename(columns={'x':'word', 'y':'polarity'}, inplace=True)

  # Verify
  if (PLOT_OUTPUT == 'All'):
    lexicon_nrc_df.head()
    lexicon_nrc_df.info()

Found hash_sentiment_nrc.csv in lexicon_directory)
cp_cmd = copy ../sa_lexicons/hash_sentiment_nrc.csv ./
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5468 entries, 0 to 5467
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5468 non-null   int64 
 1   x           5468 non-null   object
 2   y           5468 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 128.3+ KB


In [ ]:
# Convert DataFrame to Dict[word] = polarity

if NCR_Arc == True:

  id = lexicon_nrc_df.word.values
  values = lexicon_nrc_df.polarity.values

  lexicon_nrc_dt =dict(zip(id, values))
  # lexicon_jockersrinker_dt

In [ ]:
# Calculate NRC Sentiment [0,1,2]

def nrc_discrete2continous_sentiment(text):
  '''
  Given a plain text string, give it to
    Stanford Stanza (OpenNLP) to calculate sentiment for each word on a 3 point scale 0-2
  Return a sentiment value for the entire sentence (sum of word sentiments/log(len of sentence)) 
    that approximates a normal distribution for all values
    In order to get more fine grained measure of overall Sentence sentiment
    Sentiment values will be Normalized/Standardized so absolute precision is not required
  '''
  text_sentiment_total = 0.
  text_ls = text.split()
  text_len = len(text_ls)
  for aword in text_ls:
    text_sentiment_total += lex_discrete2continous_sentiment(str(aword), lexicon_nrc_dt)
  text_sentiment_norm = text_sentiment_total/(np.log(text_len)+10)

  return text_sentiment_norm
  

In [ ]:
# Test

if NCR_Arc == True:
  sent_test='I hate Mondays.'
  print(nrc_discrete2continous_sentiment(sent_test))

-9.010135447483094


In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Calculate all Sentiment values and variants
if NCR_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=nrc_discrete2continous_sentiment, sentiment_type='function')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'nrc_chaps': {'sentiment_max': 992.1420156684644,
  'sentiment_min': 136.36421709165032},
 'nrc_parags': {'sentiment_max': 133.15206794388564,
  'sentiment_min': -88.59571651997537},
 'nrc_sects': {'sentiment_max': 682.0582426859623,
  'sentiment_min': -126.83645806124086},
 'nrc_sents': {'sentiment_max': 92.90655652278458,
  'sentiment_min': -61.110488553550724},
 'senticnet_chaps': {'sentiment_max': 354.84699999999896,
  'sentiment_min': 235.29899999999932},
 'senticnet_parags': {'sentiment_max': 21.69200000000001,
  'sentiment_min': -5.824},
 'senticnet_sects': {'sentiment_max': 187.32800000000006,
  'sentiment_min':

### **Calculate Afinn Sentiment Polarities (Optional: Auto)**

* https://github.com/fnielsen/afinn

In [ ]:
model_base = 'afinn'
model_name = 'afinn_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
!pip install afinn

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp37-none-any.whl size=53452 sha256=7b521747ef9d87c87e10f4002f11ef976ca9e7a88cb49dcdd212f31146e967eb
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


In [ ]:
# Install and configure for English

from afinn import Afinn
afinn = Afinn(language='en')

# Test

# afinn.score('I had the worst day.')

In [ ]:
# Calculate AFINN Sentiment [0,1,2]

def afinn_discrete2continous_sentiment(text):
  '''
  Given a plain text string, give it to
    Stanford Stanza (OpenNLP) to calculate sentiment for each word on a 3 point scale 0-2
  Return a sentiment value for the entire sentence (sum of word sentiments/log(len of sentence)) 
    that approximates a normal distribution for all values
    In order to get more fine grained measure of overall Sentence sentiment
    Sentiment values will be Normalized/Standardized so absolute precision is not required
  '''
  text_sentiment_total = 0.
  text_ls = text.split()
  text_len = len(text_ls)
  for aword in text_ls:
    text_sentiment_total += afinn.score(aword)
  text_sentiment_norm = text_sentiment_total/(np.log(text_len)+0.1)

  return float(text_sentiment_norm)  # return float vs np.float64

In [ ]:
# Test

if AFINN_Arc == True:
  sent_test='I hate Mondays.'
  print(afinn_discrete2continous_sentiment(sent_test))

-2.5028944124489


In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Calculate all Sentiment values and variants
if AFINN_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=afinn_discrete2continous_sentiment, sentiment_type='function')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'afinn_chaps': {'sentiment_max': 23.566333953237162,
  'sentiment_min': -13.963760054626414},
 'afinn_parags': {'sentiment_max': 30.0, 'sentiment_min': -30.0},
 'afinn_sects': {'sentiment_max': 16.284196895116033,
  'sentiment_min': -15.574896102419311},
 'afinn_sents': {'sentiment_max': 40.0, 'sentiment_min': -30.0},
 'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'nrc_chaps': {'sentiment_max': 992.1420156684644,
  'sentiment_min': 136.36421709165032},
 'nrc_parags': {'sentiment_max': 133.15206794388564,
  'sentiment_min': -88.59571651997537},
 'nrc_sects': {'sentiment_max': 682.0582426859623,
  'sentiment_min': -126.83645806124086},
 'nrc_sents': {'sentiment_max':

### **Calculate VADER Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'vader'
model_name = 'vader_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
# Sentiment evaluation function
sid = SentimentIntensityAnalyzer()

# Test
sid.polarity_scores('hello world')

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

if VADER_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=sid.polarity_scores, sentiment_type='compound')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'afinn_chaps': {'sentiment_max': 23.566333953237162,
  'sentiment_min': -13.963760054626414},
 'afinn_parags': {'sentiment_max': 30.0, 'sentiment_min': -30.0},
 'afinn_sects': {'sentiment_max': 16.284196895116033,
  'sentiment_min': -15.574896102419311},
 'afinn_sents': {'sentiment_max': 40.0, 'sentiment_min': -30.0},
 'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'nrc_chaps': {'sentiment_max': 992.1420156684644,
  'sentiment_min': 136.36421709165032},
 'nrc_parags': {'sentiment_max': 133.15206794388564,
  'sentiment_min': -88.59571651997537},
 'nrc_sects': {'sentiment_max': 682.0582426859623,
  'sentiment_min': -126.83645806124086},
 'nrc_sents': {'sentiment_max':

### **Calculate TextBlob Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'textblob'
model_name = 'textblob_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
from textblob import TextBlob

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

def textblob_sentiment(text_str):
  '''
  Given a text string
  Return a sentiment value between -1.0 to +1.0 using TextBlob
  '''
  return TextBlob(text_str).sentiment.polarity

# Calculate all Sentiment values and variants
if TextBlob_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=textblob_sentiment, sentiment_type='function')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_name, text_unit='sentence')
get_lexstats(corpus_parags_df, model_name, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_name, text_unit='section')
get_lexstats(corpus_chaps_df, model_name, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'afinn_chaps': {'sentiment_max': 23.566333953237162,
  'sentiment_min': -13.963760054626414},
 'afinn_parags': {'sentiment_max': 30.0, 'sentiment_min': -30.0},
 'afinn_sects': {'sentiment_max': 16.284196895116033,
  'sentiment_min': -15.574896102419311},
 'afinn_sents': {'sentiment_max': 40.0, 'sentiment_min': -30.0},
 'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'nrc_chaps': {'sentiment_max': 992.1420156684644,
  'sentiment_min': 136.36421709165032},
 'nrc_parags': {'sentiment_max': 133.15206794388564,
  'sentiment_min': -88.59571651997537},
 'nrc_sects': {'sentiment_max': 682.0582426859623,
  'sentiment_min': -126.83645806124086},
 'nrc_sents': {'sentiment_max':

### **Calculate Pattern Sentiment Polarities (Optional: Auto)**

In [ ]:
model_base = 'pattern'
model_name = 'pattern_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
!pip install pattern

     |████████████████████████████████| 22.3MB 288kB/s 
     |████████████████████████████████| 92kB 6.9MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 5.6MB 14.0MB/s 
     |████████████████████████████████| 5.6MB 35.6MB/s 
     |████████████████████████████████| 430kB 38.5MB/s 
     |████████████████████████████████| 3.2MB 29.1MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
  Created wheel for pattern: filename=Pattern-3.6-cp37-none-any.whl size=22332722 sha256=13c7d32c8d2075c92024c6d00bb7db87db45aa789172fa7751b5c2f5bd169747
  Stored in directory: /root/.cache/pip/wheels/dc/9a/0e/5fb1a603ed4e3aa8722a88e9cf4a82da7d1b63e3d2cc34bee5
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp37-cp37m-linux_x86_64.whl size=100114 sha256=996cd4bf5c925c2a0fae68e2f254a935178b9cf9ea4f37fc9066c410f381bf8e
  Stored in directory: /root/.cache/pip/wheels/75/ca/e8/ad4e7ce3df18bcd91c7d84dd28c7c08db491a2a2360efed363
  Created wheel f

In [ ]:
from pattern.en import sentiment as pattern_sa

In [ ]:
# Test

sent_test='I hate Mondays.'
pattern_sa(sent_test)[0]

-0.8

In [ ]:
# Calculate Pattern Sentiment [0,1,2]

def pattern_discrete2continous_sentiment(text):
  '''
  Given a plain text string, give it to
    Stanford Stanza (OpenNLP) to calculate sentiment for each word on a 3 point scale 0-2
  Return a sentiment value for the entire sentence (sum of word sentiments/log(len of sentence)) 
    that approximates a normal distribution for all values
    In order to get more fine grained measure of overall Sentence sentiment
    Sentiment values will be Normalized/Standardized so absolute precision is not required
  '''
  text_sentiment_total = 0.
  text_ls = text.split()
  text_len = len(text_ls)
  for aword in text_ls:
    text_sentiment_total += pattern_sa(str(aword))[0]
  text_sentiment_norm = text_sentiment_total/(np.log(text_len)+0.01)

  return text_sentiment_norm

In [ ]:
# Test

if Pattern_Arc == True:
  sent_test='I hate Mondays.'
  print(pattern_discrete2continous_sentiment(sent_test))

-0.7216228867182435


In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# Calculate all Sentiment values and variants
if Pattern_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=pattern_discrete2continous_sentiment, sentiment_type='function')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'afinn_chaps': {'sentiment_max': 23.566333953237162,
  'sentiment_min': -13.963760054626414},
 'afinn_parags': {'sentiment_max': 30.0, 'sentiment_min': -30.0},
 'afinn_sects': {'sentiment_max': 16.284196895116033,
  'sentiment_min': -15.574896102419311},
 'afinn_sents': {'sentiment_max': 40.0, 'sentiment_min': -30.0},
 'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'nrc_chaps': {'sentiment_max': 992.1420156684644,
  'sentiment_min': 136.36421709165032},
 'nrc_parags': {'sentiment_max': 133.15206794388564,
  'sentiment_min': -88.59571651997537},
 'nrc_sects': {'sentiment_max': 682.0582426859623,
  'sentiment_min': -126.83645806124086},
 'nrc_sents': {'sentiment_max':

### **Calculate Stanza/OpenNLP Sentiment Polarities (Optional: Auto)**

* https://github.com/piyushpathak03/NLP-using-STANZA/blob/main/Stanza.ipynb

In [ ]:
model_base = 'stanza'
model_name = 'stanza_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
!pip install stanza

     |████████████████████████████████| 337kB 4.0MB/s 


In [ ]:
%time

import stanza

if Stanza_Arc == True:
  stanza.download('en')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


2021-07-08 10:52:01 INFO: Downloading default packages for language: en (English)...


2021-07-08 10:53:26 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
if Stanza_Arc == True:
  nlp = stanza.Pipeline('en', processors='tokenize,sentiment')

2021-07-08 10:53:26 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2021-07-08 10:53:27 INFO: Use device: cpu
2021-07-08 10:53:27 INFO: Loading: tokenize
2021-07-08 10:53:27 INFO: Loading: sentiment
2021-07-08 10:53:27 INFO: Done loading processors!


In [ ]:
# Test stanza directly

# doc = nlp('Ram is a bad boy')
# for i, sentence in enumerate(doc.sentences):
#     print(i, sentence.sentiment)

In [ ]:
# Calculate Stanza Sentiment [0,1,2]

def stanza_discrete2continous_sentiment(text):
  '''
  Given a plain text string, give it to
    Stanford Stanza (OpenNLP) to calculate sentiment for each word on a 3 point scale 0-2
  Return a sentiment value for the entire sentence (sum of word sentiments/log(len of sentence)) 
    that approximates a normal distribution for all values
    In order to get more fine grained measure of overall Sentence sentiment
    Sentiment values will be Normalized/Standardized so absolute precision is not required
  '''
  text_sentiment_tot = 0.
  text_ls = text.split()
  text_len = len(text_ls)
  for aword in text_ls:
    adoc = nlp(aword)
    for i, sentence in enumerate(adoc.sentences):
      text_sentiment_tot += float(sentence.sentiment)
  text_sentiment_norm = text_sentiment_tot/(np.log(text_len)+0.1)

  return text_sentiment_norm

In [ ]:
# Test

if Stanza_Arc == True:
  sent_test='I hate Mondays.'
  print(stanza_discrete2continous_sentiment(sent_test))

1.6685962749659333


In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# NOTE: requires about 50mins (20210708 at 0730) Colab Pro: GPU+RAM

# Calculate all Sentiment values and variants
if Stanza_Arc == True:
  get_sentiments(model_base=model_base, sentiment_fn=stanza_discrete2continous_sentiment, sentiment_type='function')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

{'afinn_chaps': {'sentiment_max': 23.566333953237162,
  'sentiment_min': -13.963760054626414},
 'afinn_parags': {'sentiment_max': 30.0, 'sentiment_min': -30.0},
 'afinn_sects': {'sentiment_max': 16.284196895116033,
  'sentiment_min': -15.574896102419311},
 'afinn_sents': {'sentiment_max': 40.0, 'sentiment_min': -30.0},
 'bing_chaps': {'sentiment_max': 591.1048063253189,
  'sentiment_min': -665.5682824208717},
 'bing_parags': {'sentiment_max': 233.62539105697772,
  'sentiment_min': -231.75792169846113},
 'bing_sects': {'sentiment_max': 721.8836270105633,
  'sentiment_min': -635.8012840187225},
 'bing_sents': {'sentiment_max': 143.60753220597746,
  'sentiment_min': -159.0419852720603},
 'nrc_chaps': {'sentiment_max': 992.1420156684644,
  'sentiment_min': 136.36421709165032},
 'nrc_parags': {'sentiment_max': 133.15206794388564,
  'sentiment_min': -88.59571651997537},
 'nrc_sects': {'sentiment_max': 682.0582426859623,
  'sentiment_min': -126.83645806124086},
 'nrc_sents': {'sentiment_max':

In [ ]:
corpus_sents_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7679 entries, 0 to 7680
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sent_no                     7679 non-null   int64  
 1   parag_no                    7679 non-null   int64  
 2   sent_raw                    7679 non-null   string 
 3   char_len                    7679 non-null   int64  
 4   token_len                   7679 non-null   int64  
 5   sent_clean                  7679 non-null   object 
 6   sentimentr                  7679 non-null   float64
 7   sentimentr_lnorm_meanstd    7679 non-null   float64
 8   sentimentr_medianiqr        7679 non-null   float64
 9   sentimentr_lnorm_medianiqr  7679 non-null   float64
 10  syuzhet                     7679 non-null   float64
 11  syuzhet_lnorm_meanstd       7679 non-null   float64
 12  syuzhet_medianiqr           7679 non-null   float64
 13  syuzhet_lnorm_medianiqr     7679 

### **Calculate Huggingface BERT Default Sentiment Polarities (Optional: Auto)**

In [ ]:
!pip install transformers

In [ ]:
MAX_LINE_LEN = 509

model_base = 'hfbert'
model_name = 'hfbert_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Sentiment analysis pipeline
hf_sadef_clf = pipeline('sentiment-analysis')

# Test
hf_sadef_clf('Such a nice weather outside !')

In [ ]:
def hfbert_sentiment(text_str='Happy Happy Joy Joy'):
  '''
  Given a text string
  Return a float sentiment value for this text string with the given model/sentiment_fn
  '''

  text_str_ls = text_str.split(' ')
  if len(text_str_ls) > MAX_LINE_LEN:
    text_str = string_trimlen(atext_str=text_str, max_len=MAX_LINE_LEN)

  sentiment_res = hf_sadef_clf(text_str)

  # print(f"sentiment_res[0]: {sentiment_res[0]['label']}")
  if sentiment_res[0]['label'].lower() == 'positive':
    # print('POSITIVE')
    sign_multiplier = 1.0
  else:
    # print('NEGATIVE')
    sign_multiplier = -1.0

  sentiment_value = sign_multiplier * sentiment_res[0]['score']

  return sentiment_value

# Test
# test_res = hfbert_sentiment(text_str='the it is')
# print(test_res)

In [ ]:
# Test
test_res = hfbert_sentiment(text_str='the it is')
print(test_res)

In [ ]:
[x.upper() for x in 'the cat sat'.split()]

In [ ]:
def string_trimlen(atext_str, max_len):
  '''
  Given a text string of tokens
  Return a truncated string of max_len tokens if necessary
  e.g. Transformers can often only process 512 (509/510 with special tokens)
  '''
  
  common_neutral_words_ls = "the of and to a in for is on that by this with i you it or be are from at as your have an was we will us about page my has our information time they site he what which their use there his when here who these its x than had year day into two email n re b them t number then where m through she years r said de".split(' ')
  # common_neutral_words_ls = 'the of to and a an is it in you that he was are I his they be at this or had by word what some we were there when your said an each she which their if about then them these here thing him two has did number sound people who been place where day year'.split(' ')
  # print(f'len common: {len(common_neutral_words_ls)}')
  # text_str_ls = [x.lower for x in text_str.split()]
  text_str_ls = atext_str.split()
  # print(f'text_str_ls type: {type(text_str_ls[0])}')

  if len(text_str_ls) <= max_len:
    return text_str
  
  for i, aneutral_word in enumerate(common_neutral_words_ls):
    string_current = str(" ".join([str(x) for x in text_str_ls]))
    # print(f"top of loop with aword={aneutral_word} and text_str_ls: {str(string_current)}")
    aneutral_word_lower = aneutral_word.lower()
    for aused_word in text_str_ls:
      aused_word_lower = aused_word.lower()
      # print(f'Comparing aused_word={aused_word_lower} with aneutral_word={aneutral_word_lower}')
      if aused_word_lower == aneutral_word_lower:
        # if aword_lower in text_str_ls:
        # print(f'REMOVING aword=[{aused_word_lower}] from sentence')
        # while aused_word_lower in text_str_ls: text_str_ls.remove(aused_word)
        text_str_ls = list(filter((aused_word).__ne__, text_str_ls))
        # text_str_ls.remove(aneutral_word, )
        # print(f"   new shorter list len = {len(text_str_ls)}, string={' '.join(text_str_ls)}")
    if len(text_str_ls) <= max_len:
      # print('returning')
      text_condensed_str = ' '.join([x for x in text_str_ls])
      return text_condensed_str

    # print(f'iteration #{i}')

  text_condensed_str = ' '.join([x for x in text_str_ls])
  return text_condensed_str

# Test
original_string = "The rain in Spain falls mainly in the plain"
print(f'original string: {original_string}')
trimmed_string = string_trimlen(atext_str=original_string, max_len=5)
print(f'trimmed string: {trimmed_string}')



original string: The rain in Spain falls mainly in the plain
trimmed string: rain Spain falls mainly plain


In [ ]:
corpus_sents_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7679 entries, 0 to 7680
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sent_no                     7679 non-null   int64  
 1   parag_no                    7679 non-null   int64  
 2   sent_raw                    7679 non-null   string 
 3   char_len                    7679 non-null   int64  
 4   token_len                   7679 non-null   int64  
 5   sent_clean                  7679 non-null   object 
 6   scores                      7679 non-null   object 
 7   vader                       7679 non-null   float64
 8   vader_lnorm_meanstd         7679 non-null   float64
 9   vader_medianiqr             7679 non-null   float64
 10  vader_lnorm_medianiqr       7679 non-null   float64
 11  textblob                    7679 non-null   float64
 12  textblob_lnorm_meanstd      7679 non-null   float64
 13  textblob_medianiqr          7679 

In [ ]:
list(corpus_sents_df.iloc[:20].groupby('parag_no').vader_lnorm_medianiqr.sum())

[1.6180915371329878,
 0.0,
 0.0,
 10.354832627539965,
 -2.187564766839378,
 0.8156377571047255,
 1.821243523316062]

In [ ]:
def get_transformer_sentiments(model_base, sentiment_fn, sentiment_type='value_fl'):
  '''
  Given a model_base name and sentiment evaluation function
  Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants
  '''

  # Calculate Sentiment Polarities

  if sentiment_type == 'value_fl':

    corpus_sents_df[model_base] = corpus_sents_df['sent_raw'].apply(lambda text: hfbert_sentiment(str(text)))
    corpus_parags_df[model_base] = corpus_sents_df.iloc[:20].groupby('parag_no').vader_lnorm_medianiqr.sum()
    
    # corpus_sents_df[model_base] = corpus_sents_df['sent_clean'].apply(lambda text: sentiment_fn(str(text)))
    # corpus_parags_df[model_base] = corpus_parags_df['parag_clean'].apply(lambda text: sentiment_fn(str(text)))
    # corpus_sects_df[model_base] = corpus_sects_df['sect_clean'].apply(lambda text: sentiment_fn(str(text)))
    # corpus_chaps_df[model_base] = corpus_chaps_df['chap_clean'].apply(lambda text: sentiment_fn(str(text)))
  
  elif sentiment_type == 'compound':
    # VADER

    # Calculate dictionary of {neg/neu/pos/compound} values for sent_clean
    corpus_sents_df['scores'] = corpus_sents_df['sent_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_parags_df['scores'] = corpus_parags_df['parag_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_sects_df['scores'] = corpus_sects_df['sect_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_chaps_df['scores'] = corpus_chaps_df['chap_clean'].apply(lambda text: sentiment_fn(str(text)))

    # Extract Compound Sentiment
    corpus_sents_df[model_base]  = corpus_sents_df['scores'].apply(lambda score_dict: score_dict['compound'])
    corpus_parags_df[model_base]  = corpus_parags_df['scores'].apply(lambda score_dict: score_dict['compound'])
    corpus_sects_df[model_base]  = corpus_sects_df['scores'].apply(lambda score_dict: score_dict['compound'])
    corpus_chaps_df[model_base]  = corpus_chaps_df['scores'].apply(lambda score_dict: score_dict['compound'])

  elif sentiment_type == 'function':
    # TextBlob

    # Calculate dictionary of {neg/neu/pos/compound} values for sent_clean
    corpus_sents_df[model_base] = corpus_sents_df['sent_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_parags_df[model_base] = corpus_parags_df['parag_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_sects_df[model_base] = corpus_sects_df['sect_clean'].apply(lambda text: sentiment_fn(str(text)))
    corpus_chaps_df[model_base] = corpus_chaps_df['chap_clean'].apply(lambda text: sentiment_fn(str(text)))

  else:
    print(f'ERROR: sentiment_type={sentiment_type} but must be one of (lexicon, compound, function)')
    return


  # Create new column names
  col_meanstd = f'{model_base}_meanstd'
  col_medianiqr = f'{model_base}_medianiqr'
  col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'
  col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'

  return

"""
  # Get Chapter Standardization with MeanSTD and RobustStandardization with MedianIQRScaling
  corpus_chaps_df[col_meanstd]  = mean_std_scaler.fit_transform(np.array(corpus_chaps_df[model_base]).reshape(-1, 1))
  corpus_chaps_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_chaps_df[model_base]).reshape(-1, 1))
  # Normalize the Chapter Sentiment by dividing by Chapter Length
  chaps_len_ls = list(corpus_chaps_df['token_len'])
  chaps_sentiment_ls = list(corpus_chaps_df[model_base])
  chaps_sentiment_norm_ls = [chaps_sentiment_ls[i]/chaps_len_ls[i] for i in range(len(chaps_len_ls))]
  # RobustStandardize Chapter sentiment values
  corpus_chaps_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(chaps_sentiment_norm_ls)).reshape(-1, 1))
  corpus_chaps_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_chaps_df[model_base]).reshape(-1, 1))
  corpus_chaps_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(chaps_sentiment_norm_ls)).reshape(-1, 1))

  # Get Section Standardization with MeanSTD and RobustStandardization with MedianIQRScaling
  corpus_sects_df[col_meanstd]  = mean_std_scaler.fit_transform(np.array(corpus_sects_df[model_base]).reshape(-1, 1))
  corpus_sects_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_sects_df[model_base]).reshape(-1, 1))
  # Normalize the Section Sentiment by dividing by Section Length
  sects_len_ls = list(corpus_sects_df['token_len'])
  sects_sentiment_ls = list(corpus_sects_df[model_base])
  sects_sentiment_norm_ls = [sects_sentiment_ls[i]/sects_len_ls[i] for i in range(len(sects_len_ls))]
  # RobustStandardize Section sentiment values
  corpus_sects_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(sects_sentiment_norm_ls)).reshape(-1, 1))
  corpus_sects_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_sects_df[model_base]).reshape(-1, 1))
  corpus_sects_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(sects_sentiment_norm_ls)).reshape(-1, 1))

  # Normalize the Paragraph Sentiment by dividing by Chapter Length
  parags_len_ls = list(corpus_parags_df['token_len'])
  parags_sentiment_ls = list(corpus_parags_df[model_base])
  parags_sentiment_norm_ls = [parags_sentiment_ls[i]/parags_len_ls[i] for i in range(len(parags_len_ls))]
  # RobustStandardize Paragraph sentiment values
  corpus_parags_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(parags_sentiment_norm_ls)).reshape(-1, 1))
  corpus_parags_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_parags_df[model_base]).reshape(-1, 1))
  corpus_parags_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(parags_sentiment_norm_ls)).reshape(-1, 1))

  # Normalize the Sentence Sentiment by dividing by Chapter Length
  sents_len_ls = list(corpus_sents_df['token_len'])
  sents_sentiment_ls = list(corpus_sents_df[model_base])
  sents_sentiment_norm_ls = [sents_sentiment_ls[i]/sents_len_ls[i] for i in range(len(sents_len_ls))]
  # RobustStandardize Sentence sentiment values
  corpus_sents_df[col_lnorm_meanstd]  = mean_std_scaler.fit_transform(np.array(pd.Series(sents_sentiment_norm_ls)).reshape(-1, 1))
  corpus_sents_df[col_medianiqr]  = median_iqr_scaler.fit_transform(np.array(corpus_sents_df[model_base]).reshape(-1, 1))
  corpus_sents_df[col_lnorm_medianiqr]  = median_iqr_scaler.fit_transform(np.array(pd.Series(sents_sentiment_norm_ls)).reshape(-1, 1))
""";

In [ ]:
model_base

'hfbert'

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

# NOTE: 

get_transformer_sentiments(model_base=model_base, sentiment_fn=hfbert_sentiment, sentiment_type='value_fl')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

### **NLPTown BERT**

In [ ]:
!pip3 install torch

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
model_base = 'nlptown'
model_name = 'nlptown_lnorm_medianiqr'

col_medianiqr = f'{model_base}_medianiqr'
col_meanstd = f'{model_base}_meanstd'

col_lnorm_medianiqr = f'{model_base}_lnorm_medianiqr'
col_lnorm_meanstd = f'{model_base}_lnorm_meanstd'

In [ ]:
def nlptown_sentiment(text_str):
  '''
  Given a text string
  Return the Sentiment Value computed by BERT NLPTown
  '''
  
  tokens = tokenizer.encode(text_str, return_tensors='pt')

  # Predict Tokens
  result = model(tokens)
  result

  sentiment_base = int(torch.argmax(result.logits)) # Sentiment 0-4

  # Normalize all 5 probabilities sum to 1
  result_np = result.logits.detach().numpy()
  result_sum_fl = np.absolute(result_np).sum()
  result_norm_np = result_np.T/result_sum_fl
  result_ls = list(result_norm_np)

  # Adjusted Sentiment
  sentiment_adj = result_ls[sentiment_base]
  sentiment_final_np = sentiment_base + sentiment_adj

  sentiment_final_float = float(sentiment_final_np)

  return sentiment_final_float


"""
  # return predict_sentiment, result.logits.detach().numpy()


# Test
test_sentiment, result_np = get_sentiment_nlptown('I love and adore the wonderfully excellent things.')
test_sentiment, result_np = get_sentiment_nlptown('I hate and despise the terribly rotten things.')
test_sentiment, result_np = get_sentiment_nlptown('What it is.')
test_text = 'I love and adore the wonderfully excellent things.'
test_text = 'I hate and despise the terribly rotten things.'
test_text = 'What is it'
test_text = 'The sky is blue'

test_sentiment, result_np = get_sentiment_nlptown(test_text)


print(f'test_sentiment={test_sentiment}')
print(f'result_np.shape = {result_np.shape}')
result_sum_fl = np.absolute(result_np).sum()
result_norm_np = result_np.T/result_sum_fl
print(f'result_norm_np = {result_norm_np}')

print(f'result_ls = {len(result_ls)}')
adj_test_sentiment = result_ls[test_sentiment-1] #  + abs(result_norm_np[test_sentiment-1])
print(f'adj test_sentiment={test_sentiment- 1 + adj_test_sentiment}')
""";

# Test
# test_text = 'I love and adore the wonderfully excellent things.'
test_text = 'I hate and despise the terribly rotten things.'
# test_text = 'What is it'
# test_text = 'The sky is blue'

sentiment_text = get_sentiment_nlptown(test_text)
print(f'sentiment_text: {sentiment_text}');

In [ ]:
# Calculate all the Sentence, Paragraph, Section and Chapter Sentiment Scores and Standardized variants

get_sentiments(model_base=model_base, sentiment_fn=nlptown_sentiment, sentiment_type='function')

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, model_base, text_unit='sentence')
get_lexstats(corpus_parags_df, model_base, text_unit='paragraph')
get_lexstats(corpus_sects_df, model_base, text_unit='section')
get_lexstats(corpus_chaps_df, model_base, text_unit='chapter')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

### **RoBERTa Large English Tuned on 15 SA Dataset**
* **siebert/sentiment-roberta-large-english**

This model is a fine-tuned checkpoint of RoBERTa-large (Liu et al. 2019). It enables reliable binary sentiment analysis for various types of English-language text. For each instance, it predicts either positive (1) or negative (0) sentiment. The model was fine-tuned and evaluated on 15 data sets from diverse text sources to enhance generalization across different types of texts (reviews, tweets, etc.). Consequently, it outperforms models trained on only one type of text (e.g., movie reviews from the popular SST-2 benchmark) when used on new data as shown below.

Jon Chun
20 Jun 2021

Reference:

* https://huggingface.co/siebert/sentiment-roberta-large-english

* https://huggingface.co/siebert/sentiment-roberta-large-english

In [ ]:
from transformers import pipeline

In [ ]:
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

# Test
print(sentiment_analysis("I love this!"))

In [ ]:
def robertalg15_sentiment(sentiment_fn, text_str='Happy Happy Joy Joy'):
  '''
  Given a sentiment function and text string
  Return a float sentiment value for this text string with the given model/sentiment_fn
  '''

  sentiment_res = sentiment_fn(text_str)

  # print(f"sentiment_res[0]: {sentiment_res[0]['label']}")
  if sentiment_res[0]['label'].lower() == 'positive':
    # print('POSITIVE')
    sign_multiplier = 1.0
  else:
    # print('NEGATIVE')
    sign_multiplier = -1.0

  sentiment_value = sign_multiplier * sentiment_res[0]['score']

  return sentiment_value

# Test
# test_res = get_sentiment_hfbert(hf_sadef_clf, text_str='the it is')
# print(test_res)

In [ ]:
%time

# NOTE: upto 30-45mins

# Calculate Sentiment Polarities

# Calculate dictionary of {neg/neu/pos/compound} values for sent_clean
# corpus_sents_df['nrc'] = corpus_sents_df['sent_clean'].apply(lambda text: lex_discrete2continous_sentiment(str(text), lexicon_nrc_dt))
corpus_sents_df['robertalg15'] = corpus_sents_df['sent_clean'].apply(lambda text: get_labelscore2sentiment(sentiment_analysis, str(text)))
corpus_parags_df['robertalg15'] = corpus_parags_df['parag_clean'].apply(lambda text: get_labelscore2sentiment(sentiment_analysis, str(text)))

# Normalize by Paragraph Length
# TODO:

if (PLOT_OUTPUT == 'All'):
  corpus_sents_df.head(2)
  corpus_parags_df.head(2)

In [ ]:
parags_sents_median_ls = corpus_sents_df.groupby(['parag_no'])['robertalg15'].median()
print(f'type(parags_sent_median_ls): {type(parags_sents_median_ls)}')
print(f'len(parags_sent_median_ls): {len(parags_sents_median_ls)}')
print(f'parags_sent_median_ls: {parags_sents_median_ls}')

# corpus_parags_df['nlptown_sentsmedian'] = corpus_parags_df['parag_clean'].apply(lambda text: get_sentiment_nlptown(str(text)))

In [ ]:
corpus_parags_df.shape

In [ ]:
corpus_parags_df['robertalg15_sentsmedian'] = pd.Series(parags_sents_median_ls)

In [ ]:
corpus_parags_df[['robertalg15', 'robertalg15_sentsmedian']].rolling(70).mean().plot()

In [ ]:
# Get/Set Sentiment Statistics

get_lexstats(corpus_sents_df, 'robertalg15', text_unit='sentence')
get_lexstats(corpus_parags_df, 'robertalg15', text_unit='paragraph')

# Validate
corpus_lexicons_stats_dt

# corpus_lexicons_stats_dt['vader']['sents']['sentiment_max']

In [ ]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [ ]:
# Setup for RoBERTa Large English 15datasets: siebert/sentiment-roberta-large-english

sa_model = 'robertalg15'

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}


# Set model
sa_model = 'robertalg15'
model_name = "siebert/sentiment-roberta-large-english"

# Load tokenizer and model, create trainer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model);

In [ ]:
# Prepare Text from DataFrame

sents_pred_df = corpus_sents_df.copy()
sents_pred_texts = sents_pred_df['sent_raw'].astype('str').tolist() # Want to catch NaN, .dropna().astype('str').tolist()

# Tokenize texts and create prediction data set

sents_tokenized_texts = tokenizer(sents_pred_texts,truncation=True,padding=True)
sents_pred_dataset = SimpleDataset(sents_tokenized_texts)

In [ ]:
def polprob2sentiment(pol_str, prob_fl):
  '''
  Given a Polarity string (Negative or Positive) and a Probability float (0.0-1.0)
  Return a Sentiment float value (-1.0 to 1.0)
  '''
  sign_fl = 1.0
  if pol_str.lower().startswith('neg'):
    # print(f'pol_str: {pol_str} is Negative')
    sign_fl = -1.0
  elif pol_str.lower().startswith('pos'):
    # print(f'pol_str: {pol_str} is Positive')
    pass
  else:
    print(f'ERROR: pol_str: {pol_str} is neither Negative nor Positive')
    sign_fl = 0.0

  return sign_fl * prob_fl

# Test
polprob2sentiment('Positive', 0.91)

In [ ]:
temp_ser = temp_sentiment_df.apply(lambda x: polprob2sentiment(str(x.label), float(x.score)))

In [ ]:
# Transform predictions to labels

# sents_preds = sents_predictions.predictions.argmax(-1)
sents_preds = sents_pred_dataset.predictions.argmax(-1)


sents_labels = pd.Series(sents_preds).map(model.config.id2label)
sents_scores = (np.exp(sents_predictions[0])/np.exp(sents_predictions[0]).sum(-1,keepdims=True)).max(1)

# Create DataFrame with texts, predictions, labels, and scores

temp_sentiment_df = pd.DataFrame(list(zip(sents_pred_texts,sents_preds,sents_labels,sents_scores)), columns=['text','pred','label','score'])
# temp_sentiment_df.head()

# Convert label (Neg/Pos) and score (Prob) to a +/-Sentiment Float Value

corpus_sents_df[sa_model] = temp_sentiment_df.apply(lambda x: polprob2sentiment(x.label,x.score), axis=1)
corpus_sents_df.head()

In [ ]:
# Aggregate Sentence Sentiments to populate Paragraph Sentiment DataFrame

# def sentiment_sents2parags(ts_df, model_name='roberta_lg15'):
parags_sentiment_ls = sentiment_sents2parags(corpus_sents_df, sa_model)
corpus_parags_df[sa_model] = pd.Series(parags_sentiment_ls)
corpus_parags_df.head(2)
corpus_parags_df.tail(2)

# **EDA (Repeat for each Sentiment Model)** (Auto)

#### **Histograms**

**Sentiment Histogram Plots**

INSTRUCTIONS:

* Histograms are provided for the (a) Length-Normed and (b) Scaled (Median Interquartile Range) Sentiment values for Sentences, Paragraphs and Sections. 

* There we used extensively early on to compare which Sentiment Time series preprocessing techniques worked best with our various Novel corpora according to two criteria: 

* (a) Vertical Scaling Method with the ability to transform histograms of sentiment values to well-behaved near-gaussian distributions and clipping outliers. After experimenting with various techniques including: mean/STD, median/MAD, and various two stage outlier/normalization methods median/IQR proved best (define).

* (b) Horizontal Scaling

In [ ]:
plot_histogram(model_name=model_name, text_unit='sentence', save2file=False)

In [ ]:
# col_medianiqr = f'{model_name}_medianiqr'
plot_histogram(model_name=col_medianiqr, text_unit='sentence', save2file=False)

In [ ]:
corpus_sents_df.info()

In [ ]:
# Plot Histogram of Sentence lengths

# col_lnorm_medianiqr = f'{model_name}_lnorm_medianiqr'
plot_histogram(model_name=col_lnorm_medianiqr, text_unit='sentence', save2file=False)

In [ ]:
plot_histogram(model_name=model_name, text_unit='paragraph', save2file=False)

In [ ]:
# col_medianiqr = f'{model_name}_medianiqr'
plot_histogram(model_name=col_medianiqr, text_unit='paragraph', save2file=False)

In [ ]:
# Plot Histogram of Paragraph lengths

# col_lnorm_medianiqr = f'{model_name}_lnorm_medianiqr'
plot_histogram(model_name=col_lnorm_medianiqr, text_unit='paragraph', save2file=False)

In [ ]:
plot_histogram(model_name=model_name, text_unit='section', save2file=False)

In [ ]:
# col_medianiqr = f'{model_name}_medianiqr'
plot_histogram(model_name=col_medianiqr, text_unit='section', save2file=False)

In [ ]:
# Plot Histogram of Section lengths

# col_lnorm_medianiqr = f'{model_name}_lnorm_medianiqr'
plot_histogram(model_name=col_lnorm_medianiqr, text_unit='section', save2file=False)

#### **Raw Sentiment Plots**

In [ ]:
plot_raw_sentiments(model_name=model_name, semantic_type='sentence', save2file=False)

In [ ]:
# col_medianiqr = f'{model_name}_medianiqr'
plot_raw_sentiments(model_name=col_medianiqr, semantic_type='sentence', save2file=False)

In [ ]:
# Plot Raw Sentence Sentiments

# col_lnorm_medianiqr = f'{model_name}_lnorm_medianiqr'
plot_raw_sentiments(model_name=col_lnorm_medianiqr, semantic_type='sentence', save2file=False)

In [ ]:
plot_raw_sentiments(model_name=model_name, semantic_type='paragraph', save2file=False)

In [ ]:
# col_medianiqr = f'{model_name}_medianiqr'
plot_raw_sentiments(model_name=col_medianiqr, semantic_type='paragraph', save2file=False)

In [ ]:
# Plot Raw Paragraph Sentiments

# col_lnorm_medianiqr = f'{model_name}_lnorm_medianiqr'
plot_raw_sentiments(model_name=model_name, semantic_type='paragraph', save2file=False)

In [ ]:
# TODO: Add Section Crux Nos

In [ ]:
plot_raw_sentiments(model_name=model_name, semantic_type='section', save2file=False)

plt.legend(loc='best')

In [ ]:
# col_medianiqr = f'{model_name}_medianiqr'
# col_meanstd = f'{model_name}_meanstd'

plot_raw_sentiments(model_name=col_medianiqr, semantic_type='section', save2file=False)
plot_raw_sentiments(model_name=col_meanstd, semantic_type='section', save2file=False)

plt.legend(loc='best')

In [ ]:
# Plot Raw Standardized Section Sentiments

# NOTE: Compared with Length-Normed, the Raw Standardizations lose most SATS features

# col_lnorm_medianiqr = f'{model_name}_lnorm_medianiqr'
# col_lnorm_meanstd = f'{model_name}_lnorm_meanstd'

plot_raw_sentiments(model_name=col_lnorm_medianiqr, semantic_type='section', save2file=False)
plot_raw_sentiments(model_name=col_lnorm_meanstd, semantic_type='section', save2file=False)

plt.legend(loc='best')

In [ ]:
# Plot Raw Standardized Chapter Sentiments

# col_lnorm_medianiqr = f'{model_name}_lnorm_medianiqr'
# col_lnorm_meanstd = f'{model_name}_lnorm_meanstd'

plot_raw_sentiments(model_name=col_lnorm_medianiqr, semantic_type='chapter', save2file=False)
plot_raw_sentiments(model_name=col_lnorm_meanstd, semantic_type='chapter', save2file=False)

#### **Crux Points and Surrounding Contexts**

In [ ]:
# Veify all the model sentiment variations

[x for x in corpus_sects_df.columns if x.startswith(model_base)]

**Compare Chapters vs Sections Crux Points**

INSTRUCTIONS:

* At the highest level, the Corpus is divided into Chapters which may then futher subdivided into Sections (e.g. extra spaces, punctuation like '* * *' or special printer glyph/fleuron).  Horizonal dark blue lines indicate Chapter divisions while Section boundries lie at both dark and light blue vertical lines.

* Since each Chapter may contain multiple Sections, the Section Sentiment plot is more detailed/jagged than the Chapter Sentiment plots. By plotting both together, the smoother Chapter Sentiment plot gives a more general sense of the Corpus Sentiment Arc while the next, more-detailed Section Sentiment plot enables a more detailed investigation/localization of Crux Point neighborhoods.

* At this early stage both the Chapter and Section Sentiment plots are too general to provide accurate/fixed Crux localization. As such, only aggregrate Sentiment values for each Chapter/Section are assigned to the mid-point of each Chapter/Section.

In [ ]:
col_lnorm_medianiqr

In [ ]:
# col_lnorm_medianiqr = 'vader_lnorm_medianiqr'

# corpus_chaps_df.drop(columns=['textblob_lnorm_medianiqr_lnorm_medianiqr', 'textblob_lnorm_medianiqr_medianiqr', 'textblob_lnorm_medianiqr_lnorm_meanstd'], inplace=True, axis=1)

In [ ]:
corpus_chaps_df.shape

In [ ]:
corpus_chaps_df.columns
# corpus_chaps_df.iloc[:20][['chap_no','sent_no_start','sent_no_mid','char_len','token_len','vader', 'vader_lnorm_medianiqr', 'textblob', 'textblob_lnorm_medianiqr']]

In [ ]:
corpus_chaps_df.vader_lnorm_medianiqr.min()

In [ ]:
# Plot Annotated Section Cruxes of Standardized Sentiment Time Series

sec_y_ht = -20

corpus_cruxes_dt[col_lnorm_medianiqr] = plot_crux_sections(model_names_ls=[col_lnorm_medianiqr], semantic_type='chapter', label_token_ct=3, title_xpos=0.8, title_ypos=1.05, sec_y_height=sec_y_ht, save2file=False)

corpus_cruxes_dt[col_lnorm_medianiqr] = plot_crux_sections(model_names_ls=[col_lnorm_medianiqr], semantic_type='section', label_token_ct=-1, title_xpos=0.8, title_ypos=1.05, sec_y_height=sec_y_ht, save2file=False)


**Sections Crux Points in Detail**

In [ ]:
corpus_chaps_df.info()

In [ ]:
# Plot Annotated Section Cruxes of Standardized Sentiment Time Series
sec_y_ht = -1.2

corpus_cruxes_dt[col_lnorm_medianiqr] = plot_crux_sections(model_names_ls=[col_lnorm_medianiqr], semantic_type='section', label_token_ct=5, title_xpos=0.8, title_ypos=1.05, sec_y_height=sec_y_ht, save2file=False)


**Verify Crux Point Sentence Number and Text Match**

INSTRUCTIONS:

* At [Crux_Sentence_Text] enter the first few words that uniquely identify the Crux Sentence and confirm the Sentence No matches the information in the plot above. (NOTE: Search is for an exact match including case and puncutation)

In [ ]:
Crux_Sentence_Text = "I was impatient to" #@param {type:"string"}

# Verify individual Crux Sentence Numbers matches Content

crux_sent_no = int(corpus_sents_df[corpus_sents_df['sent_raw'].str.contains(Crux_Sentence_Text)]['sent_no'])

print(f'The Sentence:\n\n    {Crux_Sentence_Text}\n\nMatches Sentence #{crux_sent_no}')


**Review Context Around Any Crux Point**

INSTRUCTIONS:

* Enter [Crux_Sentence_No] that matches a Crux point/Sentence No you want to explore

* Enter [No_Paragraphs_on_Each_Side] to retrieve this many Paragraphs before and after the Paragraph containing your Crux Sentence (e.g. 2 will bring back 5 paragraphs centered around the Paragraph containing the Crux Sentence)

* Select [Highlight_Crux_Sentence] to have the Crux Sentence converted to ALL CAPS for easier identification. The Paragraph containing the Crux Sentence will be prefaced with a '<*>' as well.

In [ ]:
# Select details about the Crux Point Context to Retrieve

# print(f'Last Sentence No: {corpus_sents_df.shape[0]}')
Crux_Sentence_No =  1051#@param {type:"number"}
No_Paragraphs_on_Each_Side = 2 #@param {type:"slider", min:0, max:10, step:1}
Highlight_Crux_Sentence = True #@param {type:"boolean"}

corpus_sents_len = corpus_sents_df.shape[0]

# if (Crux_Sentence_No >= No_Paragraphs_on_Each_Side) & (Crux_Sentence_No+No_Paragraphs_on_Each_Side <= corpus_parag_len):
# get_sentnocontext_report()
# try:
get_sentnocontext_report(the_sent_no=Crux_Sentence_No, the_n_sideparags=No_Paragraphs_on_Each_Side, the_sent_highlight=Highlight_Crux_Sentence)
# except:
#   print('ERROR')
# else:
#   print(f'ERROR: The combination of your [Crux_Sentence_No] and [No_Pargraphs_on_Each_Side]\n       results in a window outside the range of the Corpus Paragraphs.\n\n       Try again with different values.')

**Compare Paragraph vs Section Crux Points**

In [ ]:
# Verify the valid ranges for Sentences and Paragraphs

corpus_sents_len = corpus_sents_df.shape[0]
print(f'There are {corpus_sents_len} Sentences in the Corpus')
corpus_parags_len = corpus_parags_df.shape[0]
print(f'There are {corpus_parags_len} Paragraphs in the Corpus')

# Create a new Corpus Paragraph DataFrame (corpus_parags_zoom_df) that is streteched out to have as many sample points as there are Sentences
#   That is, go from an original corpus_parags_df of #Paragraph datapoints to an expanded corpus_parags_zoom_df of #Sentences datapoints using scipy.ndimage.interpolation.zoom

corpus_parags_zoom_df = pd.DataFrame()

resample_ratio = corpus_sents_df.shape[0]/corpus_parags_df.shape[0]   # ratio = no_sents/no_parags

corpus_parags_df_numcols_ls = corpus_parags_df.select_dtypes(include=['number']).columns

for acol in corpus_parags_df_numcols_ls:
  parags_zoom_temp_np = zoom(np.array(corpus_parags_df[acol]), resample_ratio)
  corpus_parags_zoom_df[acol] = pd.Series(parags_zoom_temp_np)

print('\n')
print(f'New expanded corpus_parags_zoom_df.shape = {corpus_parags_zoom_df.shape}')
print(f'           matches corpus_sents_df.shape = {corpus_sents_df.shape}')

**Adjust the Paragraph Sentiment Plot to compare it with the Section Sentiment Plot for this Corpus**

INSTRUCTIONS:

* Adjust [Scale_Vertical_Rolling_Paragraph] until the vertical min/max spans of the two plots are approximately equal

* Adjust [Set_Paragraph_Rolling_Window_Percent] to set horizonal smoothness of Rolling Paragraph (typically 5,10 or 20%)


In [ ]:
col_lnorm_medianiqr

In [ ]:
Scale_Vertical_Rolling_Paragraph = 2.7 #@param {type:"slider", min:0, max:20, step:0.1}
Set_Paragraph_Rolling_Window_Percent = 3 #@param {type:"slider", min:0, max:30, step:1}

# Compare Section Midpoints vs Sentence SMA Sentiment Values

scale_sma_paragraph = Scale_Vertical_Rolling_Paragraph
sentence_count = corpus_sents_df.shape[0]
if Set_Paragraph_Rolling_Window_Percent == 0:
  sma_parag_win = 1
else:
  sma_parag_win = int((Set_Paragraph_Rolling_Window_Percent/100)*sentence_count)


# corpus_parags_zoom_df['vader_lnorm_medianiqr'].rolling(window=sma_parag_win, center=True).mean().apply(lambda x: x*scale_sma_paragraph).plot(label=f'{model_base} Paragraphs', alpha=0.3)
corpus_parags_zoom_df[col_lnorm_medianiqr].rolling(window=sma_parag_win, center=True).mean().apply(lambda x: x*scale_sma_paragraph).plot(label=f'{model_base} Paragraphs', alpha=0.3)

_ = plot_crux_sections(model_names_ls=[col_lnorm_medianiqr], semantic_type='section', label_token_ct=0, title_xpos=0.5, title_ypos=1.0, sec_y_height=8, save2file=False)
plt.title(f'{CORPUS_FULL} \n SMA vs midpoint Paragraph MedianIQR Sentiment with Crux Points via SciPy.peaks')
plt.legend(loc='best');

#### **Zoom into a Section**

INSTRUCTIONS:

* At [Select_Section_No] pick a Section of the Corpus to zoom into

* Adjust [Scale_Vertical_Rolling_Sentences] until the vertical min/max spans of the two plots are approximately equal 

* Adjust [Set_Sentence_Rolling_Window_Percent] to set horizonal smoothness of Rolling Paragraph (typically 5,10 or 20%)

In [ ]:
# Explore a Corpus Section Up-Close

section_count = corpus_sects_df.shape[0]
print(f'There are {section_count} Sections in this corpus,\n  pick one numbered between 0 and {section_count-1}')

In [ ]:
Select_Section_No =  15#@param {type:"integer"}
Scale_Vertical_Rolling_Sentences = 0.2 #@param {type:"slider", min:0, max:20, step:0.1}
Set_Sentence_Rolling_Window_Percent = 3 #@param {type:"slider", min:0, max:30, step:1}

# Make Copies instead of just using References / Only Reference, not copy()
# section_sents_df = pd.DataFrame()
# section_parags_df = pd.DataFrame()

section_sents_df, section_parags_df = get_section_timeseries(Select_Section_No)

# section_sents_df.head()

print(f'section_sents_df.shape: {section_sents_df.shape}')
print(f'section_parags_df.shape: {section_parags_df.shape}')

In [ ]:
model_name

In [ ]:
# Add expanded Paragraph sentiment to corpus_sents_df
# section_sents_parags_df['vader_lnorm_medianiqr_parag'] = expand_parags2sents(parags_df='corpus_parags_df', sents_df='corpus_sents_df')
parags_midpoint_ls = []
col_name_parag = f'{model_name}_parag'
section_sents_df[col_name_parag], parags_midpoint_ls = expand_parags2sents(parags_df='corpus_parags_df', sents_df='corpus_sents_df', model_name=model_name)

# Verify Sentences and Expanded Paragraph lengths match
print(f'\nIn Section #{Select_Section_No}\n')
print(f'            Sentence Count: {section_sents_df.shape[0]}')
print(f"(expanded) Paragraph Count: {str(section_sents_df['parag_no'].unique().shape[0])}") 

##### **Section Histograms**

EDA Unbalanced Paragraph and Sentence Features within selected Section

In [ ]:
# Verify the non-uniform distribution of Paragraph lengths within selected Section (thus necessity for noralizing Paragraph Sentiment by Paragraph length)

section_sents_df['parag_no'].value_counts().hist(bins=30)
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nHistogram of Number of Sentences per Paragraph in Section #{Select_Section_No}')
plt.xlabel('Number of Sentences per Paragraph')
plt.ylabel('Frequency');

In [ ]:
section_parags_df.columns

In [ ]:
section_sents_df[model_name].hist(bins=30)
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nHistogram of Sentence Sentiment in Section #{Select_Section_No}')
plt.xlabel('Sentence Sentiment')
plt.ylabel('Frequency');

In [ ]:
# Histogram of Paragraph Sentiments within selected Section
 
# create a unified Section DataFrame with equal length Sentences and (expanded) Paragraphs Sentiment Series
# section_sents_parags_df = section_sents_df.copy()
# section_sents_parags_df['vader_lnorm_medianiqr_parag_approx'] = parag_sentiment_expanded_ls


section_sents_df[col_name_parag].hist(bins=30)
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nHistogram of Length-Normed Paragraph Sentiment in Section #{Select_Section_No}')
plt.xlabel('Length-Normed Sentiment of Paragraph')
plt.ylabel('Frequency');

**Naive Raw and LOWESS Smoothed Paragraph Sentiment plots within selected Section**

NOTE: Horizonal x-axis narrative time axis not adjusted for variable paragraph lengths - simply used midpoints assuming equal length Paragraphs.

**Raw and SMA Sentence Sentiment Plot within selected Section**

In [ ]:
section_sents_df.info()

In [ ]:
# section_crux_sents_dt

# type(section_crux_sents_dt['vader_lnorm_medianiqr_roll50_frac14_win10'][0][0])

##### **Section SMA**

In [ ]:
sec_y_ht = 0.02

plot_smas(section_view=False, model_name='textblob_lnorm_medianiqr', text_unit='sentence', wins_ls=[5,10,15,20], alpha=0.5, y_height=sec_y_ht, subtitle_str=f'(Model: {model_base.capitalize()})', save2file=False)

In [ ]:
# SMA with Raw Sentiment values over entire Corpus

sec_y_ht = -0.06

plot_smas(section_view=False, model_name=model_base, text_unit='sentence', wins_ls=[5,10,15,20], alpha=0.5, y_height=sec_y_ht, subtitle_str=f'(Model: {model_base.capitalize()})', save2file=False)

In [ ]:
# SMA with Length-Normed MedianIQR Sentiment values over entire Corpus

sec_y_ht = -0.11

plot_smas(section_view=False, model_name=model_name, text_unit='sentence', wins_ls=[5,10,15,20], alpha=0.5, y_height=sec_y_ht, subtitle_str=f'(Model: {model_base.capitalize()})', save2file=False)


In [ ]:
sec_y_ht = 0.65

# At Section boundries draw blue vertical lines 
paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
for i, aparag in enumerate(paragraph_boundries_ls):
  if i%5 == 0:
    # Plot every 5th paragraph
    sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
    plt.text(sent_no, sec_y_ht, f'Paragraph #{aparag}', alpha=0.2, rotation=90)
    plt.axvline(sent_no, color='blue', alpha=0.1)
  # 'BigNews1', xy=(sent_no, 0.5), xytext=(-10, 25), textcoords='offset points',                   rotation=90, va='bottom', ha='center', annotation_clip=True)

  # plt.text(sent_no, -.5, 'goodbye',rotation=90, zorder=0)
      
for win_size in range(50,250,25):
  section_sents_df[model_name].rolling(win_size, center=True).mean().plot(alpha=0.5, label=f'SMA win={win_size}')
plt.title(f'{CORPUS_FULL} (Model: {model_base.capitalize()})\nSMA Length-Normed Sentence Sentiment in Section #{Select_Section_No}')
plt.legend(loc='best');

In [ ]:
# Plot Sentence by Sentence Sentiment within selected Section

sec_y_ht = -0.5

paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
for i, aparag in enumerate(paragraph_boundries_ls):
  if i%5 == 0:
    # Plot every 5th paragraph
    sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
    plt.text(sent_no, sec_y_ht, f'Paragraph #{aparag}', alpha=0.2, rotation=90)
    plt.axvline(sent_no, color='blue', alpha=0.1)

awins_ls = [10]
get_smas(section_sents_df, model_name=model_name, text_unit='sentence', wins_ls=awins_ls, alpha=0.5, scale_factor=1., subtitle_str=f'Section #{Select_Section_No}', mean_adj=0., do_plot=True, save2file=False)
get_smas(section_sents_df, model_name=col_medianiqr, text_unit='sentence', wins_ls=awins_ls, alpha=0.5, scale_factor=1.8, subtitle_str='', mean_adj=0., do_plot=True, save2file=False)
get_smas(section_sents_df, model_name=model_base, text_unit='sentence', wins_ls=awins_ls, alpha=0.5, scale_factor=8., subtitle_str=f'Section #{Select_Section_No}', mean_adj=0., do_plot=True, save2file=False);
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nSMA Length-Normed Sentence Sentiment in Section #{Select_Section_No} (win={awins_ls[0]}%)')
plt.legend(loc='best');

In [ ]:
# Plot Paragraph by Sentence Sentiment within selected Section

get_smas(section_sents_df, model_name=model_name, text_unit='paragraph', wins_ls=[5], alpha=0.5, scale_factor=1., subtitle_str=f'Section #{Select_Section_No}', mean_adj=0., do_plot=True, save2file=False)
get_smas(section_sents_df, model_name=col_medianiqr, text_unit='paragraph', wins_ls=[5], alpha=0.5, scale_factor=1.8, subtitle_str='', mean_adj=0., do_plot=True, save2file=False)
get_smas(section_sents_df, model_name=model_base, text_unit='paragraph', wins_ls=[5], alpha=0.5, scale_factor=8., subtitle_str=f'Section #{Select_Section_No}', mean_adj=0., do_plot=True, save2file=False)
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nSMA Length-Normed Sentence Sentiment in Section #{Select_Section_No} (win={awins_ls[0]}%)')
plt.legend(loc='best');

In [ ]:
# Plot Paragraph by Paragraph Sentiment within selected Section

sec_y_ht = -15

paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
for i, aparag_no in enumerate(paragraph_boundries_ls):
  if i%5 == 0:
    # Plot every 5th paragraph
    sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
    plt.text(aparag_no, sec_y_ht, f'Paragraph #{aparag_no}', alpha=0.2, rotation=90)
    plt.axvline(aparag_no, color='blue', alpha=0.1)

awins_ls = [5]
get_smas(section_parags_df, model_name=model_name, text_unit='paragraph', wins_ls=awins_ls, alpha=0.5, scale_factor=1., subtitle_str=f'Section #{Select_Section_No}', mean_adj=0., do_plot=True, save2file=False)
get_smas(section_parags_df, model_name=col_medianiqr, text_unit='paragraph', wins_ls=awins_ls, alpha=0.5, scale_factor=8., subtitle_str='', mean_adj=0., do_plot=True, save2file=False)
get_smas(section_parags_df, model_name=model_base, text_unit='paragraph', wins_ls=awins_ls, alpha=0.5, scale_factor=10., subtitle_str=f'Section #{Select_Section_No}', mean_adj=0., do_plot=True, save2file=False)
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nSMA Length-Normed Paragraph Sentiment in Section #{Select_Section_No} (win={awins_ls[0]}%)')
plt.legend(loc='best');

##### **Raw Sentiments**

In [ ]:
# Plot Raw vs MedianIQR Sentence Sentiment within selected Section

sec_y_ht = -4.0

paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
for i, aparag in enumerate(paragraph_boundries_ls):
  if i%5 == 0:
    # Plot every 5th paragraph
    sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
    plt.text(sent_no, sec_y_ht, f'Paragraph #{aparag}', alpha=0.2, rotation=90)
    plt.axvline(sent_no, color='blue', alpha=0.1)

plt.plot(model_base, data=section_sents_df, alpha=0.3, label=f'Raw Sentence Sentiment ({model_base})')
plt.plot(col_medianiqr, data=section_sents_df, alpha=0.5, label=f'MedianIQR Sentence Sentiment ({model_base})')
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nRaw vs MedianIQR Sentence Sentiment in Section #{Select_Section_No}')
plt.legend(loc='best');

In [ ]:
# MedianIQR vs Length-Normed MedianIQR Sentence Sentiment within selected Section

sec_y_ht = -17

paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
for i, aparag in enumerate(paragraph_boundries_ls):
  if i%5 == 0:
    # Plot every 5th paragraph
    sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
    plt.text(sent_no, sec_y_ht, f'Paragraph #{aparag}', alpha=0.2, rotation=90)
    plt.axvline(sent_no, color='blue', alpha=0.1)

plt.plot(model_name, data=section_sents_df, alpha=0.3, label=f'Length-Normed MedianIQR Sentence Sentiment ({model_base})')
plt.plot(col_medianiqr, data=section_sents_df, alpha=0.3, label=f'MedianIQR Sentence Sentiment ({model_base})')
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nLength-Normed vs non-Normed Sentence Sentiment in Section #{Select_Section_No}')
plt.legend(loc='best');

In [ ]:
# Plot Paragraph by Paragraph Sentiment within selected Section

sec_y_ht = -15

paragraph_boundries_ls = list(section_sents_df['parag_no'].unique())
for i, aparag_no in enumerate(paragraph_boundries_ls):
  if i%5 == 0:
    # Plot every 5th paragraph
    sent_no = section_sents_df[section_sents_df['parag_no'] == aparag]['sent_no'].min()
    plt.text(aparag_no, sec_y_ht, f'Paragraph #{aparag_no}', alpha=0.2, rotation=90)
    plt.axvline(aparag_no, color='blue', alpha=0.1)

plt.plot(model_name, data=section_parags_df, alpha=0.3, label=f'Length-Normed MedianIQR Paragraph Sentiment ({model_base})')
plt.plot(col_medianiqr, data=section_parags_df, alpha=0.3, label=f'MedianIQR Paragraph Sentiment ({model_base})')
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nLength-Normed vs non-Normed Paragraph Sentiment in Section #{Select_Section_No}')
plt.legend(loc='best');

##### **LOWESS Smoothed**

In [ ]:
model_name

In [ ]:
# MedianIQR vs Length-Normed MedianIQR Sentence Sentiment within selected Section

win_lowess_no = 10
sec_y_ht = -17

section_crux_ls = get_crux_points(ts_df=section_sents_df, col_series=model_name, text_type='sentence', win_lowess=win_lowess_no, y_height=sec_y_ht, do_plot=True, save2file=False);

In [ ]:
# TODO: Only printing sentiment for first crux point

section_sents_df.iloc[62]['sent_raw']
print('\n')
section_sents_df.iloc[62]

In [ ]:
Get_Peak_Cruxes = True #@param {type:"boolean"}
Get_n_Cruxes = 5 #@param {type:"slider", min:1, max:10, step:1}
No_Paragraphs_on_Each_Side = 3 #@param {type:"slider", min:0, max:5, step:1}


crux_sortsents_report(section_crux_ls, top_n=Get_n_Cruxes, get_peaks=Get_Peak_Cruxes, n_sideparags=No_Paragraphs_on_Each_Side)

In [ ]:
# LOWESS Smoothed Sentences within chosen Selection No

my_afrac = 1./12   # 1./12 ~ 0.08

temp_df = get_lowess(section_sents_df, [model_name], plot_subtitle='LOWESS Smoothed MedianIRQ Sentence Sentiment', alabel=f'LOWESS (afrac={my_afrac})', 
                afrac=my_afrac, ait=7, alpha=0.8, do_plot=True, save2file=False)
temp_df.columns
col_lowess = f'{model_name}_{my_afrac:2.3f}lowess'
col_lowess_clean = col_lowess.replace('_0.','_')
section_sents_df[col_lowess_clean] = temp_df['median']

plt.title(f'{CORPUS_FULL} (Model: {model_base})\nRaw Sentence Sentiments with selected Section #{Select_Section_No} (LOWESS frac={my_afrac:.2f})')
plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value')
plt.legend('',frameon=False);

In [ ]:
# LOWESS Smoothed Paragraphs within chosen Selection No

my_afrac = 1./8 # 1./8 ~ 0.125

temp_df = get_lowess(section_parags_df, [model_name], plot_subtitle='LOWESS Smoothed Mean Rolling Sentence Sentiment', alabel=f'LOWESS Smoothed (afrac={my_afrac})', 
                afrac=my_afrac, ait=7, alpha=0.8, do_plot=True, save2file=False)

section_parags_df[f'{model_name}_{my_afrac:.2f}_lowess'] = temp_df['median']

plt.title(f'{CORPUS_FULL} (Model: {model_base})\nRaw Sentence Sentiments with selected Section #{Select_Section_No} (LOWESS frac={my_afrac:.2f})')
plt.xlabel(f'Paragraph No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value')
plt.legend('',frameon=False);

In [ ]:
# TODO: Convert to input widgets
# grid_fracs = [1./3, 1./4, 1./6, 1./10, 1./14, 1./16]
# grid_fracs = [1./10, 1./14, 1./16]
# grid_fracs = [0.14, 0.16, 0.18, 0.2, 0.25, 0.3, 0.35, 0.4]
# grid_fracs = [0.14, 0.16, 0.18, 0.2, 0.25, 0.3, 0.35, 0.4]
grid_fracs = [0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.3]
win_lowess=9


# section_sents_df['vader_lnorm_medianiqr'].plot(label=f'Raw Sentence Sentiments with selected Section #{Select_Section_No}')
# plt.plot('vader_lnorm_medianiqr', data=section_sents_df)
plt.title(f'LOWESS Smoothed Sentence Sentiments with selected Section #{Select_Section_No}')
plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value');

section_crux_sents_dt = {}

for afrac in grid_fracs:
  # print(f'type(my_afrac) = {type(my_afrac)}, value = {my_afrac}')
  #   _ = get_lowess(section_sents_df, ['vader_lnorm_medianiqr'], plot_subtitle='Naive Raw + MedianIQR Midpoints', alabel=f'LOWESS Smoothed (afrac={my_afrac})', 
  #                afrac=my_afrac, ait=7, alpha=my_afrac, do_plot=True, save2file=False);

  # afrac = 1./7
  # model_name = 'vader_lnorm_medianiqr' # model_name
  sm_x, sm_y = sm_lowess(endog=section_sents_df[model_name].values, exog=section_sents_df.index.values, frac=afrac, it=3, return_sorted = True).T
  # _ = get_lowess(ts_df='section_sents_df', models_ls=[col_roll_str], text_unit='sentence', plot_subtitle='', alabel='', afrac=1./10, ait=5, alpha=0.5, do_plot=True, save2file=False)
  section_crux_ls = list(get_crux_points(section_sents_df, col_series=model_name, win_lowess=win_lowess, do_plot=False))
  col_lowess_str = f'{model_name}_frac{int((afrac-int(afrac))*100)}_win{win_lowess}'
  # print(f"col_lowess_str: {col_lowess_str}")
  section_crux_sents_dt[col_lowess_str] = section_crux_ls # list(zip(sm_x, sm_y))
  # x, y = zip(*data)

  # Set vertical y-axis magnification
  y_mag = 30
  plt.plot(sm_x, y_mag*sm_y)
  plt.title(f'{CORPUS_FULL} (Model: {model_base})\nDifferent LOWESS Smoothed SMA Sentence Sentiments and Crux Points within selected Section #{Select_Section_No}')


for key,value in section_crux_sents_dt.items():
  model_name = key
  crux_points_ls = value
  plt.scatter(*zip(*crux_points_ls))

plt.title(f'{CORPUS_FULL} (Model: {model_base})\nDifferent SMA Windows of Raw Sentence Sentiments and Crux Points within selected Section #{Select_Section_No}')
plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value')
plt.legend(loc='best');


# Plot the mean of all SMA MedianIQR Sentiment Time Series
# col_meanroll = f'{model_name}_rollmean'
# section_sents_df[col_meanroll] = section_sents_df[col_rolls_ls].mean(axis=1)
# section_sents_df[col_meanroll].plot(label='mean', color='black', linewidth=3)

# Plot corresponding Crux Points
# model_name = 'vader_lnorm_medianiqr'
section_crux_ls = get_crux_points(section_sents_df, col_series=model_name, win_lowess=10) # 'vader_lnorm_medianiqr_0.07_lowess')
section_sents_df.shape[0]
print('\n');

In [ ]:
# TODO: Convert to input widgets
# grid_fracs = [1./3, 1./4, 1./6, 1./10]
# grid_fracs = [0.14, 0.16, 0.18, 0.2, 0.25, 0.3, 0.35, 0.4]
grid_fracs = [0.1, 0.12, 0.14, 0.16, 0.18, 0.2]

# section_sents_df['vader_lnorm_medianiqr'].plot(label=f'Raw Sentence Sentiments with selected Section #{Select_Section_No}')
# plt.plot('vader_lnorm_medianiqr', data=section_sents_df)
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nLOWESS Smoothed Sentence Sentiments with selected Section #{Select_Section_No}')
plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value');

for my_afrac in grid_fracs:
  # print(f'type(my_afrac) = {type(my_afrac)}, value = {my_afrac}')
  _ = get_lowess(section_sents_df, [model_name], plot_subtitle='Naive Raw + MedianIQR Midpoints', alabel=f'LOWESS (afrac={my_afrac})', 
                 afrac=my_afrac, ait=7, alpha=my_afrac, do_plot=True, save2file=False);

  # corpus_cruxes_dt['vader_lnorm_medianiqr'] = plot_crux_sections(model_names_ls=['vader_lnorm_medianiqr'], semantic_type='section', label_token_ct=5, title_xpos=0.8, title_ypos=1.05, save2file=False)
plt.title(f'{CORPUS_FULL}\n LOWESS Smoothed Sentence Sentiments with selected Section #{Select_Section_No}')
plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value');


In [ ]:
# TODO: Convert to input widgets
# grid_fracs = [1./3, 1./4, 1./6, 1./10]
# grid_fracs = [0.14, 0.16, 0.18, 0.2, 0.25, 0.3, 0.35, 0.4]
grid_fracs = [0.12, 0.14, 0.16, 0.18, 0.2]

# section_sents_df[model_name].plot(label=f'Raw Sentence Sentiments with selected Section #{Select_Section_No}')
# plt.plot(model_name, data=section_sents_df)
plt.title(f'{CORPUS_FULL} (Model: {model_base})\nLOWESS Smoothed Paragraph Sentiments with selected Section #{Select_Section_No}')
plt.xlabel(f'Paragraph No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value');

for my_afrac in grid_fracs:
  # print(f'type(my_afrac) = {type(my_afrac)}, value = {my_afrac}')
  _ = get_lowess(section_parags_df, [model_name], plot_subtitle='MedianIQR Midpoints', alabel=f'LOWESS (afrac={my_afrac})', 
                 afrac=my_afrac, ait=7, alpha=my_afrac, do_plot=True, save2file=False);

  # corpus_cruxes_dt[model_name] = plot_crux_sections(model_names_ls=[model_name], semantic_type='section', label_token_ct=5, title_xpos=0.8, title_ypos=1.05, save2file=False)
plt.title(f'{CORPUS_FULL}\n LOWESS Smoothed Paragraph Sentiments with selected Section #{Select_Section_No}')
plt.xlabel(f'Paragraph No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value');


In [ ]:
section_sents_df.columns

In [ ]:
# SMA Sentences

# grid_afracs = [0.14, 0.16, 0.18, 0.2, 0.25, 0.3, 0.35, 0.4]
grid_fracs = [0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.3]
# grid_fracs = [1./6, 1./7, 1./8, 1./9, 1./10, 1./15, 1./20]
scale_roll = 1.
win_lowess_per = 30
win_lowess = int(win_lowess/100 * section_sents_df.shape[0])

col_meanroll_lowess_ls = []

col_meanroll = f'{model_name}_mean_roll050'
for afrac in grid_fracs:
  lowess_smooth_df = get_lowess(section_sents_df, [col_meanroll], plot_subtitle='SMA Mean of MedianIQR ', alabel=f'LOWESS afrac={afrac:.3f}', 
                afrac=afrac, ait=7, alpha=0.3, do_plot=True, save2file=False)
  # print(f'type: {lowess_smooth_df.columns}')

# col_lowess_str = f'{col_mean_roll}_lowess_frac{10*win_lowess}'
col_meanroll_lowess_str = f'{col_meanroll}_frac{int((afrac-int(afrac))*100)}_win{win_lowess}'
col_meanroll_lowess_ls.append(col_meanroll_lowess_str)
section_sents_df[col_meanroll_lowess_str] = section_sents_df[model_name].apply(lambda x: x*scale_roll).rolling(win_lowess, center=True).mean()
section_sents_df[col_meanroll_lowess_str].plot(alpha=0.7)
get_lowess(section_sents_df, [col_meanroll_lowess_str], plot_subtitle='SMA Mean of MedianIQR ', alabel=f'LOWESS (afrac={my_afrac:.3f})', 
                afrac=afrac, ait=7, alpha=1.0, do_plot=True, save2file=False);


plt.title(f'{CORPUS_FULL} (Model: {model_base})\nLOWESS Smoothed Sentence Sentiments with selected Section #{Select_Section_No}')
plt.xlabel(f'Sentence No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value')
plt.legend(loc='best');

**Raw Paragraph Sentiment Plot within selected Section**

In [ ]:
# TODO: Convert to input widgets
win_sents_ls = [5,10,15,20,25]
scale_roll = 6

plt.plot(model_name, data=section_parags_df, alpha=0.3, label=f'Raw Paragraph Sentiment within selected Segment #{Select_Section_No}')

plt.title(f'{CORPUS_FULL} (Model: {model_base})\nRaw Paragraph Sentiments with selected Section #{Select_Section_No}')
plt.xlabel(f'Paragraph No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value');

**Length-Normed Paragraph Sentiment Plot within selected Section**

In [ ]:
corpus_parags_df.columns

In [ ]:
# Plot and Compare Naive Raw and LOWESS Smoothed Paragraph Sentiment Time Series within selected Section

section_parag_lowess_df = pd.DataFrame()
section_parag_lowess_df['parag_no'] = section_parags_df['parag_no'].copy()

parags_midpoint_sentiment_ls = []
for parags_midpoint_idx in parags_midpoint_ls:
  parags_midpoint_sentiment_ls.append(float(corpus_parags_df[corpus_parags_df['parag_no'] == parags_midpoint_idx][model_name]))

col_midapprox = f'{model_name}_midapprox'
section_parag_lowess_df[col_midapprox] = parags_midpoint_sentiment_ls


section_parag_lowess_df[col_midapprox].plot(label='Raw Midpoints')
plt.xlabel(f'Niave Paragraph No within selected Section #{Select_Section_No}')
plt.ylabel(f'Sentiment Value')

_ = get_lowess(section_parag_lowess_df, [col_midapprox], plot_subtitle=f'{model_base.capitalize()} Naive Raw + MedianIQR Midpoints', alabel='LOWESS Midpoints', afrac=1./4, ait=7, do_plot=True, save2file=False);

# section_lowess_parags_df = get_lowess(section_sents_parags_df, ['vader_lnorm_medianiqr'], plot_subtitle='Approximate Paragraph MedianIQR', afrac=1./4, ait=7, do_plot=True, save2file=False);

**Length-Noramlized Raw and LOWESS Smoothed Paragraph Sentiment plots within selected Section**

NOTE: Horizonal x-axis narrative time axis adjusted for variable paragraph lengths - used midpoints of unequal length Paragraphs to more accurately visualize Sentiment Arc and precisely localize Crux Points.

In [ ]:
# Verify details on currently selected Section
print(f'Details on Section #{Select_Section_No}')
print('------------------------')
print(f' Paragraph Count: {section_parags_df.shape[0]}')
print(f' Sentence Count:  {section_sents_df.shape[0]}')

In [ ]:
# %load_ext google.colab.data_table

In [ ]:
# Plot Raw and Rolling Sentence Sentiments within selected Section

win_per = 5  # Rolling Window size in percentage of total Corpus length

section_sents_parags_df.plot(x='sent_no', y='vader_lnorm_medianiqr')

plt.title(f'Raw and Rolling Sentence Sentiments within selected Section #{Select_Section_No}')
plt.xlabel(f'Sentence No within selected Section #{Select_Section_No} (Length-Normalized in terms of Paragraphs)')
plt.ylabel(f'Sentiment Value')

section_sents_parags_df['vader_lnorm_medianiqr'].rolling(int((win_per/100)*section_sents_parags_df.shape[0])).mean().plot(label="Approx Paragraph VADER SMA (win=5%)");

# section_sents_parags_df.plot(x='sent_no', y='vader_lnorm_medianiqr', label='Sentence VADER MedianIQR')
# section_sents_parags_df['vader_lnorm_medianiqr'].rolling(int(0.05*section_sents_parags_df.shape[0])).mean().plot(label="Sentence VADER SMA (win=5%)");

**Get Crux Points within selected Section**

INSTRUCTIONS:

* Select [Get_Peaks] to search for Peaks (unselect to search for Valley)

* Pick [Crux_Rank] (1-5) to get the 1st to 5th biggest Peak or Valley Crux Paragraph

* Pick [Context_Paragraphs_Each_Side] (0-5) to get n paragraphs before and n paragraphs after the selected Crux Paragraph

In [ ]:
"""

# ARCHIVED

def get_sentnocontext(ts_df, model_name='vader', get_peaks=True, crux_rank=1, n_sideparags=1):
  # get_cruxparags_section()
  '''
  Given a Section DataFrame with model_name sentiment column and crux peak/valley, rank and side paragraphs context
  Return a list with the appropriate Crux Paragraph within this Section, and context
  '''

  '''
  Given a sentence number in the Corpus
  Return the containing paragraph and n-paragraphs on either side
  (e.g. if n=2, return 2+1+2=5 paragraphs)
  '''

  crux_parags_context_ls = []

  if get_peaks == True:
    sort_asc_flag=False
  else:
    sort_asc_flag=True

  crux_parag_no = ts_df.sort_values(by=[model_name], ascending=sort_asc_flag).iloc[crux_rank-1]['parag_no']

  print(f'crux_parag_no: {crux_parag_no}')

  if n_sideparags == 0:
    crux_parags_context_ls = list(corpus_parags_df[corpus_parags_df['parag_no'] == crux_parag_no]['parag_raw'])

  else:
    parag_start = crux_parag_no - n_sideparags
    parag_end = crux_parag_no + n_sideparags + 1
    crux_parags_context_ls = list(corpus_parags_df.iloc[parag_start:parag_end]['parag_raw'])

  return crux_parags_context_ls

# Test
parags_context_ls = get_sentnocontext(ts_df=section_parags_df, model_name='vader_lnorm_medianiqr', get_peaks=True, crux_rank=1, n_sideparags=1)
parags_context_ls
""";

In [ ]:
# def get_crux_parags_report(ts_df, model_name='vader', get_peaks=True, crux_rank=1, the_n_sideparags=No_Paragraphs_on_Each_Side, the_sent_highlight=Highlight_Crux_Sentence):

# def get_sentnocontext_report(ts_df, model_name='vader', get_peaks=True, crux_rank=1, the_n_sideparags=No_Paragraphs_on_Each_Side, the_sent_highlight=Highlight_Crux_Sentence):
'''
Given a DataFrame with model_name sentiment column and crux peak/valley, rank and side paragraphs context
Return a list with the appropriate Crux Paragraph, and context
'''
"""

def get_sentnocontext_report(the_sent_no=7, the_n_sideparags=1, the_sent_highlight=True):
  '''
  Wrapper function around  get_sentnocontext()
  Prints a nicely formatted context report
  '''

  context_noparags = the_n_sideparags*2+1

  print('-------------------------------------------------------------')
  print(f'The {context_noparags} Paragraph(s) Context around the Sentence #{Crux_Sentence_No} Crux Point:')
  print('-------------------------------------------------------------')
  print(f"\nCrux Sentence Raw Text: -------------------------------\n\n    {corpus_sents_df[corpus_sents_df['sent_no'] == the_sent_no]['sent_raw']}") # iloc[the_sent_no]['sent_raw']}")

  print(f"\n{context_noparags} Paragraph(s) Context: ------------------------------")
  # context_parags_ls = get_sentnocontext(sent_no=the_sent_no, n_sideparags=the_n_sideparags, sent_highlight=the_sent_highlight)
  context_parags_ls = get_sentnocontext(sent_no=the_sent_no, n_sideparags=the_n_sideparags, sent_highlight=the_sent_highlight)
  context_len = len(context_parags_ls)
  context_mid = context_len//2
  for i, aparag in enumerate(context_parags_ls):
    if i==context_mid:
      # print(f'\n>>> Paragraph #{i}: <<< Crux Point Sentence CAPITALIZED within this Paragraph\n\n    {aparag}')
      print(f'\n<*> {aparag}')
    else:
      # print(f'\n    Paragraph #{i}:\n\n    {aparag}')
      print(f'\n    {aparag}')

  return

# Test
# get_sentnocontext_report(sent_no=1051, n_sideparags=1, sent_highlight=True)
""";

In [ ]:
# Get_Peaks = True #@param {type:"boolean"}
Crux_Rank = 2 #@param {type:"slider", min:1, max:5, step:1}
No_Paragraphs_on_Each_Side = 2 #@param {type:"slider", min:0, max:10, step:1}
Highlight_Crux_Sentence = True #@param {type:"boolean"}

# try:
  
# get_sentnocontext_report(ts_df=section_sents_df, model_name=model_name, get_peaks=Get_Peaks, crux_rank=Crux_Rank, the_n_sideparags=No_Paragraphs_on_Each_Side, the_sent_highlight=Highlight_Crux_Sentence)
get_sentnocontext_report(corpus_sents_df, the_sent_no=Crux_Rank, the_n_sideparags=No_Paragraphs_on_Each_Side, the_sent_highlight=Highlight_Crux_Sentence)

# except:
#   print('ERROR')

In [ ]:
section_parags_df.shape

In [ ]:
print(section_parags_df.sort_values(by=['vader_lnorm_medianiqr'], ascending=False).iloc[0]) # ['parag_no'])

In [ ]:
parags_context_ls = get_cruxparag_context(ts_df=section_sents_parags_df, model_name='vader_lnorm_medianiqr', get_peaks=True, crux_rank=1, n_sideparags=2, sent_highlight=True)
parags_context_ls

In [ ]:
section_sents_parags_df.info()

## **Compare Models**

In [ ]:
def drop_dupcols(df):
  '''
  Given a DataFrame
  Drop repeatitive columns
  '''

  col_drop_ls = []

  col_ls = list(df.columns)
  print(f'BEFORE: Columns #{len(df.columns)}')

  for i, acol in enumerate(col_ls):
    acol_word_ls = acol.split('_')
    # print(f'acol_word_ls: {acol_word_ls}')
    if (len(acol_word_ls)) == len(set(acol_word_ls)):
      continue
    else:
      col_drop_ls.append(acol)

  df.drop(columns=col_drop_ls, inplace=True, axis=1)

  print(f'AFTER: Columns #{len(df.columns)}')

  return col_drop_ls

dropped_cols_ls = drop_dupcols(corpus_parags_df)
print(f'dropped: {dropped_cols_ls}')

In [ ]:
corpus_parags_df.columns

In [ ]:
# List of Tuples (Model, Scaling Factor) to plot together with same size for comparison

models_sma_ls = [('vader_lnorm_medianiqr',1),
                 ('textblob_lnorm_medianiqr',20),
                 ('afinn_lnorm_medianiqr',4),
                 ('sentimentr_lnorm_medianiqr',1),
                 ('syuzhet_lnorm_medianiqr',1),
                 ('bing_lnorm_medianiqr',0.1),
                 ('sentiword_lnorm_medianiqr',10),
                 ('senticnet_lnorm_medianiqr',.5),
                 ('nrc_lnorm_medianiqr',0.2),
                 ('pattern_lnorm_medianiqr',20),
                 ('stanza_lnorm_medianiqr',0.5),
                 ('hfbert_lnorm_medianiqr',5),
                 ('nlptown_lnorm_medianiqr',5),
                 ('robertalg15_lnorm_medianiqr',5)]

In [ ]:
# List of Tuples (Model, Scaling Factor) to plot together with same size for comparison

models_sma_ls = [('vader',1),
                 ('textblob',20),
                 ('afinn',4),
                 ('sentimentr',1),
                 ('syuzhet',1),
                 ('bing',0.1),
                 ('sentiword',10),
                 ('senticnet',.5),
                 ('nrc',0.2),
                 ('pattern',20),
                 ('stanza',0.5),
                 ('hfbert',5),
                 ('nlptown',5),
                 ('robertalg15',5)]

In [ ]:
corpus_sents_df[['stanza_lnorm_medianiqr','stanza']].rolling(500,center=True).mean().plot()

In [ ]:
def plot_autoscaled_ts(ts_df=corpus_sents_df, ts_ls=['vader_lnorm_medianiqr', 'textblob_lnorm_medianiqr', 
                                                     'syuzhet_lnorm_medianiqr', 'sentimentr_lnorm_medianiqr',
                                                     'bing_lnorm_medianiqr', 'afinn_lnorm_medianiqr',
                                                     'pattern_lnorm_medianiqr', 'stanza_lnorm_medianiqr']):
  '''
  Given a DataFrame and list of Columns/Time Series
  Automatically scale all to the same range and plot together
  '''

  ts_spans_ls = []

  current_min = ts_df[ts_ls[0]].min()
  current_max = ts_df[ts_ls[0]].max()
  ts_spans_ls.append(float(current_max - current_min))

  for ats in ts_ls[1:]:
    current_min = ts_df[ats].min()
    current_max = ts_df[ats].max()
    ts_spans_ls.append(float(current_max - current_min))

  # find index of maximum span
  max_index = ts_spans_ls.index(max(ts_spans_ls))
  max_span_value = ts_spans_ls[max_index]
  max_span_model = ts_ls[max_index]
  print(f'max span is: {max_span_value} from {max_span_model}')

  for i, ats in enumerate(ts_ls):
    y_scaling_factor = max_span_value/ts_spans_ls[i]
    print(f'ats={ats} with scaling={y_scaling_factor}')
    ts_y_scaled_ser = ts_df[ats].apply(lambda x: x*y_scaling_factor)
    # plt.plot()
    plot_df = pd.DataFrame()
    plot_df['x_value'] = ts_df.index
    plot_df['y_scaled'] = ts_y_scaled_ser
    plot_df['y_scaled_roll050'] = plot_df['y_scaled'].rolling(350, center=True).mean()
    plot_df['y_scaled_roll050'].plot(label=f'{ats}')
    plt.legend(loc='best')
    # sns.lineplot(data=plot_df, x='x_value', y='y_scaled', alpha=0.5, label=f'{ats}')

  return ts_spans_ls, ts_ls

# Test
ts_spans_ls, ts_ls = plot_autoscaled_ts()
zip_ls = zip(ts_spans_ls, ts_ls)
for aspan, amodel in zip_ls:
  print(f'model: {amodel} with span: {aspan}')

In [ ]:
ts_df=corpus_sents_df, ts_ls=['vader_lnorm_medianiqr', 'textblob_lnorm_medianiqr', 
                                                     'syuzhet_lnorm_medianiqr', 'sentimentr_lnorm_medianiqr',
                                                     'bing_lnorm_medianiqr', 'afinn_lnorm_medianiqr',
                                                     'pattern_lnorm_medianiqr', 'stanza_lnorm_medianiqr']):

In [ ]:
# Test
ts_spans_ls, ts_ls = plot_autoscaled_ts(ts_df=corpus_parags_df)
zip_ls = zip(ts_spans_ls, ts_ls)
for aspan, amodel in zip_ls:
  print(f'model: {amodel} with span: {aspan}')

In [ ]:
corpus_sents_df['pattern_lnorm_medianiqr'].hist(bins=100) # rolling(100, center=True).mean().plot()

In [ ]:
models_sma_ls = [('vader_lnorm_medianiqr',1),
                 ('textblob_lnorm_medianiqr',20),
                 ('sentimentr_lnorm_medianiqr',1),
                 ('syuzhet_lnorm_medianiqr',1),
                 ('stanza_lnorm_medianiqr',0.2)]

win_per = 5  # 5=5% of full corpus length
win_roll = int(corpus_sents_df.shape[0]* win_per/100)

for amodel, amag in models_sma_ls:
  corpus_parags_df[amodel].rolling(win_roll, center=True).mean().apply(lambda x: amag*x).plot(linewidth=4, label=amodel)
plt.legend(loc='best')

# **Save Raw Sentiment Time Series**

## **Sentence and Paragraph DataFrames (incl raw/clean text)**

In [ ]:
# Save Preprocessed Corpus Sentences DataFrame

author_str = ''.join(CORPUS_AUTHOR.split()).lower()
title_str = ''.join(CORPUS_TITLE.split()).lower()
datetime_now = datetime.utcnow().strftime("%Y%m%d_%H%M")


# Save the original Raw and Cleaned Corpus text at both Sentence and Paragraph Levels

corpus_text_sentences_raw_filename = f'corpus_text_sentences_raw_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus text raw sentences to file: {corpus_text_sentences_raw_filename}')
corpus_sents_df['sent_raw'].to_csv(corpus_text_sentences_raw_filename)

corpus_text_sentences_clean_filename = f'corpus_text_sentences_clean_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus text clean sentences to file: {corpus_text_sentences_clean_filename}')
corpus_sents_df['sent_clean'].to_csv(corpus_text_sentences_clean_filename)


corpus_text_paragraphs_raw_filename = f'corpus_text_paragraphs_raw_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus text raw paragraphs to file: {corpus_text_paragraphs_raw_filename}')
corpus_parags_df['parag_raw'].to_csv(corpus_text_paragraphs_raw_filename)

corpus_text_paragraphs_clean_filename = f'corpus_text_sentences_clean_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus text clean sentences to file: {corpus_text_paragraphs_clean_filename}')
corpus_parags_df['parag_clean'].to_csv(corpus_text_paragraphs_clean_filename)



# Save the Sentiment Values at both the Sentence and Paragraph Levels

corpus_sents_filename = f'corpus_sentences_lexrules_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus Sentences to file: {corpus_sents_filename}')
corpus_sents_df.to_csv(corpus_sents_filename)

corpus_parags_filename = f'corpus_paragraphs_lexrules_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus Paragraphs to file: {corpus_parags_filename}')
corpus_parags_df.to_csv(corpus_parags_filename)

Saving Corpus text raw sentences to file: corpus_text_sentences_raw_ianmcewan_machineslikeme.csv
Saving Corpus text clean sentences to file: corpus_text_sentences_clean_ianmcewan_machineslikeme.csv
Saving Corpus text raw paragraphs to file: corpus_text_paragraphs_raw_ianmcewan_machineslikeme.csv
Saving Corpus text clean sentences to file: corpus_text_sentences_clean_ianmcewan_machineslikeme.csv
Saving Corpus Sentences to file: corpus_sentences_lexrules_ianmcewan_machineslikeme.csv
Saving Corpus Paragraphs to file: corpus_paragraphs_lexrules_ianmcewan_machineslikeme.csv


In [ ]:
# Verify exported Sentence Sentiments file

!head -n 5 $corpus_sents_filename

,sent_no,parag_no,sent_raw,char_len,token_len,sent_clean,sentimentr,sentimentr_lnorm_meanstd,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,syuzhet,syuzhet_lnorm_meanstd,syuzhet_medianiqr,syuzhet_lnorm_medianiqr,bing,bing_lnorm_meanstd,bing_medianiqr,bing_lnorm_medianiqr,sentiword,sentiword_lnorm_meanstd,sentiword_medianiqr,sentiword_lnorm_medianiqr,senticnet,senticnet_lnorm_meanstd,senticnet_medianiqr,senticnet_lnorm_medianiqr,nrc,nrc_lnorm_meanstd,nrc_medianiqr,nrc_lnorm_medianiqr,afinn,afinn_lnorm_meanstd,afinn_medianiqr,afinn_lnorm_medianiqr,scores,vader,vader_lnorm_meanstd,vader_medianiqr,vader_lnorm_medianiqr,textblob,textblob_lnorm_meanstd,textblob_medianiqr,textblob_lnorm_medianiqr,pattern,pattern_lnorm_meanstd,pattern_medianiqr,pattern_lnorm_medianiqr,stanza,stanza_lnorm_meanstd,stanza_medianiqr,stanza_lnorm_medianiqr
0,0,0,"But remember, please, the Law by which we live, We are not built to comprehend a lie ...",88,18,but remember please the law by which we live we are not b

## **Section, Chapter Crux DataFrames (summary stats/sentiments only)**

In [ ]:
corpus_chaps_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 63 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   chap_no                     10 non-null     int64  
 1   chap_raw                    10 non-null     object 
 2   sent_no_start               10 non-null     int64  
 3   sent_no_mid                 10 non-null     int64  
 4   char_len                    10 non-null     int64  
 5   token_len                   10 non-null     int64  
 6   chap_clean                  10 non-null     object 
 7   sentimentr                  10 non-null     float64
 8   sentimentr_meanstd          10 non-null     float64
 9   sentimentr_medianiqr        10 non-null     float64
 10  sentimentr_lnorm_meanstd    10 non-null     float64
 11  sentimentr_lnorm_medianiqr  10 non-null     float64
 12  syuzhet                     10 non-null     float64
 13  syuzhet_meanstd             10 non-nul

In [ ]:
# Create a Chapter Summary DataFrame extracting only key information (no text)

corpus_chaps_summary_df = corpus_chaps_df[['chap_no','sent_no_start','sent_no_mid','char_len','token_len',
                 'sentimentr','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'syuzhet','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'bing','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'sentiword','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'senticnet','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'nrc','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'afinn','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'vader','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'textblob','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'pattern','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'stanza','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 ]]

In [ ]:
corpus_sects_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 63 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sect_no                     30 non-null     int64  
 1   sect_raw                    30 non-null     object 
 2   sent_no_start               30 non-null     int64  
 3   sent_no_mid                 30 non-null     int64  
 4   char_len                    30 non-null     int64  
 5   token_len                   30 non-null     int64  
 6   sect_clean                  30 non-null     object 
 7   sentimentr                  30 non-null     float64
 8   sentimentr_meanstd          30 non-null     float64
 9   sentimentr_medianiqr        30 non-null     float64
 10  sentimentr_lnorm_meanstd    30 non-null     float64
 11  sentimentr_lnorm_medianiqr  30 non-null     float64
 12  syuzhet                     30 non-null     float64
 13  syuzhet_meanstd             30 non-nu

In [ ]:
# Create a Section Summary DataFrame extracting only key information (no text)

corpus_sects_summary_df = corpus_sects_df[['sect_no','sent_no_start','sent_no_mid','char_len','token_len',
                 'sentimentr','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'syuzhet','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'bing','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'sentiword','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'senticnet','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'nrc','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'afinn','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'vader','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'textblob','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'pattern','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 'stanza','sentimentr_medianiqr','sentimentr_lnorm_medianiqr',
                 ]]

corpus_sects_summary_df

,sect_no,sent_no_start,sent_no_mid,char_len,token_len,sentimentr,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,syuzhet,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,bing,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,sentiword,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,senticnet,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,nrc,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,afinn,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,vader,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,textblob,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,pattern,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,stanza,sentimentr_medianiqr,sentimentr_lnorm_medianiqr
0,0,3,117,18434,3230,48.70,1.071705,0.652573,53.40,1.071705,0.652573,0.000000,1.071705,0.652573,2.667242,1.071705,0.652573,141.684,1.071705,0.652573,436.710452,1.071705,0.652573,8.791435,1.071705,0.652573,0.9998,1.071705,0.652573,0.054853,1.071705,0.652573,2.815865,1.071705,0.652573,404.283911,1.071705,0.652573
1,1,237,300,10014,1739,38.10,0.707600,1.270280,32.85,0.707600,1.270280,153.243256,0.707600,1.270280,6.761631,0.707600,1.270280,71.176,0.707600,1.270280,285.988108,0.707600,1.270280,2.637394,0.707600,1.270280,0.9982,0.707600,1.270280,0.123126,0.707600,1.270280,2.762543,0.707600,1.270280,236.046724,0.707600,1.270280
2,2,370,418,6530,1156,9.90,-0.261056,0.063651,8.90,-0.261056,0.063651,185.896102,-0.261056,0.063651,7.185714,-0.261056,0.063651,52.263,-0.261056,0.063651,64.444226,-0.261056,0.063651,2.650291,-0.261056,0.063651,0.9923,-0.261056,0.063651,0.101188,-0.261056,0.063651,1.615229,-0.261056,0.063651,164.457546,-0.261056,0.063651
3,3,472,602,17520,3181,52.30,1.195363,0.775899,50.05,1.195363,0.775899,352.715515,1.195363,0.775899,6.860745,1.195363,0.775899,79.014,1.195363,0.775899,276.663938,1.195363,0.775899,13.092741,1.195363,0.775899,0.9997,1.195363,0.775899,0.083555,1.195363,0.775899,3.371887,1.195363,0.775899,399.389787,1.195363,0.775899
4,4,737,822,16185,2809,64.60,1.617862,1.368686,63.60,1.617862,1.368686,580.630883,1.617862,1.368686,12.280833,1.617862,1.368686,128.686,1.617862,1.368686,400.985181,1.617862,1.368686,14.151337,1.617862,1.368686,0.9997,1.617862,1.368686,0.121595,1.617862,1.368686,4.507794,1.617862,1.368686,361.231504,1.617862,1.368686
5,5,911,1051,18778,3362,-15.20,-1.123229,-1.119469,-17.05,-1.123229,-1.119469,-492.687024,-1.123229,-1.119469,-14.015729,-1.123229,-1.119469,65.540,-1.123229,-1.119469,-126.836458,-1.123229,-1.119469,-7.165771,-1.123229,-1.119469,-0.9327,-1.123229,-1.119469,0.051037,-1.123229,-1.119469,2.011444,-1.123229,-1.119469,411.728185,-1.123229,-1.119469
6,6,1196,1301,15142,2659,10.00,-0.257621,-0.370640,3.00,-0.257621,-0.370640,-112.412493,-0.257621,-0.370640,0.161490,-0.257621,-0.370640,79.253,-0.257621,-0.370640,162.049120,-0.257621,-0.370640,-1.125590,-0.257621,-0.370640,0.9972,-0.257621,-0.370640,0.088099,-0.257621,-0.370640,3.378399,-0.257621,-0.370640,334.675354,-0.257621,-0.370640
7,7,1408,1506,15847,2791,43.50,0.893087,0.698542,46.45,0.893087,0.698542,312.955065,0.893087,0.698542,7.294118,0.893087,0.698542,88.815,0.893087,0.698542,373.322042,0.893087,0.698542,6.959140,0.893087,0.698542,0.9996,0.893087,0.698542,0.119275,0.893087,0.698542,3.771970,0.893087,0.698542,356.034572,0.893087,0.698542
8,8,1603,1699,15673,2799,13.75,-0.128811,-0.266511,18.35,-0.128811,-0.266511,78.229492,-0.128811,-0.266511,14.053226,-0.128811,-0.266511,98.259,-0.128811,-0.266511,150.434321,-0.128811,-0.266511,5.218667,-0.128811,-0.266511,0.9879,-0.128811,-0.266511,0.094227,-0.128811,-0.266511,3.257644,-0.128811,-0.266511,354.123841,-0.128811,-0.266511
9,9,1794,1847,9149,1648,15.35,-0.073851,0.131492,13.85,-0.073851,0.131492,0.000000,-0.073851,0.131492,-8.061710,-0.073851,0.131492,42.552,-0.073851,0.131492,74.603920,-0.073851,0.131492,3.454985,-0.073851,0.131492,0.9989,-0.073851,0.131492,0.103047,-0.073851,0.131492,2.652499,-0.073851,0.131492,227.231718,-0.073851,0.131492


In [ ]:
# Save the original Corpus text at 4 levels of semantic grouping: sentences, paragraphs, sections and chapters

# Save Section and Chapter DataFrames Metainformation (e.g. sent_no_start) and Sentiment Values
corpus_sects_summary_filename = f'corpus_section_summary_lexrules_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus Section Summary to file: {corpus_sects_summary_filename}')
corpus_sects_summary_df.to_csv(corpus_sects_summary_filename)

corpus_chaps_summary_filename = f'corpus_chapter_summary_lexrules_{author_str}_{title_str}.csv' # _{datetime_now}.csv'
print(f'Saving Corpus Chapters Summary to file: {corpus_chaps_summary_filename}')
corpus_chaps_summary_df.to_csv(corpus_chaps_summary_filename)

# Save Corpus Cruxes Dictionary is saved to a JSON file
corpus_cruxes_summary_filename = f'corpus_cruxes_summary_lexrules_{author_str}_{title_str}.json' # _{datetime_now}.csv'
print(f'Saving Corpus Cruxes Summary to file: {corpus_cruxes_summary_filename}')
with open(corpus_cruxes_summary_filename, 'w') as convert_file:
  convert_file.write(json.dumps(corpus_cruxes_dt))

Saving Corpus Section Summary to file: corpus_section_summary_lexrules_ianmcewan_machineslikeme.csv
Saving Corpus Chapters Summary to file: corpus_chapter_summary_lexrules_ianmcewan_machineslikeme.csv
Saving Corpus Cruxes Summary to file: corpus_cruxes_summary_lexrules_ianmcewan_machineslikeme.json


2

In [ ]:
# Verify exported Section Summary file

!head -n 5 $corpus_sects_summary_filename


,sect_no,sent_no_start,sent_no_mid,char_len,token_len,sentimentr,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,syuzhet,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,bing,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,sentiword,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,senticnet,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,nrc,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,afinn,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,vader,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,textblob,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,pattern,sentimentr_medianiqr,sentimentr_lnorm_medianiqr,stanza,sentimentr_medianiqr,sentimentr_lnorm_medianiqr
0,0,3,117,18434,3230,48.69999999999996,1.0717045942464558,0.6525729670629598,53.399999999999956,1.0717045942464558,0.6525729670629598,0.0,1.0717045942464558,0.6525729670629598,2.6672416666666656,1.0717045942464558,0.6525729670629598,141.68399999999946,1.0717045942464558,0.6525729670629598,436.71045158298784,1.0717045942464558,0.6525

In [ ]:
%whos

Variable                                Type                          Data/Info
-------------------------------------------------------------------------------
Afinn                                   type                          <class 'afinn.afinn.Afinn'>
CORPUS_AUTHOR                           str                           Ian McEwan
CORPUS_FILENAME                         str                           mlm_final_handclean_sections.txt
CORPUS_FULL                             str                           Machines Like Me by: Ian McEwan
CORPUS_SUBDIR                           str                           ./research/2021/sa_book_c<...>sa/imcewan_machineslikeme
CORPUS_TITLE                            str                           Machines Like Me
CubicSpline                             type                          <class 'scipy.interpolate._cubic.CubicSpline'>
FILE_OUTPUT                             str                           Major
HTML                                    type      

# **END OF NOTEBOOK**

# **Standardize and Remove Outliers (Auto)**

## **Remove Outliers**

### **Before Removing Outliers**

In [ ]:
for model_name in MODELS_LS:
  print(f'Plotting {model_name}')
  sns.lineplot(data=corpus_sents_df, x='sent_no', y=model_name, alpha=0.3, legend='brief', label=model_name)
      
plt.title(f'{CORPUS_FULL} (Model: {model_name}) \nRaw Sentence Sentiment Plot')

In [ ]:
# sns.lineplot(data=corpus_sents_df, x='sent_no', y='y_scaled', legend='brief', label='y_scaled')
      
# plt.title(f'{CORPUS_FULL} (Model: {model_name}) \nSMA Smoothed Sentence Sentiment Plot (windows={win_ls})')
# plt.legend(loc='best')


In [ ]:
# Plot all Raw Sentence Sentiment Arcs
#   Adjust scale_factor and mean_adj by hand to see EDA agreement among plots

for model_name in MODELS_LS:
  print(f'Plotting {model_name}')
  sns.lineplot(data=corpus_sents_df, x='sent_no', y=model_name, alpha=0.3, legend='brief', label=model_name)
      
plt.title(f'{CORPUS_FULL} (Model: {model_name}) \nRaw Sentence Sentiment Plot')
# plt.legend(loc='best');

### **Remove Outliers**

In [ ]:
# Trim outliers to max of 3*Median Abs Variance in Standardized Sentiment Time Series
#   and overwrite results in model_name column

# TODO: Add widget to select which models to include

# Sentence
for amodel_str in MODELS_LS:
  col_noouts_str = amodel_str + '_noouts'
  print(f'Sentence: {col_noouts_str} --------------------')
  corpus_sents_df[col_noouts_str] = clip_outliers(corpus_sents_df[amodel_str])
  
  print(f'  old Standardized max: {corpus_sents_df[amodel_str].max()}')
  print(f'  old Standardized min: {corpus_sents_df[amodel_str].min()}')
  print(f'  new max: {corpus_sents_df[col_noouts_str].max()}')
  print(f'  new min: {corpus_sents_df[col_noouts_str].min()}')
  
# col_rename_dt = rename_cols(corpus_sents_df, models_ls) # ERROR: created 1 new col with col_rename_dt dictionary name instead of mapping correctly
# col_rename_dt
# _ = corpus_sents_df.rename(columns=col_rename_dt, inplace=True, errors='raise');

# Paragraph
for amodel_str in MODELS_LS:
  col_noouts_str = amodel_str + '_noouts'
  print(f'Paragraph: {col_noouts_str} --------------------')
  corpus_parags_df[col_noouts_str] = clip_outliers(corpus_parags_df[amodel_str])

  print(f'  old Standardized max: {corpus_parags_df[amodel_str].max()}')
  print(f'  old Standardized min: {corpus_parags_df[amodel_str].min()}')
  print(f'  new max: {corpus_parags_df[col_noouts_str].max()}')
  print(f'  new min: {corpus_parags_df[col_noouts_str].min()}')

# Section
for amodel_str in MODELS_LS:
  col_noouts_str = amodel_str + '_noouts'
  print(f'Section: {col_noouts_str} --------------------')
  corpus_sects_df[col_noouts_str] = clip_outliers(corpus_sects_df[amodel_str])

  print(f'  old Standardized max: {corpus_sects_df[amodel_str].max()}')
  print(f'  old Standardized min: {corpus_sects_df[amodel_str].min()}')
  print(f'  new max: {corpus_sects_df[col_noouts_str].max()}')
  print(f'  new min: {corpus_sects_df[col_noouts_str].min()}')

In [ ]:
# Trim outliers to max of 3*Median Abs Variance in Standardized Sentiment Time Series
#   and overwrite results in model_name column

# TODO: Add widget to select which models to include
"""
# Sentences
for amodel in MODELS_LS:
  col_stand = amodel + '_stand'
  col_standout = amodel + '_standout'
  print(f'Sentences: {col_stand} --------------------')
  # corpus_sents_df[amodel] = corpus_sents_df[col_stand]
  corpus_sents_df[col_standout] = clip_outliers(corpus_sents_df[col_stand])
  
  print(f'  old Standardized max: {corpus_sents_df[col_stand].max()}')
  print(f'  old Standardized min: {corpus_sents_df[col_stand].min()}')
  print(f'  new max: {corpus_sents_df[col_standout].max()}')
  print(f'  new min: {corpus_sents_df[col_standout].min()}')
  
# col_rename_dt = rename_cols(corpus_sents_df, models_ls) # ERROR: created 1 new col with col_rename_dt dictionary name instead of mapping correctly
# col_rename_dt
# _ = corpus_sents_df.rename(columns=col_rename_dt, inplace=True, errors='raise');

# Paragraphs
for amodel in MODELS_LS:
  col_stand = amodel + '_stand'
  col_standout = amodel + '_standout'
  print(f'Paragraphs: {col_stand} --------------------')
  # corpus_parags_df[amodel] = corpus_parags_df[col_stand]
  corpus_parags_df[col_standout] = clip_outliers(corpus_parags_df[col_stand])
  print(f'  old Standardized max: {corpus_parags_df[col_stand].max()}')
  print(f'  old Standardized min: {corpus_parags_df[col_stand].min()}')
  print(f'  new max: {corpus_parags_df[col_standout].max()}')
  print(f'  new min: {corpus_parags_df[col_standout].min()}')
"""

### **After Removing Outliers**

In [ ]:
# Plot all Raw Sentence Sentiment Arcs
#   Adjust scale_factor and mean_adj by hand to see EDA agreement among plots

# Exlopre to find which Sentiment Time Series still have outliers after initial 2.5*Median Abs Dev Clipping
MODELS_SENTS_EXCLUDE_LS = ['nrc','bing','afinn','stanza']  # Likely these TS are not normal or heavy tailed so 2.5*MedAbsDev did not clip well
MODELS_SENTS_CUSTOM_LS = [x for x in MODELS_LS if x not in MODELS_SENTS_EXCLUDE_LS] 

for model_name in MODELS_SENTS_CUSTOM_LS:
  model_noouts = f'{model_name}_noouts'
  print(f'Plotting {model_noouts}')
  sns.lineplot(data=corpus_sents_df, x='sent_no', y=model_noouts, alpha=0.3, legend='brief', label=model_name)
      
plt.title(f'{CORPUS_FULL} (Model: {model_name}) \nRaw Sentence Sentiment w/Trimmed Outliers Plot')
# plt.legend(loc='best');

In [ ]:
# Plot all Raw Paragraph Sentiment Arcs
#   Adjust scale_factor and mean_adj by hand to see EDA agreement among plots

for model_name in MODELS_LS:
  model_standout = f'{model_name}_standout'
  print(f'Plotting {model_standout}')
  sns.lineplot(data=corpus_parags_df, x='parag_no', y=model_standout, alpha=0.3, legend='brief', label=model_name)
      
plt.title(f'{CORPUS_FULL} (Model: {model_name}) \nRaw Paragraph Sentiment Plot')
# plt.legend(loc='best');

## **Standardize Sentiment Time Series**

### **Before Standardizing**

In [ ]:
for model_name in MODELS_LS:
  model_noouts_str = f'{model_name}_noouts'
  print(f'Plotting {model_noouts_str}')
  sns.lineplot(data=corpus_sents_df, x='sent_no', y=model_noouts_str, alpha=0.3, legend='brief', label=model_name)
      
plt.title(f'{CORPUS_FULL} (Model: {model_noouts_str}) \nRaw Sentence Sentiment Plot')

### **Standardized the NoOutliers Sentiment Time Series**

In [ ]:
# Standardize Sentence and Paragraphs Sentiment Time Series (Section TS was Standardized above)

std_scaler = StandardScaler()

# orig_cols_ls = list(set(corpus_all_df.columns) - set(['sent_no','parag_no','sent_raw','sent_clean']))
# cols_ls = []

# Sentences
for acol in MODELS_LS:
    acol_new = acol + '_standouts'
    temp_np = std_scaler.fit_transform(np.array(corpus_sents_df[acol].values.reshape(-1,1)))
    corpus_sents_df[acol_new] = pd.Series(temp_np.squeeze())

# Paragraphs
for acol in MODELS_LS:
    acol_new = acol + '_standouts'
    temp_np = std_scaler.fit_transform(np.array(corpus_parags_df[acol].values.reshape(-1,1)))
    corpus_parags_df[acol_new] = pd.Series(temp_np.squeeze())

# Paragraphs
for acol in MODELS_LS:
    acol_new = acol + '_standouts'
    temp_np = std_scaler.fit_transform(np.array(corpus_sects_df[acol].values.reshape(-1,1)))
    corpus_sects_df[acol_new] = pd.Series(temp_np.squeeze())


### **After Standardizing**

In [ ]:
for model_name in MODELS_LS:
  model_standouts_str = f'{model_name}_standouts'
  print(f'Plotting {model_standouts_str}')
  sns.lineplot(data=corpus_sents_df, x='sent_no', y=model_standouts_str, alpha=0.3, legend='brief', label=model_name)
      
plt.title(f'{CORPUS_FULL} (Model: {model_standouts_str}) \nRaw Sentence Sentiment Plot')

In [ ]:
MODELS_LS

## **Deselect Poorly Behaved Sentiment Time Series Model**

In [ ]:
VADER_Arc = True #@param {type:"boolean"}
TextBlob_Arc = True #@param {type:"boolean"}
Stanza_Arc = True #@param {type:"boolean"}
SentimentR_Arc = True #@param {type:"boolean"}
Syuzhet_Arc = True #@param {type:"boolean"}
AFINN_Arc = True #@param {type:"boolean"}
Bing_Arc = True #@param {type:"boolean"}
Pattern_Arc = True #@param {type:"boolean"}
SentiWord_Arc = True #@param {type:"boolean"}
SenticNet_Arc = True #@param {type:"boolean"}
NCR_Arc = True #@param {type:"boolean"}
MPQA_Arc = False #@param {type:"boolean"}
SentiStrength_Arc = False #@param {type:"boolean"}


In [ ]:
# Create and Verify custom list of Models to include

MODELS_CUSTOM_LS = []

if VADER_Arc:
  MODELS_CUSTOM_LS.append('vader')
if TextBlob_Arc:
  MODELS_CUSTOM_LS.append('textblob')
if Stanza_Arc:
  MODELS_CUSTOM_LS.append('stanza')
if SentimentR_Arc:
  MODELS_CUSTOM_LS.append('sentimentr')
if Syuzhet_Arc:
  MODELS_CUSTOM_LS.append('syuzhet')
if AFINN_Arc:
  MODELS_CUSTOM_LS.append('afinn')
if Bing_Arc:
  MODELS_CUSTOM_LS.append('bing')
if Pattern_Arc:
  MODELS_CUSTOM_LS.append('pattern')
if SentiWord_Arc:
  MODELS_CUSTOM_LS.append('sentiword')
if SenticNet_Arc:
  MODELS_CUSTOM_LS.append('senticnet')
if NCR_Arc:
  MODELS_CUSTOM_LS.append('nrc')

print(f'Here are the Models we are using to ensemble and save:\n   {MODELS_CUSTOM_LS}')

models_incl_ls = []
for amodel in MODELS_CUSTOM_LS:
  models_incl_ls.append(amodel[:2])
models_incl_str = ''.join(models_incl_ls)

print(f'Here is a custom name abbr: {models_incl_str}')

## **Calculate Median of All (Trimmed Outliers then Standardized) Sentiment Time Series**

In [ ]:
# Create a list of models with Outliers trimmed and Standardized Time Series 

MODELS_CUSTOM_STANDOUTS_LS = []

for amodel in MODELS_CUSTOM_LS:
  model_standout_str = f'{amodel}_standouts'
  MODELS_CUSTOM_STANDOUTS_LS.append(model_standout_str)

print(f'List of NoOutliers/Standardized Models to compute Median on: {MODELS_CUSTOM_STANDOUTS_LS}')

In [ ]:
# Disregard poorly behaved time series identified and stored in MODELS_SENTS_EXCLUDE_LS 

# TODO: Add widget to select which models to include

corpus_sents_df['median_standouts_custom_lex'] = corpus_sents_df[MODELS_CUSTOM_STANDOUTS_LS].median(axis=1)
# corpus_sents_df.head(2)

corpus_sents_df['std_standouts_custom_lex'] = corpus_sents_df[MODELS_CUSTOM_STANDOUTS_LS].std(axis=1)
# corpus_sents_df.head(2)

corpus_sents_df['mean_standouts_custom_lex'] = corpus_sents_df[MODELS_CUSTOM_STANDOUTS_LS].mean(axis=1)
# corpus_sents_df.head(2)

In [ ]:
# (For now) Paragraph Sentiment TS are better behaved and don't require excluding any Model

corpus_parags_df['median_standouts_custom_lex'] = corpus_parags_df[MODELS_CUSTOM_STANDOUTS_LS].median(axis=1)
# corpus_parags_df.head(2)

corpus_parags_df['std_standouts_custom_lex'] = corpus_parags_df[MODELS_CUSTOM_STANDOUTS_LS].std(axis=1)
# corpus_parags_df.head(2)

corpus_parags_df['mean_standouts_custom_lex'] = corpus_parags_df[MODELS_CUSTOM_STANDOUTS_LS].mean(axis=1)
# corpus_parags_df.head(2)

In [ ]:
# (For now) Paragraph Sentiment TS are better behaved and don't require excluding any Model

corpus_sects_df['median_standouts_custom_lex'] = corpus_sects_df[MODELS_CUSTOM_STANDOUTS_LS].median(axis=1)
# corpus_parags_df.head(2)

corpus_sects_df['std_standouts_custom_lex'] = corpus_sects_df[MODELS_CUSTOM_STANDOUTS_LS].std(axis=1)
# corpus_parags_df.head(2)

corpus_sects_df['mean_standouts_custom_lex'] = corpus_sects_df[MODELS_CUSTOM_STANDOUTS_LS].mean(axis=1)
# corpus_parags_df.head(2)

## **Save Processed Sentiment Time Series**

In [ ]:
# Save Preprocessed Corpus Sentences DataFrame

author_str = ''.join(CORPUS_AUTHOR.split()).lower()
title_str = ''.join(CORPUS_TITLE.split()).lower()
datetime_now = datetime.utcnow().strftime("%Y%m%d_%H%M")

corpus_sents_filename = f'corpus_sentences_lexrules_{models_incl_str}_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving Corpus Sentences to file: {corpus_sents_filename}')
corpus_sents_df.to_csv(corpus_sents_filename)

corpus_parags_filename = f'corpus_paragraphs_lexrules_{models_incl_str}_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving Corpus Paragraphs to file: {corpus_parags_filename}')
corpus_parags_df.to_csv(corpus_parags_filename)

corpus_sects_filename = f'corpus_sections_lexrules_{models_incl_str}_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving Corpus Sections to file: {corpus_sects_filename}')
corpus_sects_df.to_csv(corpus_sects_filename)

corpus_cruxes_filename = f'corpus_cruxes_lexrules_{models_incl_str}_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving Corpus Cruxes to file: {corpus_cruxes_filename}')
with open(corpus_cruxes_filename, 'w') as convert_file:
  convert_file.write(json.dumps(corpus_cruxes_dt))

# **EDA Visualizations and Comparisons**

In [ ]:
# Verify

corpus_sents_df.head(2)
corpus_sents_df.shape

In [ ]:
# Verify

corpus_parags_df.head(2)
corpus_parags_df.shape

In [ ]:
# Verify

corpus_sects_df.head(2)
corpus_sects_df.shape

In [ ]:
corpus_cruxes_dt

In [ ]:
# norm_cols_ls = ['distbertsst_norm', 'nlptown_norm','xlnet_sst5_norm','bert_imdb_norm', 'bertuc_googapps_norm', 'roberta_lg15_norm']
"""

# ARCHIVE

cols_norm_ls = []
cols_stand_ls = []

for acol in corpus_all_df.columns:
  if acol.endswith('_norm'):
    print(f'Adding {acol} to norm_cols_ls')
    cols_norm_ls.append(acol)
  elif acol.endswith('_stand'):
    print(f'Adding {acol} to stand_cols_ls')
    cols_stand_ls.append(acol)
  else:
    pass

print(f'\nNormalized Columns: {cols_norm_ls}')

print(f'\nStandardized Columns: {cols_stand_ls}')
""";

### **LOWESS Smoothed Single Plot**

**Normalized Sentiment Smoothed with LOWESS**

In [ ]:
MODELS_STAND_LS = []
for amodel in MODELS_LS:
  MODELS_STAND_LS.append(f'{amodel}_stand')

print(f'MODELS_STAND_LS: {MODELS_STAND_LS}')

**Standardized Sentiment Smoothed with LOWESS**

In [ ]:
# Plot and Compare all LOWESS Smoothed *Standardized* Sentiment Time Series

corpus_lowess_stand_df = get_lowess(corpus_all_df, cols_stand_ls, plot_subtitle='Standardized', afrac=1./10, ait=5, do_plot=True, save2file=False)

### **High Level: Section View**

##### **Raw Sentiment**

In [ ]:
# Plot Raw Section Sentiments
"""

# ARCHIVED

for amodel in MODELS_CUSTOM_STANDOUTS_LS:
  amodel_stand_str = f'{amodel}'
  plot_raw_sentiments(model_name=amodel_stand_str, semantic_type='section', save2file=False)
"""

In [ ]:
# Raw Standardized Section Sentiment Time Series

_ = plot_stand_crux_sections(ts_df=corpus_sects_df, model_names_ls=MODELS_CUSTOM_STANDOUTS_LS, semantic_type='section', label_token_ct=5, title_xpos=0.5, title_ypos=1, save2file=False)

In [ ]:
# Plot the Median of the Customized Set of NoOutliers/Standardized Section Sentiment Time Series

corpus_sects_df['median_standouts_custom_lex'].plot()
plt.title(f'{CORPUS_FULL}\n Median of NoOutliers/Standardized Section Sentiment Time Series');

##### **SMA**

In [ ]:
Window_Width = 4 #@param {type:"slider", min:2, max:20, step:1}

# DEFAULT 5-20

In [ ]:
# SMA of Custom Set of NoOutliers/Standardized Section Sentiment Time Series

# NOTE: EDA/Adjust the win_ls to explore different window_size by hand to see EDA agreement among plots

for model_name in MODELS_CUSTOM_STANDOUTS_LS:
  # print(f'Plotting {acol_name}')
  get_smas(corpus_sects_df, model_name, text_unit='section', subtitle_str='(NOTE: different x-scale)', win_ls=[Window_Width])

##### **LOWESS**

In [ ]:
# Standardized Section LOWESS Smoothed Sentiment Time Series

_ = get_lowess(corpus_sects_df, models_ls=MODELS_STAND_LS, text_unit='section', afrac=1./4, do_plot=True, save2file=False)

In [ ]:
LOWESS_fraction = 0.15 #@param {type:"slider", min:0.1, max:0.3, step:0.01}

# Default: 1./6 (approx 0.17)

In [ ]:
# Standardized Median Section LOWESS Smoothed Sentiment Time Series

_ = get_lowess(corpus_sects_df, models_ls=['median'], text_unit='section', afrac=LOWESS_fraction, do_plot=True, save2file=False)

### **Middle Level: Paragraph Views**

##### **Raw Sentiment**

In [ ]:
# Plot Custom Set of NoOutliers/Standardized Paragraph Sentiments Time Series 

for amodel in MODELS_CUSTOM_STANDOUTS_LS:
  plot_raw_sentiments(model_name=amodel, semantic_type='paragraph', save2file=False)

In [ ]:
# Plot the Median for the Custom Set of NoOutlier/Standardized Paragraph Sentiment Time Series

corpus_parags_df['median_standouts_custom_lex'].plot()
plt.title(f'{CORPUS_FULL}\n Median of Custom Set of NoOutlier/Standardized Paragraph Sentiment Time Series');

##### **SMA**

In [ ]:
Window_Width = 10 #@param {type:"slider", min:5, max:20, step:1}

# DEFAULT 5-20

In [ ]:
# SMA Standardized Paragraph Sentiment Arcs

# NOTE: EDA/adjust the win_size below by hand to see EDA agreement among plots (5-20 defaults)

for model_name in MODELS_CUSTOM_STANDOUTS_LS:
  get_smas(corpus_parags_df, model_name, text_unit='paragraph', win_ls=[Window_Width])

In [ ]:
Window_Percentage = 5 #@param {type:"slider", min:5, max:20, step:1}

# DEFAULT: 5-20

In [ ]:
# SMA Plot the Median for the Custom Set of NoOutlier/Standardized Paragraph Sentiment Time Series

# NOTE: EDA/adjust the win_size below by hand to see EDA agreement among plots

win_percentage = Window_Percentage  # 5 means rolling window size is 5% of corpus length (5-20 default)

win_size = int(corpus_parags_df.shape[0]*(win_percentage*0.01))

plot_title = f'{CORPUS_FULL}\n Median of Custom Set of NoOutlier/Standardized Paragraph Sentiment Time Series (win={win_percentage}%)'
corpus_parags_df['median_standouts_custom_lex'].rolling(win_size).mean().plot(title=plot_title)

##### **LOWESS**

In [ ]:
LOWESS_fraction = 0.17 #@param {type:"slider", min:0.1, max:0.3, step:0.01}

# Default: 1./6 (approx 0.17)

In [ ]:
# Standardized Paragraph LOWESS Smoothed Sentiment Time Series

# NOTE: EDA/adjust the win_size below by hand to see EDA agreement among plots (0.10-0.20 default)

_ = get_lowess(corpus_parags_df, models_ls=MODELS_CUSTOM_STANDOUTS_LS, text_unit='paragraph', afrac=LOWESS_fraction, do_plot=True, save2file=False)

In [ ]:
LOWESS_fraction = 0.12 #@param {type:"slider", min:0.1, max:0.3, step:0.01}

# Default: 1./6 to 1./8 (approx 0.17 to 0.12)

In [ ]:
# Plot it

_ = get_lowess(corpus_parags_df, models_ls=['median_standouts_custom_lex'], text_unit='paragraph', afrac=LOWESS_fraction, do_plot=True, save2file=False)

### **Low Level: Sentence Views**

##### **SMA**

In [ ]:
Window_Percentage = 10 #@param {type:"slider", min:5, max:20, step:1}

# DEFAULT: 5-20

In [ ]:
# Plot all Standardized Sentence Sentiment Arcs
#   Adjust scale_factor and mean_adj by hand to see EDA agreement among plots

models_stand_ls = []
for model_name in MODELS_CUSTOM_STANDOUTS_LS:
  get_smas(corpus_sents_df, model_name, text_unit='sentence', alpha=0.3, win_ls=[Window_Percentage])

# print(f'models_stand_ls: {models_stand_ls}')
corpus_sents_df['median_stand'] = corpus_sents_df[models_stand_ls].median()
plt.plot(corpus_sents_df.sent_no, corpus_sents_df.median_stand, color='black', label='Median')
plt.legend(loc='best')
plt.show()

##### **LOWESS**

In [ ]:
LOWESS_fraction = 0.12 #@param {type:"slider", min:0.1, max:0.3, step:0.01}

# Default: 1./6 to 1./8 (approx 0.17 to 0.12)

In [ ]:
# Standardized Paragraph LOWESS Smoothed Sentiment Time Series

_ = get_lowess(corpus_sents_df, models_ls=MODELS_CUSTOM_STANDOUTS_LS, text_unit='sentence', afrac=LOWESS_fraction, do_plot=True, save2file=False)

In [ ]:
LOWESS_fraction = 0.12 #@param {type:"slider", min:0.1, max:0.3, step:0.01}

# Default: 1./6 to 1./8 (approx 0.17 to 0.12)

In [ ]:
# Median of Custom Set of NoOutlier/Standardized Sentiment Time Series

_ = get_lowess(corpus_sents_df, models_ls=['median_standouts_custom_lex'], text_unit='paragraph', afrac=1./8, do_plot=True, save2file=False)

## **Save Standardized-NoOutliers Sentiment Values**

In [ ]:
# Save all Processed DataFrames

author_str = ''.join(CORPUS_AUTHOR.split()).lower()
title_str = ''.join(CORPUS_TITLE.split()).lower()
datetime_now = datetime.utcnow().strftime("%Y%m%d_%H%M")


# Save Preprocessed Corpus Sentences DataFrame
corpus_sents_filename = f'corpus_sentences_only_lexrules_standouts_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving to file: {corpus_sents_filename}')
corpus_sents_df.to_csv(corpus_sents_filename)


# Save Preprocessed Corpus Paragraphs DataFrame
corpus_parags_filename = f'corpus_paragraphs_only_lexrules_standouts_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving to file: {corpus_parags_filename}')
corpus_parags_df.to_csv(corpus_parags_filename)


# Save Preprocessed Corpus Section DataFrame
corpus_sects_filename = f'corpus_sections_only_lexrules_standouts_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving to file: {corpus_sects_filename}')
corpus_sects_df.to_csv(corpus_sects_filename)


# Save Cruxes
corpus_cruxes_filename = f'corpus_cruxes_lexrules_{models_incl_str}_{author_str}_{title_str}_{datetime_now}.csv'
print(f'Saving Corpus Cruxes to file: {corpus_cruxes_filename}')
with open(corpus_cruxes_filename, 'w') as convert_file:
  convert_file.write(json.dumps(corpus_cruxes_dt))

# **Metrics**

## **Sentiment Stability**

## **Crux Point Analysis**

### **Gather (n) Highest/Lowest Sentiment Values for Each Model**

In [ ]:
def getn_cruxes(crux_dt, model_name='vader', get_n=6):
  '''
  Given a Crux Dictionary, a Model item/dict within, and and integer n
  Return the n highest and n lowest sentiment values
  NOTE: if get_n == 0, return all Crux Points for all Models
  '''

  cruxes_all_df = pd.DataFrame
  cruxes_n_top_df = pd.DataFrame()

  cruxes_all_df = pd.DataFrame.from_dict(crux_dt[model_name])

  cruxes_all_df = cruxes_all_df.transpose().reset_index().rename(columns={'index':'var'})

  cruxes_all_df.rename(columns={'var':'sent_no',0:model_name,1:'sent_raw'}, inplace=True)
  cruxes_all_df.drop(columns=['sent_raw'], inplace=True)
  cruxes_all_df.rename(columns={model_name:'sentiment'}, inplace=True)
  cruxes_all_df['sentiment'] = cruxes_all_df['sentiment'].astype('float')
  cruxes_all_df['model_name'] = model_name
  cruxes_all_df = cruxes_all_df[['sent_no','model_name','sentiment']]

  if get_n > 0:
    cruxes_n_top_df = cruxes_all_df.nlargest(get_n, 'sentiment')
    cruxes_n_top_df = cruxes_n_top_df.append(cruxes_all_df.nsmallest(get_n, 'sentiment'))
  elif get_n ==0:
    cruxes_n_top_df = cruxes_all_df
  else:
    print(f'ERROR: argument get_n must be either 0 (return all Cruxes) or greater than 0')
    
  return cruxes_n_top_df

# Test

cruxes_n_top_df = getn_cruxes(corpus_cruxes_dt, model_name='vader', get_n=3)
cruxes_n_top_df.head(6)
cruxes_n_top_df.info()

In [ ]:
# Acculumate all the Crux Points from All Models into one cruxes_n_top_all_df DataFrame

cruxes_n_top_all_df = pd.DataFrame()

for amodel in MODELS_LS:
  print(f'Appending Cruxes from {amodel}')
  cruxes_n_top_df = getn_cruxes(corpus_cruxes_dt, model_name=amodel, get_n=12)
  cruxes_n_top_all_df = cruxes_n_top_all_df.append(cruxes_n_top_df, ignore_index=True)

for amodel in MODELS_LS:
  crux_ct = len(cruxes_n_top_all_df[cruxes_n_top_all_df['model_name'] == amodel])
  print(f'{amodel.capitalize()} has {crux_ct} Cruxes')

print(f'There are a total of {cruxes_n_top_all_df.shape[0]} in all Models')

In [ ]:
# Plot Crux Points in 2D Space: Scatterplot

sns.lmplot('sent_no', 'sentiment', data=cruxes_n_top_all_df, fit_reg=False)


In [ ]:
# Plot Crux Points in 1D Space: Histogram

sns.distplot(cruxes_n_top_all_df.sent_no, bins=2000)


In [ ]:
sns.clustermap(cruxes_n_top_all_df.sent_no)


## **Sentiment Arc Analysis**

# **END OF NOTEBOOK**

In [ ]:
# Verify the *.csv file was created

latest_file = get_recentfile('csv')
print(f'Most recenst *.csv file: {latest_file}')

!ls -altr $latest_file
print('\n')
!head -n 2 $latest_file

In [ ]:
# Plot Raw Sentiment Values

plt.plot(corpus_sents_df['vader'], alpha=0.3, label='VADER')
plt.plot(corpus_sents_df['textblob'], alpha=0.3, label='TextBlob')
plt.plot(corpus_sents_df['stanza'], alpha=0.3, label='Stanza')
plt.plot(corpus_sents_df['afinn'], alpha=0.3, label='AFINN')
plt.plot(corpus_sents_df['sentimentr'], alpha=0.3, label='SentimentR')
plt.plot(corpus_sents_df['syuzhet'], alpha=0.3, label='Syuzhet')
plt.plot(corpus_sents_df['bing'], alpha=0.3, label='Bing')
plt.plot(corpus_sents_df['pattern'], alpha=0.3, label='Bing')
plt.plot(corpus_sents_df['sentiword'], alpha=0.3, label='SentiWord')
plt.plot(corpus_sents_df['senticnet'], alpha=0.3, label='SenticNet')
plt.plot(corpus_sents_df['nrc'], alpha=0.3, label='NRC')
plt.plot(corpus_sents_df['median_lex'], label='Median')
plt.plot(corpus_sents_df['mean_lex'], label='Mean')
plt.legend()
plt.title(f'{CORPUS_FULL} \n Raw Sentence Sentiment (All)')

if (PLOT_OUTPUT in ['All']):
  # Save graph to file.
  plot_filename = 'raw_sent_sa_all.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

In [ ]:
# Plot 5% Median SMA for Raw Sentiment Values

plt.plot(corpus_sents_df['vader'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='VADER')
plt.plot(corpus_sents_df['textblob'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='TextBlob')
plt.plot(corpus_sents_df['stanza'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='Stanza')
plt.plot(corpus_sents_df['afinn'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='AFINN')
plt.plot(corpus_sents_df['sentimentr'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='SentimentR')
plt.plot(corpus_sents_df['syuzhet'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='Syuzhet')
plt.plot(corpus_sents_df['bing'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='Bing')
plt.plot(corpus_sents_df['pattern'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='Pattern')
plt.plot(corpus_sents_df['sentiword'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='SentiWord')
plt.plot(corpus_sents_df['senticnet'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='SenticNet')
plt.plot(corpus_sents_df['nrc'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='NRC')
plt.plot(corpus_sents_df['median_lex'].rolling(5*win_1per, center=True).mean(), alpha=0.9, label='Median', color='black')
plt.plot(corpus_sents_df['std_lex'].rolling(5*win_1per, center=True).mean(), alpha=0.9, label='STD', color='red')
plt.plot(corpus_sents_df['mean_lex'].rolling(5*win_1per, center=True).mean(), alpha=0.9, label='Mean', color='grey')
plt.legend()
plt.title(f'{CORPUS_FULL} \n 5% Median SMA for Raw Sentence Sentiment (All)')

if (PLOT_OUTPUT in ['All','Major']):
  # Save graph to file.
  plot_filename = 'plot_sent_sa_050100mean_all.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

**Calculate Median of All Normed Sentiment Polarities**

In [ ]:
# DELETE? Create DataFrame with just raw Sentiment Values

sentiment_sentno_cols_ls = sentiment_only_cols_ls.copy()
sentiment_sentno_cols_ls.insert(0, 'sent_no')

corpus_sents_sa_raw_df = corpus_sents_df[sentiment_sentno_cols_ls].copy()

corpus_sents_sa_raw_df.head()

In [ ]:
# Review Sentiment Value Statistics for all Models

corpus_lexicons_stats_dt

In [ ]:
# Normalize all Sentiment Values to same Sentiment Range -1.0 to +1.0 using sklearn.MinMaxScaler()

min_max_scaler = MinMaxScaler(feature_range=(-1.0,1.0))
corpus_sents_norm_np = min_max_scaler.fit_transform(corpus_sents_df[sentiment_only_cols_ls])
corpus_sents_norm_df = pd.DataFrame(corpus_sents_norm_np)
corpus_sents_norm_df.columns = sentiment_only_cols_ls
corpus_sents_norm_df.insert(0, 'sent_no', corpus_sents_df['sent_no'])

corpus_sents_norm_df.head()

In [ ]:
corpus_sents_norm_df['median'] = corpus_sents_norm_df[sentiment_only_cols_ls].median(axis=1)
corpus_sents_norm_df.head()

In [ ]:
corpus_sents_norm_df['std'] = corpus_sents_norm_df[sentiment_only_cols_ls].std(axis=1)
corpus_sents_norm_df.head()

In [ ]:
corpus_sents_norm_df['mean'] = corpus_sents_norm_df[sentiment_only_cols_ls].mean(axis=1)
corpus_sents_norm_df.head()

In [ ]:
# Verify all Sentiment within -1.0 to +1.0

corpus_sents_norm_df.describe()

**Save all Normed Sentiment Value to File**

In [ ]:
# Save all the calculated Norm Sentiment Values

sats_norm_filename_str = f'corpus_sa_norm_range_all.csv'
print(sats_norm_filename_str)

sentiment_pathfilename_str = gen_pathfiletime(sats_norm_filename_str)

print(f'Saving all Norm Sentiment Values from All Models to file: {sentiment_pathfilename_str}')
corpus_sents_norm_df.to_csv(sentiment_pathfilename_str, index=False)

In [ ]:
# Verify the *.csv file was created

latest_file = get_recentfile('csv')

!ls -altr $latest_file
print('\n')
!head -n 5 $latest_file

In [ ]:
# Plot Normed Sentiment Values

plt.plot(corpus_sents_norm_df['vader'], alpha=0.3, label='VADER')
plt.plot(corpus_sents_norm_df['textblob'], alpha=0.3, label='TextBlob')
plt.plot(corpus_sents_norm_df['stanza'], alpha=0.3, label='Stanza')
plt.plot(corpus_sents_norm_df['afinn'], alpha=0.3, label='AFINN')
plt.plot(corpus_sents_norm_df['sentimentr'], alpha=0.3, label='SentimentR')
plt.plot(corpus_sents_norm_df['syuzhet'], alpha=0.3, label='Syuzhet')
plt.plot(corpus_sents_norm_df['bing'], alpha=0.3, label='Bing')
plt.plot(corpus_sents_norm_df['sentiword'], alpha=0.3, label='SentiWord')
plt.plot(corpus_sents_norm_df['senticnet'], alpha=0.3, label='SenticNet')
plt.plot(corpus_sents_norm_df['nrc'], alpha=0.3, label='NRC')
plt.plot(corpus_sents_norm_df['mean'], label='Mean', color='grey')
plt.plot(corpus_sents_norm_df['median'], label='Median', color='black')

plt.title(f'{CORPUS_FULL} \n Sentence Sentiment - Normed Range (All)')
plt.legend()

if (PLOT_OUTPUT in ['All']):
  # Save graph to file.
  plot_filename = 'raw_norm_range_sent_sa_all.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

In [ ]:
# Plot 5% SMA Normed Sentiment Values

plt.plot(corpus_sents_norm_df['vader'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='VADER')
plt.plot(corpus_sents_norm_df['textblob'].rolling(5*win_1per, center=True).mean(),  alpha=0.3, label='TextBlob')
plt.plot(corpus_sents_norm_df['stanza'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='Stanza')
plt.plot(corpus_sents_norm_df['afinn'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='AFINN')
plt.plot(corpus_sents_norm_df['sentimentr'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='SentimentR')
plt.plot(corpus_sents_norm_df['syuzhet'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='Syuzhet')
plt.plot(corpus_sents_norm_df['bing'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='Bing')
plt.plot(corpus_sents_norm_df['sentiword'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='SentiWord')
plt.plot(corpus_sents_norm_df['senticnet'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='SenticNet')
plt.plot(corpus_sents_norm_df['nrc'].rolling(5*win_1per, center=True).mean(), alpha=0.3, label='NRC')
plt.plot(corpus_sents_norm_df['median'].rolling(5*win_1per, center=True).mean(), alpha=0.8, label='Median', color='black')
plt.plot(corpus_sents_norm_df['mean'].rolling(5*win_1per, center=True).mean(), alpha=0.6, label='Mean', color='grey')
plt.plot(corpus_sents_norm_df['std'].rolling(5*win_1per, center=True).mean(), alpha=0.8, label='STD', color='red')

plt.title(f'{CORPUS_FULL} \n 5% SMA Sentence Sentiment - Normed Range -1.0 to +1.0 (All)')
plt.legend()

if (PLOT_OUTPUT in ['All','Major']):
  # Save graph to file.
  plot_filename = 'plot_5sma_norm_range_sent_sa_all.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

**[ALL] Sentiment Arcs with Normed Range and Mean**

In [ ]:
corpus_sents_norm_df.head()

In [ ]:
# Recenter [ALL] Time Series with normed Range -1.0 to 1.0 and Mean = 0

vader_mean = corpus_sents_norm_df['vader'].mean()
corpus_sents_norm_df['vader_2norm'] = corpus_sents_norm_df['vader'].apply(lambda x: x-vader_mean)

textblob_mean = corpus_sents_norm_df['textblob'].mean()
corpus_sents_norm_df['textblob_2norm'] = corpus_sents_norm_df['vader'].apply(lambda x: x-textblob_mean)

stanza_mean = corpus_sents_norm_df['stanza'].mean()
corpus_sents_norm_df['stanza_2norm'] = corpus_sents_norm_df['stanza'].apply(lambda x: x-stanza_mean)

afinn_mean = corpus_sents_norm_df['afinn'].mean()
corpus_sents_norm_df['afinn_2norm'] = corpus_sents_norm_df['afinn'].apply(lambda x: x-afinn_mean)

sentimentr_mean = corpus_sents_norm_df['sentimentr'].mean()
corpus_sents_norm_df['sentimentr_2norm'] = corpus_sents_norm_df['sentimentr'].apply(lambda x: x-sentimentr_mean)

syuzhet_mean = corpus_sents_norm_df['syuzhet'].mean()
corpus_sents_norm_df['syuzhet_2norm'] = corpus_sents_norm_df['syuzhet'].apply(lambda x: x-syuzhet_mean)

bing_mean = corpus_sents_norm_df['bing'].mean()
corpus_sents_norm_df['bing_2norm'] = corpus_sents_norm_df['bing'].apply(lambda x: x-bing_mean)

sentiword_mean = corpus_sents_norm_df['sentiword'].mean()
corpus_sents_norm_df['sentiword_2norm'] = corpus_sents_norm_df['sentiword'].apply(lambda x: x-sentiword_mean)

senticnet_mean = corpus_sents_norm_df['senticnet'].mean()
corpus_sents_norm_df['senticnet_2norm'] = corpus_sents_norm_df['senticnet'].apply(lambda x: x-senticnet_mean)

nrc_mean = corpus_sents_norm_df['nrc'].mean()
corpus_sents_norm_df['nrc_2norm'] = corpus_sents_norm_df['nrc'].apply(lambda x: x-senticnet_mean)

median_mean = corpus_sents_norm_df['median'].mean()
corpus_sents_norm_df['median_2norm'] = corpus_sents_norm_df['median'].apply(lambda x: x-median_mean)

# plt.plot(corpus_sents_norm_df['vader'].rolling(5*win_1per).mean(), alpha=0.3, label='VADER')
plt.plot(corpus_sents_norm_df['vader_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='VADER-2n')

# plt.plot(corpus_sents_norm_df['textblob'].rolling(5*win_1per).mean(), alpha=0.3, label='TextBlob')
plt.plot(corpus_sents_norm_df['textblob_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='TextBlob-2n')

# plt.plot(corpus_sents_norm_df['stanza'].rolling(5*win_1per).mean(), alpha=0.3, label='Stanza')
plt.plot(corpus_sents_norm_df['stanza_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Stanza-2n')

# plt.plot(corpus_sents_norm_df['afinn'].rolling(5*win_1per).mean(), alpha=0.3, label='AFINN')
plt.plot(corpus_sents_norm_df['afinn_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='AFINN-2n')

# plt.plot(corpus_sents_norm_df['sentimentr'].rolling(5*win_1per).mean(), alpha=0.3, label='SentimentR')
plt.plot(corpus_sents_norm_df['sentimentr_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='SentimentR-2n')

# plt.plot(corpus_sents_norm_df['syuzhet'].rolling(5*win_1per).mean(), alpha=0.3, label='Syuzhet')
plt.plot(corpus_sents_norm_df['syuzhet_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Syuzhet-2n')

# plt.plot(corpus_sents_norm_df['bing'].rolling(5*win_1per).mean(), alpha=0.3, label='Bing')
plt.plot(corpus_sents_norm_df['bing_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Bing-2n')

# plt.plot(corpus_sents_norm_df['sentiword'].rolling(5*win_1per).mean(), alpha=0.3, label='SentiWord')
plt.plot(corpus_sents_norm_df['sentiword_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='SentiWord-2n')

# plt.plot(corpus_sents_norm_df['senticnet'].rolling(5*win_1per).mean(), alpha=0.3, label='SenticNet')
plt.plot(corpus_sents_norm_df['senticnet_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='SenticNet-2n')

# plt.plot(corpus_sents_norm_df['nrc'].rolling(5*win_1per).mean(), alpha=0.3, label='NRC')
plt.plot(corpus_sents_norm_df['nrc_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='NRC-2n')

# plt.plot(corpus_sents_norm_df['median'].rolling(5*win_1per).mean(), alpha=0.3, label='Median')
plt.plot(corpus_sents_norm_df['median_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Median-2n');

plt.title(f'{CORPUS_FULL} \n 5% SMA Sentence Sentiment - Normed Range + Mean (All)')
plt.legend()

if (PLOT_OUTPUT in ['All','Major']):
  # Save graph to file.
  plot_filename = 'plot_050100sma_norm_rm_sent_sa_all.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

In [ ]:
# Save Sentiment Values Normedx2 (Range and Mean)

sats_norm_rm_str = f'corpus_sa_norm_rangemean_all.csv'
print(sats_norm_rm_str)

sentiment_norm_pathfilename_str = gen_pathfiletime(sats_norm_rm_str)

print(f'Saving all Norm Sentiment Values from All Models to file: {sentiment_norm_pathfilename_str}')
corpus_sents_norm_df.to_csv(sentiment_norm_pathfilename_str, index=False)

In [ ]:
# Verify the *.csv file was created

latest_file = get_recentfile('csv')

!ls -altr $latest_file
print('\n')
!head -n 5 $latest_file

# **(Optional) Remove Model from Corpus Sentiment Time Series

**[OUTLIERS DROPPED] Sentiment Arcs with Normed Range and Mean**

In [ ]:
# TODO: Add AutoML to provide <outlier metrics> on each Sentiment Arc

sentiment_only_cols_ls

**Deselect Outlier Sentiment Arc then Run the next Code Cell (to get a more Accurate Ensemble)**

In [ ]:
VADER_Arc = True #@param {type:"boolean"}
TextBlob_Arc = True #@param {type:"boolean"}
Stanza_Arc = True #@param {type:"boolean"}
SentimentR_Arc = True #@param {type:"boolean"}
Syuzhet_Arc = True #@param {type:"boolean"}
AFINN_Arc = True #@param {type:"boolean"}
Bing_Arc = True #@param {type:"boolean"}
Pattern_Arc = True #@param {type:"boolean"}
SentiWord_Arc = True #@param {type:"boolean"}
SenticNet_Arc = True #@param {type:"boolean"}
NCR_Arc = True #@param {type:"boolean"}
MPQA_Arc = False #@param {type:"boolean"}
SentiStrength_Arc = False #@param {type:"boolean"}

In [ ]:
for amodel in models_ls:
  print(amodel)

In [ ]:
win_1per

In [ ]:
# Recenter Similar Normed Time Series with Mean set to Zero (Dropping Outliers)

for model_name:
sentiment_noout_cols_ls = []

# plt.plot(corpus_sents_norm_df['vader'].rolling(5*win_1per).mean(), alpha=0.3, label='VADER')
if VADER_Arc:
  plt.plot(corpus_sents_norm_df['vader_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='VADER-rm')
  sentiment_noout_cols_ls.append('vader_2norm')

# plt.plot(corpus_sents_norm_df['textblob'].rolling(5*win_1per).mean(), alpha=0.3, label='TextBlob')
if TextBlob_Arc:
  plt.plot(corpus_sents_norm_df['textblob_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='TextBlob-rm')
  sentiment_noout_cols_ls.append('textblob_2norm')

# plt.plot(corpus_sents_norm_df['stanza'].rolling(5*win_1per).mean(), alpha=0.3, label='Stanza')
if Stanza_Arc:
  plt.plot(corpus_sents_norm_df['stanza_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Stanza-rm')
  sentiment_noout_cols_ls.append('stanza_2norm')

# plt.plot(corpus_sents_norm_df['afinn'].rolling(5*win_1per).mean(), alpha=0.3, label='AFINN')
if AFINN_Arc:
  plt.plot(corpus_sents_norm_df['afinn_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='AFINN-rm')
  sentiment_noout_cols_ls.append('afinn_2norm')

# plt.plot(corpus_sents_norm_df['sentimentr'].rolling(5*win_1per).mean(), alpha=0.3, label='SentimentR')
if SentimentR_Arc:
  plt.plot(corpus_sents_norm_df['sentimentr_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='SentimentR-rm')
  sentiment_noout_cols_ls.append('sentimentr_2norm')

# plt.plot(corpus_sents_norm_df['syuzhet'].rolling(5*win_1per).mean(), alpha=0.3, label='Syuzhet')
if Syuzhet_Arc:
  plt.plot(corpus_sents_norm_df['syuzhet_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Syuzhet-rm')
  sentiment_noout_cols_ls.append('syuzhet_2norm')

# plt.plot(corpus_sents_norm_df['bing'].rolling(5*win_1per).mean(), alpha=0.3, label='Bing')
if Bing_Arc:
  plt.plot(corpus_sents_norm_df['bing_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Bing-rm')
  sentiment_noout_cols_ls.append('bing_2norm')

# plt.plot(corpus_sents_norm_df['bing'].rolling(5*win_1per).mean(), alpha=0.3, label='Bing')
if Pattern_Arc:
  plt.plot(corpus_sents_norm_df['bing_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='Bing-rm')
  sentiment_noout_cols_ls.append('bing_2norm')

# plt.plot(corpus_sents_norm_df['sentiword'].rolling(5*win_1per).mean(), alpha=0.3, label='SentiWord')
if SentiWord_Arc:
  plt.plot(corpus_sents_norm_df['sentiword_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='SentiWord-rm')
  sentiment_noout_cols_ls.append('sentiword_2norm')

# plt.plot(corpus_sents_norm_df['senticnet'].rolling(5*win_1per).mean(), alpha=0.3, label='SenticNet')
if SenticNet_Arc:
  plt.plot(corpus_sents_norm_df['senticnet_2norm'].rolling(5*win_1per).mean(), alpha=0.3, label='SenticNet-rm')
  sentiment_noout_cols_ls.append('senticnet_2norm')

# plt.plot(corpus_sents_norm_df['median'].rolling(5*win_1per).mean(), alpha=0.3, label='Median')
plt.plot(corpus_sents_norm_df['median_2norm'].rolling(5*win_1per).mean(), alpha=0.6, label='Median-rm', color='black');

# Recalculated Median with Outlier(s) Removed
corpus_sents_norm_df['median_2norm_noout'] = corpus_sents_norm_df[sentiment_noout_cols_ls].median(axis=1)
plt.plot(corpus_sents_norm_df['median_2norm_noout'].rolling(5*win_1per).mean(), alpha=0.6, label='Median-rm-NoOut', color='red');



plt.title(f'{CORPUS_FULL} \n 5% SMA Sentence Sentiment - Normed Range + Mean=0 (Outliers Dropped)')
plt.legend()

if (PLOT_OUTPUT in ['All','Major']):
  # Save graph to file.
  plot_filename = 'plot_050100sma_norm_rm_sent_sa_noout.png'
  plotpathfilename_str = gen_pathfiletime(plot_filename)
  plt.savefig(plotpathfilename_str, format='png', dpi=300)
  print(f'Plot saved: {plot_filename}');

In [ ]:
%whos DataFrame

In [ ]:
corpus_sents_df.head(3)

In [ ]:
# Save all Sentiment Time Series

corpus_sents_df.to_csv('sentiment_lexruleembed_imcewan_machineslikeme_20210621.csv', index=False)
corpus_sents_norm_df.to_csv('sentiment_norm_lexruleembed_imcewan_machineslikeme_20210621.csv', index=False)

***Summary of Sentiment Datafiles Produced***

In [ ]:
%whos DataFrame

In [ ]:
print('SUMMARY OF SENTIMENT ANALYSIS DATAFILE PRODUCED:')
print('------------------------------------------------')
print(f'Raw Sentiment (+5%/10% SMA) in corpus_sents_df: {sentiment_raw_pathfilename_str}')
print('\n')
print(f'Normed Sentiment (Range/Range+Mean) in corpus_sents_norm_df: {sentiment_norm_pathfilename_str}')


In [ ]:
!ls -altr *.csv

In [ ]:
sats_raw_filename_str
sats_norm1_str
sats_norm2_str


# View Combined Sentiment Scores (Redundant?)

**Calculate MedianNorm (-Outliers) Sentiment Polarities**

**Bi/Tri-Polarity Lexicons**

NRC: 
* https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm

MPQA (Upitt):
* https://mpqa.cs.pitt.edu/lexicons/effect_lexicon/
* https://github.com/nlpcl-lab/mpqa2.0-preprocessing
* https://github.com/kvangundy/basic-sentiment-analyzer/blob/master/sentimentDict.csv

SentimentAnalysis.R (20210217 124s):
* QDAP 
* DictionaryGI: Harvard-IV dictionary asused in the General Inquirer software (2005-/1637+)
* DictionaryLM: Loughran-McDonald Financial dictionary (2355-/354+/297?)
* DictionaryHE: Henry's Financial Dictionary (85-/105+)

Custom Lexicons:
* https://nealcaren.org/lessons/wordlists/ 

Lexicons
* https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#datasets


# EDA of Raw Sentiment Polarities

In [ ]:
corpus_sents_df['vader_nltk'].plot()

**Character/Token Length Histograms**

In [ ]:
corpus_sents_df.head()
corpus_sents_df.info()

In [ ]:
# Distribution of Paragraph Lengths

sns.histplot(corpus_sents_df['sent_clean'].str.len(), kde=True).set_title(f'{BOOK_TITLE_FULL} \n Paragraph Length Histogram ({sa_model})')


In [ ]:
# Distribution of Paragraph Sentiment Scores

# sns.histplot(corpus_parags_df['vader_nltk'].rolling(5).median(), kde=True).set_title(f'{BOOK_TITLE_FULL} \n Paragraph Sentiment Histogram Rolling Win=501 ({sa_model})')


**Sentiment Histograms**

In [ ]:
corpus_sentiments_df.head()
corpus_sentiments_df.info()

In [ ]:
# Distribution of Paragraph Sentiment Scores

sns.histplot(corpus_sents_df['vader_nltk'], kde=True).set_title(f'{BOOK_TITLE_FULL} \n Paragraph Sentiment Histogram Rolling Win=501 ({sa_model})')

In [ ]:
# corpus_parags_df['vader_nltk'].lineplot()
# sns.lineplot(data=corpus_parags_df['vader_nltk'].rolling(50).median())

In [ ]:
# corpus_parags_df['vader_nltk'].lineplot()
sns.lineplot(data=corpus_sents_df['vader_nltk'].rolling(50).median())

**Histograms of Sentiment Values by Model**

In [ ]:
corpus_sents_df.info()

In [ ]:
# Histograms of Sentiment Values

# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
# df = sns.load_dataset("iris")

fig, axs = plt.subplots(4, 2, figsize=(12, 18))

sns.histplot(data=corpus_sents_df, x="median", kde=True, color="skyblue", ax=axs[0, 0])
sns.histplot(data=corpus_sents_df, x="vader_mean_roll050", kde=True, color="skyblue", ax=axs[0, 1])
sns.histplot(data=corpus_sents_df, x="sentimentr_mean_roll050", kde=True, color="olive", ax=axs[1, 0])
sns.histplot(data=corpus_sents_df, x="syuzhet_mean_roll050", kde=True, color="olive", ax=axs[1, 1])
sns.histplot(data=corpus_sents_df, x="bing_mean_roll050", kde=True, color="gold", ax=axs[2, 0])
sns.histplot(data=corpus_sents_df, x="textblob_mean_roll050", kde=True, color="gold", ax=axs[2, 1])
sns.histplot(data=corpus_sents_df, x="sentiword_mean_roll050", kde=True, color="teal", ax=axs[3, 0])
sns.histplot(data=corpus_sents_df, x="senticnet_mean_roll050", kde=True, color="teal", ax=axs[3, 1])

plt.show()

# Smooth Raw Sentiment Time Series

**Simple Moving Average (SMA) by Model**

In [ ]:
# Line Plots of Sentiment Values

# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
# df = sns.load_dataset("iris")

fig, axs = plt.subplots(4, 2, figsize=(12, 18))

sns.lineplot(data=corpus_sents_df, x="sent_no", y="median", color="skyblue", ax=axs[0, 0])
sns.lineplot(data=corpus_sents_df, x="sent_no", y="vader_mean_roll050", color="skyblue", ax=axs[0, 1])
sns.lineplot(data=corpus_sents_df, x="sent_no", y="sentimentr_mean_roll050", color="olive", ax=axs[1, 0])
sns.lineplot(data=corpus_sents_df, x="sent_no", y="syuzhet_mean_roll050", color="olive", ax=axs[1, 1])
sns.lineplot(data=corpus_sents_df, x="sent_no", y="bing_mean_roll050", color="gold", ax=axs[2, 0])
sns.lineplot(data=corpus_sents_df, x="sent_no", y="textblob_mean_roll050", color="gold", ax=axs[2, 1])
sns.lineplot(data=corpus_sents_df, x="sent_no", y="sentiword_mean_roll050", color="teal", ax=axs[3, 0])
sns.lineplot(data=corpus_sents_df, x="sent_no", y="senticnet_mean_roll050", color="teal", ax=axs[3, 1])

plt.show()

In [ ]:
corpus_sents_df.info()

In [ ]:
%whos dict

In [ ]:

# g=sns.pointplot(x=0, y=1, data=df, dodge=True,plot_kws=dict(alpha=0.3))
# plt.setp(g.collections, alpha=.3) #for the markers
# plt.setp(g.lines, alpha=.3)       #for the lines

for i, sa_model in enumerate(corpus_sents_df.columns):
  if (sa_model.endswith('_roll050')):
    # if (sa_model != 'sent_no'):
    if (sa_model == 'median'):
      sns.lineplot(data=corpus_sents_df, x='sent_no', y=sa_model, color='black')
    else:
      sns.lineplot(data=corpus_sents_df, x='sent_no', y=sa_model, alpha=0.3)

# print(f'{i}: {sa_model}')

'''
sns.lineplot(data=corpus_sentiments_df, x='sent_no', y='median')
sns.lineplot(data=corpus_sentiments_df, x='sent_no', y='vader_roll500')
sns.lineplot(data=corpus_sentiments_df, x='sent_no', y='textb')
sns.lineplot(data=corpus_sentiments_df, x='sent_no', y='vader_roll500')
sns.lineplot(data=corpus_sentiments_df, x='sent_no', y='vader_roll500')
sns.lineplot(data=corpus_sentiments_df, x='sent_no', y='vader_roll500')
sns.lineplot(data=corpus_sentiments_df, x='sent_no', y='vader_roll500')
'''

**Exponential Moving Average**

In [ ]:
# Not Necessary

**LOWESS and LOESS Smoothing**

In [ ]:
sa_model

In [ ]:
def plot_lowess(df, df_cols_ls, aplot=True, afrac=1./10, ait=5):
  '''
  Given a DataFrame, list of column to plot, LOWESS params fraction and iterations,
  Return a DataFrame with LOWESS values
  If 'plot=True', also output plot
  '''

  # global corpus_sents_norm_df

  lowess_df = pd.DataFrame()

  for i,acol in enumerate(df_cols_ls):
    sm_x, sm_y = sm_lowess(endog=df[acol].values, exog=df.index.values,  frac=afrac, it=ait, return_sorted = True).T
    col_new = f'{acol}_lowess'
    lowess_df[col_new] = pd.Series(sm_x)
    if aplot:
      plt.plot(sm_x, sm_y, label=acol, alpha=0.5, linewidth=2)

      frac_str = str(round(100*afrac))
      plt.title(f'{CORPUS_FULL} \n LOWESS (frac={frac_str} Sentence Sentiment (Model: {sa_model})')
      plt.legend(title='Sentiment Series')

  return lowess_df

# Test
new_lowess_col = f'{sa_model}_lowess'
my_frac = 1./10
my_frac_per = round(100*my_frac)
new_lowess_col = f'{sa_model}_lowess_{my_frac_per}'
corpus_sents_df[new_lowess_col] = plot_lowess(corpus_sents_df, [sa_model], afrac=my_frac)
corpus_sents_df.head()

In [ ]:
corpus_sents_norm_df.columns

In [ ]:
corpus_sents_norm_df.head(3)

In [ ]:
norm_cols_ls = []
for acol in corpus_sents_norm_df.columns:
  if acol.endswith('_2norm'):
    norm_cols_ls.append(acol)

print(f'All norm_cols_ls')

temp_cols_ls = list(set(norm_cols_ls) - set(['stanza_2norm','afinn_2norm']))

print(f'Trimmed temp_cols_ls:')
print(temp_cols_ls)


plot_lowess(corpus_sents_norm_df, temp_cols_ls, 'Normed')

'''
for i, sa_model in enumerate(corpus_sents_df.columns):
  if (sa_model.endswith('_roll050')):
    # if (sa_model != 'sent_no'):
    if (sa_model == 'median'):
      sns.lineplot(data=corpus_sents_df, x='sent_no', y=sa_model, color='black')
    else:
      sns.lineplot(data=corpus_sents_df, x='sent_no', y=sa_model, alpha=0.3)
''';

In [ ]:
%time

plt.title(f'{BOOK_TITLE_FULL} \n {sa_model} with statsmodels LOWESS (frac=0.1/iter=5)')

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['median'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
plt.plot(sm_x, sm_y, label='Median', color='black', linewidth=3)

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['vader_roll500'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
sns.lineplot(sm_x, sm_y, label='VADER', color='royalblue', alpha=0.5)

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['jockers_rinker_roll500'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
sns.lineplot(sm_x, sm_y, label='Jockers-Rinker', color='red', alpha=0.5)

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['syuzhet_roll500'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
sns.lineplot(sm_x, sm_y, label='Syuzhet', color='tomato', alpha=0.5)

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['huliu_roll500'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
sns.lineplot(sm_x, sm_y, label='HuLiu', color='teal', alpha=0.5)

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['textblob_roll500'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
sns.lineplot(sm_x, sm_y, label='TextBlob', color='lime', alpha=0.5)

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['sentiword_roll500'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
sns.lineplot(sm_x, sm_y, label='SentiWord', color='goldenrod', alpha=0.5)

sm_x, sm_y = sm_lowess(endog=corpus_sentiments_df['senticnet_roll500'].values, exog=corpus_sentiments_df.index.values,  frac=1./10., 
                       it=5, return_sorted = True).T
sns.lineplot(sm_x, sm_y, label='SenticNet', color='forestgreen', alpha=0.5)

# y_upper = corpus_sentiments_df['norm_score'].max() + 0.01
# y_lower = corpus_sentiments_df['norm_score'].min() - 0.01
# y_range = y_upper - y_lower + 0.02
# plt.ylim([0.71, 0.74])
# plt.ylim([y_lower, y_upper])
# plt.plot(x, y, 'k.');

**Discrete Cosine Transform (DCT)**

Libraries:

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.dct.html#scipy.fft.dct
* https://github.com/search?q=discrete+cosine 

Tutorial

* https://realpython.com/python-scipy-fft/#the-discrete-cosine-and-sine-transforms

In [ ]:
from sktime.transformations.series.cos import CosineTransformer
from sktime.datasets import load_airline
y = load_airline()
transformer = CosineTransformer()
y_hat = transformer.fit_transform(y)

In [ ]:
!pip install pyts

**Stanford ASAP: Automatic Smoothing for Attention Prioritization in Time Series**

* https://github.com/stanford-futuredata/ASAP/blob/master/ASAP.ipynb (Python)
* https://github.com/stanford-futuredata/ASAP/blob/master/ASAP-simple.js
* http://futuredata.stanford.edu/asap/ 
* https://www.datadoghq.com/blog/auto-smoother-asap/

In [ ]:
import scipy.stats

In [ ]:
# ASAP Simple (Brute Force)
def moving_average(data, _range):
    ret = np.cumsum(data, dtype=float)
    ret[_range:] = ret[_range:] - ret[:-_range]
    return ret[_range - 1:] / _range

def SMA(data, _range, slide):
    ret = moving_average(data, _range)[::slide]
    return list(ret)

def kurtosis(values):
    return scipy.stats.kurtosis(values)

def roughness(vals):
    return np.std(np.diff(vals))

def smooth_simple(data, max_window=5, resolution=None):
    data = np.array(data)
    # Preaggregate according to resolution
    window_size = 1
    slide_size = 1
    if resolution:
        slide_size = int(len(data) / resolution)
        if slide_size > 1:
            data = SMA(data, slide_size, slide_size)
    orig_kurt   = kurtosis(data)
    min_obj     = roughness(data)
    for w in range(2, int(len(data) / max_window + 1)):
        smoothed = SMA(data, w, 1)
        if kurtosis(smoothed) >= orig_kurt:
            r = roughness(smoothed)
            if r < min_obj:
                min_obj = r
                window_size = w
    return window_size, slide_size

In [ ]:
# Plot time series before and after smoothing
def plot(data, window_size, slide_size, plot_title):
    plt.clf()
    plt.figure()
    data = SMA(data, slide_size, slide_size)
    method_names = ["Original", "ASAP Smoothed"]
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    smoothed = SMA(data, window_size, 1)
    smoothed_range = range(int(window_size/2), int(window_size/2) + len(smoothed))
    ax1.set_xlim(0, len(data))
    ax1.plot(data, linestyle='-', linewidth=1.5)
    ax2.plot(smoothed_range, smoothed, linestyle='-', linewidth=1.5)
    axes = [ax1, ax2]
    for i in range(2):
        axes[i].get_xaxis().set_visible(False)
        axes[i].text(0.02, 0.8, "%s" %(method_names[i]),
            verticalalignment='center', horizontalalignment='left',
            transform=axes[i].transAxes, fontsize=25)

    fig.set_size_inches(16, 12)
    plt.tight_layout(w_pad=1)
    plt.title(plot_title)
    plt.show()

In [ ]:
corpus_sentiments_df.head()

In [ ]:
# Taxi
# raw_data = load_csv('Taxi.csv')
# window_size, slide_size = smooth_ASAP(raw_data, resolution=1000)
# window_size, slide_size = smooth_ASAP(raw_data, resolution=1000)
window_size, slide_size = smooth_simple(list(corpus_sentiments_df['median']), resolution=1000)
print("Window Size: ", window_size)
plot_title = f'{BOOK_TITLE_FULL} \n Median Sentiment Smoothed with ASAP from Stanford (res=1000)'
plot(list(corpus_sentiments_df['median']), window_size, slide_size, plot_title)

# Calculate Error Metrics based on Distance from Median

Libraries:

* https://github.com/wannesm/dtaidistance



In [ ]:
# sentiment_lowess_df['median']
sentiment_lowess_df.tail()

In [ ]:
# Rank each Sentiment Model by Error/Distance Metrics from Median

sentiment_lowess_df['min'] = sentiment_lowess_df[['vader','jockers-rinker','syuzhet','huliu','textblob','sentiword','senticnet']].min(axis=1)
sentiment_lowess_df['max'] = sentiment_lowess_df[['vader','jockers-rinker','syuzhet','huliu','textblob','sentiword','senticnet']].max(axis=1)
sentiment_lowess_df.head()

In [ ]:
# LOWESS Smoothed Median Curve with Min/Max Confidence Intervals

plt.title(f'{BOOK_TITLE_FULL} \n LOWESS Smoothed Median Sentiment Curve with Min/Max Confidence Intervals')

sns.lineplot(data=sentiment_lowess_df, x='x_value', y='median', linewidth=3, color='black')
plt.fill_between(sentiment_lowess_df['x_value'], sentiment_lowess_df['min'], sentiment_lowess_df['max'], alpha=.3)

In [ ]:
# Error Metrics of each Model relative to the Median

# Group and Classify Sentiment Arcs

Libraries

* https://github.com/alan-turing-institute/sktime

* https://github.com/johannfaouzi/pyts 

Code

* https://colab.research.google.com/drive/1oEFfK5KTJyFQGs2xunc1cDW2OcbkZKCY



In [ ]:
# https://colab.research.google.com/drive/1oEFfK5KTJyFQGs2xunc1cDW2OcbkZKCY

# https://github.com/johannfaouzi/pyts 


# Automatically Extract Cruxes and Key Features

**Find Peaks and Valleys**

In [ ]:
# Pick a Sentiment Time Series (default Median)

crux_models_ls = ['median', 'jockers-rinker', 'syuzhet', 'huliu', 'textblob', 'sentiword', 'senticnet']

crux_model = crux_models_ls[0] # Edit

print(f'Working with Sentiment Time Series from model: {crux_model}')

In [ ]:
# Verify

sentiment_lowess_df.head()
sm_x = sentiment_lowess_df['x_value'].values
sm_y = sentiment_lowess_df[crux_model].values
sm_y

In [ ]:
sentiment_lowess_df.shape

In [ ]:
print(peak_x)

In [ ]:
!pip install adjustText

In [ ]:
from adjustText import adjust_text

In [ ]:
# Find peaks(max).

peak_indexes = signal.argrelextrema(sm_y, np.greater)
peak_indexes = peak_indexes[0]
 
# Find valleys(min).
valley_indexes = signal.argrelextrema(sm_y, np.less)
valley_indexes = valley_indexes[0]
 
# Plot main graph.
(fig, ax) = plt.subplots()
ax.plot(sm_x, sm_y)

win_half = 0 # 2500

# Plot peaks.
peak_x = peak_indexes
peak_y = sm_y[peak_indexes]
# ax.plot(peak_x + win_half, peak_y, marker='o', linestyle='none', color='green', label="Peaks")
ax.scatter(peak_x, peak_y)
for i, txt in enumerate(list(peak_x)):
    ax.annotate(f'Sent No.\n   {txt}', (peak_x[i], peak_y[i]))


# Plot valleys.
valley_x = valley_indexes
valley_y = sm_y[valley_indexes]
# ax.plot(valley_x + win_half, valley_y, marker='o', linestyle='none', color='red', label="Valleys")
ax.scatter(valley_x, valley_y)
# for i, txt in enumerate(list(valley_x)):
#     ax.annotate(f'\n\n\nSent No.\n   {txt}', (valley_x[i], valley_y[i]))

# plt.plot(x, y, 'bo')
texts = [plt.text(valley_x[i], valley_y[i], 'Sent No.\n   %s' %valley_x[i], ha='right', va='top') for i in range(len(valley_x))]
adjust_text(texts)

# Confidence Interval (Min/Max Range)
plt.fill_between(sentiment_lowess_df['x_value'], sentiment_lowess_df['min'], sentiment_lowess_df['max'], alpha=.3, color='lightskyblue')
 
 
# Save graph to file.
plt.title(f'{BOOK_TITLE_FULL} \n LOWESS Smoothed Median Sentiment Curve with Crux Points via SciPy.argrelextrema')
plt.legend(loc='best')
plt.savefig('argrelextrema.png')


In [ ]:
type(peaks_ls)

In [ ]:
# Extract Surrounding Paragraphs for context on matching Sentences

def get_parag4sentno(asent_no):
  '''
  Return the original raw paragraph containing a 
  given sentence number.
  '''
  # parag_df = pd.DataFrame()
  # print(f'Passed in sent_no: {asent_no}')
  aparag_no = int(corpus_sents_df.loc[corpus_sents_df['sent_no'] == asent_no]['parag_no'])
  # print(f'  This sent_no {asent_no} is in parag_no: {aparag_no}')
  aparag_str = corpus_sents_df.loc[corpus_sents_df['parag_no'] == aparag_no]['sent_raw'].str.cat() # ['sent_clean']
  # sentno_parag_df = corpus_sents_df[corpus_sents_df['sent_no']==asent_no]
  # print(f'Sent #{asent_no} is in the paragraph: ')
  # print(aparag)
  # print(f'returning aparag_no: [{aparag_no}]: {aparag}')
  return aparag_no, aparag_str

'''
# Testing
asent_no = 7
print(f'Searching for paragraph containing Sentence #{asent_no}')

aparag_no, aparag_str = get_parag4sentno(asent_no)
print(f'\n  Found in Paragraph #{aparag_no} \n\n{aparag_str}')
'''

match_sents_ls = list(match_sents_df.sent_no.values)
match_sents_ls

print('\nEmtional Valence Peaks\n')
print('------------------------------\n')
peaks_ls = list(peak_indexes)
for i, asent_no in enumerate(peaks_ls):
  aparag_no, aparag_str = get_parag4sentno(asent_no)
  print(f'Peak at Sentence No. {asent_no} is in Paragraph No. {aparag_no}')
  print('----------------------------')
  print(f'     {aparag_str}\n\n')


In [ ]:
# Test to see if sentiment values need to be normalized
#   or adjusted for imbalance (e.g. VADER too many 0 polarities)

corpus_cruxes_aug_dt

In [ ]:
corpus_cruxes_dt['vader_nltk_norm'] = corpus_cruxes_dt['vader_nltk']*corpus_cruxes_dt['token_len']
corpus_cruxes_dt.head()

In [ ]:
# corpus_parags_df['vader_nltk'].lineplot()
sns.lineplot(data=corpus_parags_df['vader_nltk_norm'].rolling(50).median())

# Export Manual and Automatic Sentiment Polarities and Plots